**O QUE É A RAIS?**

A RAIS é uma base de dados estatística gerada a partir da declaração da RAIS (Relação Anual de Informações Sociais) instituída para auxiliar a gestão governamental do setor do trabalho com base legal nos Decreto nº 76.900, de 23 de dezembro de 1975 e nº 10.854, de 10 de novembro de 2021. A partir da coleta, o governo visa:

1. o suprimento às necessidades de controle da atividade trabalhista no País,
2. o provimento de dados para a elaboração de estatísticas do trabalho,
3. a disponibilização de informações do mercado de trabalho às entidades governamentais,
4. embasar estudos técnicos de natureza estatística e atuarial;
5. suprir outras necessidades relacionadas ao controle de registros do FGTS, dos Sistemas de Arrecadação e de Concessão e Benefícios Previdenciários, e da identificação do trabalhador com direito ao abono salarial PIS/PASEP (APENAS RAIS IDENTIFICADA).

Atualmente, duas bases de dados são disponibilizadas anualmente de forma gratuita pela internet, a RAIS Trabalhadores (também conhecida como RAIS Vínculos) e a RAIS Estabelecimentos (período de 1985 - ano atual).

A base RAIS Trabalhadores é organizada em nível do vínculo, contém todos os vínculos declarados (ativos e não ativos em 31/12). Abaixo algumas informações que podem ser obtidas a partir do uso desta base:

Número de empregados em 31 de dezembro, segundo faixa etária, escolaridade e gênero por nível geográfico, setorial e ocupacional;
Número de empregados por tamanho de estabelecimento, segundo setor de atividade econômica;
Remuneração média dos empregos em 31 de dezembro, segundo ocupação e setor de atividade econômica por nível geográfico.

A base RAIS Estabelecimentos é organizada em nível do estabelecimento. Contém tanto os estabelecimentos com vínculo declarado quanto os estabelecimentos sem vínculo informado no exercício (RAIS negativa). Abaixo informações que podem ser obtidas a partir desta base:

Freqüência (quantidade) de estabelecimentos declarantes
Informações do Estoque (quantidade de empregos em 31/12)
Nível geográfico;
Nível setorial;
Natureza jurídica;
Tamanho do estabelecimento;
Indicador de RAIS Negativa.

Fonte das informações: http://acesso.mte.gov.br/portal-pdet/o-pdet/portifolio-de-produtos/bases-de-dados.htm e http://www.rais.gov.br/sitio/sobre.jsf   

**O QUE ESSE CÓDIGO FAZ?**

Esse código faz o download, tratamento e o carregamento dos dados da RAIS estabelecimentos para o mongodb e para o postgresql no Dbeaver.  


# **1. IMPORTANDO LIBS**:

In [2]:
print("Importando bibliotecas necessárias...")

import os #manipulação de arquivos/diretórios
from os import walk #manipulação de arquivos/diretórios
from ftplib import FTP #conexão ftp
import requests #requisições na internet
import requests_ftp #requisições ftp
import py7zr #manipulação de arquivos zipados com 7zip
import pandas as pd #manipulação de dados no python 
import numpy as np #manipulação de dados num.
from datetime import date #manipulação de dados do tipo data/hora
from datetime import datetime #manipulação de dados do tipo data/hora
#import datetime #manipulação de dados do tipo data/hora
import pyarrow #manipulação de arquivos em parquet
from ydata_profiling import ProfileReport #gerar report para análise exploratória
import psycopg2 as p
import pymongo #fazer conexão com mongodb
import json
import urllib.parse #colocar senha mongodb
from psycopg2 import sql
from psycopg2 import extras
import json

print("\nBibliotecas importadas, bora para a execução do script!!")

Importando bibliotecas necessárias...

Bibliotecas importadas, bora para a execução do script!!


# **2. DEFININDO DIR. P/ ARMAZENAMENTO TEMPORÁRIO DOS DADOS**:

In [ ]:
base = input('Insira o caminho no qual os dados serão temporariamente armazenados antes de os enviarmos ao nosso banco de dados Mongodb. A título de exemplo, veja como está no computador do Marcilio: "E:\\...\\RAIS\\ESTABELECIMENTOS\\04_temp_data"\n')
print("\nObrigado! Os arquivos serão armazenados temporariamente neste diretório da sua máquina: ", base, "\n")
caminho_reports = input('Agora insira o caminho onde você irá armazenar os relatórios de Pd Profilling, se necessário. No caso do Marcilio, ficou da seguinte forma: "E:\\...\\RAIS\\ESTABELECIMENTOS\\04_temp_data\\0401_data_reports"')
print("\nObrigado! Os relatórios serão armazenados temporariamente neste diretório da sua máquina: ", caminho_reports, "\n")

# **3. SETANDO FTP**:

In [4]:
# Portal principal da RAIS
ftp = FTP("ftp.mtps.gov.br")  ## SERVIDOR FTP
link = r"ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/"

In [5]:
# Comando que conecta a aba correta dentro do site ftp
cmd = r"pdet/microdados/RAIS/"
print(cmd) 

pdet/microdados/RAIS/


In [6]:
# Iniciando sessão no ftp e armazenando variável de requisição
requests_ftp.monkeypatch_session()
s = requests.Session()

In [8]:
# Conferindo se os diretórios dos anos escolhidos para download do ftp existem
anos = []
while True:
    try:
        ano_inicial = int(input('Digite o ANO INICIAL para o qual você deseja fazer o ETL da RAIS: '))
        ano_final = int(input('Digite o ÚLTIMO ANO para o qual você deseja fazer o ETL da RAIS: '))
        if len(str(ano_inicial)) == 4 and str(ano_inicial).startswith("20") and len(str(ano_final)) == 4 and str(ano_final).startswith("20"):
            for i in range(ano_inicial, ano_final+1):
                resp = s.list(link+str(i))
                code = resp.status_code
                if code == 226:
                    anos.append(str(i))
                    print(f"Opaa, que beleza!! Encontramos o diretório do ano de {i} no ftp.")
                else:
                    print(f"Eita, não encontramos o diretório do ano de {i} no ftp :S.")
            print("\nA lista final dos anos de referência para o ETL dos dados contém os seguintes anos: ", anos)
            break
        else:
            print("Erro: Por favor, insira anos válidos com quatro dígitos começando com '20', ex: 2022.")
    except ValueError:
        print("Erro: Por favor, insira um número inteiro, ex: 2022.")

Opaa, que beleza!! Encontramos o diretório do ano de 2021 no ftp.

A lista final dos anos de referência para o ETL dos dados contém os seguintes anos:  ['2021']


# **4. CRIANDO FUNÇÕES**:

In [9]:
# Criando função pra checar se já existe o arquivo no diretório

def checadir(pasta, arquivo):
    files = []
    for filenames in walk(pasta):
        files.extend(filenames)
        break
    if arquivo in files:
        print(f"O arquivo {arquivo} já existe. Indo para o próximo!\n")
        return True
    else:
        return False

In [10]:
# Criando função para fazer o download dos arquivos
def baixar_arquivo(ftp, nome_arquivo, pasta_local):
    print("\nBaixando o arquivo do ftp...\n")
    with open(os.path.join(pasta_local, nome_arquivo), "wb") as fp:
        ftp.retrbinary("RETR " + nome_arquivo, fp.write)
    print(f'Arquivo {nome_arquivo} baixado com sucesso!\n')

In [11]:
def extrair_arquivo_7z_txt(caminho_arquivo):
    with py7zr.SevenZipFile(caminho_arquivo, mode="r") as z:
        z.extractall(path=base)
    print(f"Arquivo txt extraído do 7zip para o caminho: {caminho_arquivo} com sucesso!\n")

In [12]:
# Função para remover arquivos
def remover_arquivo_7z(caminho_arquivo):
    os.remove(caminho_arquivo)
    print(f"Arquivo 7zip removido do caminho: {caminho_arquivo} com sucesso!\n")

In [13]:
# Dicionários de padronização sebrae

# Com base na CNAE SUBCLASSE 2.0 e na Nat. Jurídica da empresa, o Sebrae adota uma classificação do que é ou não empresa (no seu entendimento)
pad_sebrae_empresa = {
"011015":"0",
"011023":"0",
"011031":"0",
"011040":"0",
"011058":"0",
"011066":"0",
"011074":"0",
"011082":"0",
"011104":"0",
"011112":"0",
"011120":"0",
"011139":"0",
"011147":"0",
"011155":"0",
"011163":"0",
"011171":"0",
"011180":"0",
"011198":"0",
"011201":"0",
"011210":"0",
"011228":"0",
"011236":"0",
"011244":"0",
"011252":"0",
"011260":"0",
"011279":"0",
"011287":"0",
"011295":"0",
"011309":"0",
"011317":"0",
"011325":"0",
"011333":"0",
"011341":"0",
"012011":"0",
"012038":"0",
"012046":"1",
"012054":"1",
"012062":"1",
"012070":"1",
"012076":"1",
"012089":"1",
"012097":"1",
"012100":"1",
"012119":"1",
"012127":"1",
"012135":"1",
"012143":"1",
"012151":"1",
"012160":"1",
"012178":"0",
"012186":"1",
"012194":"0",
"012216":"0",
"012224":"1",
"012232":"1",
"012240":"1",
"012259":"1",
"012267":"1",
"012275":"0",
"012283":"1",
"012291":"1",
"012305":"1",
"012313":"1",
"012321":"1",
"012330":"1",
"012992":"1",
"013034":"0",
"013069":"0",
"013077":"0",
"013085":"0",
"013107":"0",
"013115":"0",
"013131":"0",
"013204":"0",
"013212":"0",
"013220":"0",
"013239":"0",
"013247":"0",
"013255":"0",
"013263":"0",
"013271":"0",
"013280":"0",
"013298":"0",
"013301":"0",
"013310":"0",
"013999":"0",
"014014":"1",
"014022":"1",
"014030":"1",
"014049":"1",
"014080":"1",
"014081":"1",
"014090":"0",
"014111":"1",
"014120":"1",
"015010":"0",
"015029":"0",
"015037":"0",
"021015":"0",
"021023":"0",
"021031":"0",
"021040":"0",
"021058":"0",
"021066":"0",
"021074":"0",
"021082":"0",
"021104":"0",
"021112":"0",
"021120":"0",
"021139":"0",
"021147":"0",
"021155":"0",
"021163":"0",
"021171":"0",
"021180":"0",
"021198":"0",
"021201":"0",
"021210":"0",
"021228":"0",
"021236":"0",
"021244":"0",
"021252":"0",
"021260":"0",
"021279":"0",
"021287":"0",
"021295":"0",
"021309":"0",
"021317":"0",
"021325":"0",
"021333":"0",
"021341":"0",
"022011":"0",
"022038":"0",
"022046":"1",
"022054":"1",
"022062":"1",
"022070":"1",
"022076":"1",
"022089":"1",
"022097":"1",
"022100":"1",
"022119":"1",
"022127":"1",
"022135":"1",
"022143":"1",
"022151":"1",
"022160":"1",
"022178":"0",
"022186":"1",
"022194":"0",
"022216":"0",
"022224":"1",
"022232":"1",
"022240":"1",
"022259":"1",
"022267":"1",
"022275":"0",
"022283":"1",
"022291":"1",
"022305":"1",
"022313":"1",
"022321":"1",
"022330":"1",
"022992":"1",
"023034":"0",
"023069":"0",
"023077":"0",
"023085":"0",
"023107":"0",
"023115":"0",
"023131":"0",
"023204":"0",
"023212":"0",
"023220":"0",
"023239":"0",
"023247":"0",
"023255":"0",
"023263":"0",
"023271":"0",
"023280":"0",
"023298":"0",
"023301":"0",
"023310":"0",
"023999":"0",
"024014":"1",
"024022":"1",
"024030":"1",
"024049":"1",
"024080":"1",
"024081":"1",
"024090":"0",
"024111":"1",
"024120":"1",
"025010":"0",
"025029":"0",
"025037":"0",
"031015":"0",
"031023":"0",
"031031":"0",
"031040":"0",
"031058":"0",
"031066":"0",
"031074":"0",
"031082":"0",
"031104":"0",
"031112":"0",
"031120":"0",
"031139":"0",
"031147":"0",
"031155":"0",
"031163":"0",
"031171":"0",
"031180":"0",
"031198":"0",
"031201":"0",
"031210":"0",
"031228":"0",
"031236":"0",
"031244":"0",
"031252":"0",
"031260":"0",
"031279":"0",
"031287":"0",
"031295":"0",
"031309":"0",
"031317":"0",
"031325":"0",
"031333":"0",
"031341":"0",
"032011":"0",
"032038":"0",
"032046":"1",
"032054":"1",
"032062":"1",
"032070":"1",
"032076":"1",
"032089":"1",
"032097":"1",
"032100":"1",
"032119":"1",
"032127":"1",
"032135":"1",
"032143":"1",
"032151":"1",
"032160":"1",
"032178":"0",
"032186":"1",
"032194":"0",
"032216":"0",
"032224":"1",
"032232":"1",
"032240":"1",
"032259":"1",
"032267":"1",
"032275":"0",
"032283":"1",
"032291":"1",
"032305":"1",
"032313":"1",
"032321":"1",
"032330":"1",
"032992":"1",
"033034":"0",
"033069":"0",
"033077":"0",
"033085":"0",
"033107":"0",
"033115":"0",
"033131":"0",
"033204":"0",
"033212":"0",
"033220":"0",
"033239":"0",
"033247":"0",
"033255":"0",
"033263":"0",
"033271":"0",
"033280":"0",
"033298":"0",
"033301":"0",
"033310":"0",
"033999":"0",
"034014":"1",
"034022":"1",
"034030":"1",
"034049":"1",
"034080":"1",
"034081":"1",
"034090":"0",
"034111":"1",
"034120":"1",
"035010":"0",
"035029":"0",
"035037":"0",
"051015":"0",
"051023":"0",
"051031":"0",
"051040":"0",
"051058":"0",
"051066":"0",
"051074":"0",
"051082":"0",
"051104":"0",
"051112":"0",
"051120":"0",
"051139":"0",
"051147":"0",
"051155":"0",
"051163":"0",
"051171":"0",
"051180":"0",
"051198":"0",
"051201":"0",
"051210":"0",
"051228":"0",
"051236":"0",
"051244":"0",
"051252":"0",
"051260":"0",
"051279":"0",
"051287":"0",
"051295":"0",
"051309":"0",
"051317":"0",
"051325":"0",
"051333":"0",
"051341":"0",
"052011":"0",
"052038":"0",
"052046":"1",
"052054":"1",
"052062":"1",
"052070":"1",
"052076":"1",
"052089":"1",
"052097":"1",
"052100":"1",
"052119":"1",
"052127":"1",
"052135":"1",
"052143":"1",
"052151":"1",
"052160":"1",
"052178":"0",
"052186":"1",
"052194":"0",
"052216":"0",
"052224":"1",
"052232":"1",
"052240":"1",
"052259":"1",
"052267":"1",
"052275":"0",
"052283":"1",
"052291":"1",
"052305":"1",
"052313":"1",
"052321":"1",
"052330":"1",
"052992":"1",
"053034":"0",
"053069":"0",
"053077":"0",
"053085":"0",
"053107":"0",
"053115":"0",
"053131":"0",
"053204":"0",
"053212":"0",
"053220":"0",
"053239":"0",
"053247":"0",
"053255":"0",
"053263":"0",
"053271":"0",
"053280":"0",
"053298":"0",
"053301":"0",
"053310":"0",
"053999":"0",
"054014":"1",
"054022":"1",
"054030":"1",
"054049":"1",
"054080":"1",
"054081":"1",
"054090":"0",
"054111":"1",
"054120":"1",
"055010":"0",
"055029":"0",
"055037":"0",
"061015":"0",
"061023":"0",
"061031":"0",
"061040":"0",
"061058":"0",
"061066":"0",
"061074":"0",
"061082":"0",
"061104":"0",
"061112":"0",
"061120":"0",
"061139":"0",
"061147":"0",
"061155":"0",
"061163":"0",
"061171":"0",
"061180":"0",
"061198":"0",
"061201":"0",
"061210":"0",
"061228":"0",
"061236":"0",
"061244":"0",
"061252":"0",
"061260":"0",
"061279":"0",
"061287":"0",
"061295":"0",
"061309":"0",
"061317":"0",
"061325":"0",
"061333":"0",
"061341":"0",
"062011":"0",
"062038":"0",
"062046":"1",
"062054":"1",
"062062":"1",
"062070":"1",
"062076":"1",
"062089":"1",
"062097":"1",
"062100":"1",
"062119":"1",
"062127":"1",
"062135":"1",
"062143":"1",
"062151":"1",
"062160":"1",
"062178":"0",
"062186":"1",
"062194":"0",
"062216":"0",
"062224":"1",
"062232":"1",
"062240":"1",
"062259":"1",
"062267":"1",
"062275":"0",
"062283":"1",
"062291":"1",
"062305":"1",
"062313":"1",
"062321":"1",
"062330":"1",
"062992":"1",
"063034":"0",
"063069":"0",
"063077":"0",
"063085":"0",
"063107":"0",
"063115":"0",
"063131":"0",
"063204":"0",
"063212":"0",
"063220":"0",
"063239":"0",
"063247":"0",
"063255":"0",
"063263":"0",
"063271":"0",
"063280":"0",
"063298":"0",
"063301":"0",
"063310":"0",
"063999":"0",
"064014":"1",
"064022":"1",
"064030":"1",
"064049":"1",
"064080":"1",
"064081":"1",
"064090":"0",
"064111":"1",
"064120":"1",
"065010":"0",
"065029":"0",
"065037":"0",
"071015":"0",
"071023":"0",
"071031":"0",
"071040":"0",
"071058":"0",
"071066":"0",
"071074":"0",
"071082":"0",
"071104":"0",
"071112":"0",
"071120":"0",
"071139":"0",
"071147":"0",
"071155":"0",
"071163":"0",
"071171":"0",
"071180":"0",
"071198":"0",
"071201":"0",
"071210":"0",
"071228":"0",
"071236":"0",
"071244":"0",
"071252":"0",
"071260":"0",
"071279":"0",
"071287":"0",
"071295":"0",
"071309":"0",
"071317":"0",
"071325":"0",
"071333":"0",
"071341":"0",
"072011":"0",
"072038":"0",
"072046":"1",
"072054":"1",
"072062":"1",
"072070":"1",
"072076":"1",
"072089":"1",
"072097":"1",
"072100":"1",
"072119":"1",
"072127":"1",
"072135":"1",
"072143":"1",
"072151":"1",
"072160":"1",
"072178":"0",
"072186":"1",
"072194":"0",
"072216":"0",
"072224":"1",
"072232":"1",
"072240":"1",
"072259":"1",
"072267":"1",
"072275":"0",
"072283":"1",
"072291":"1",
"072305":"1",
"072313":"1",
"072321":"1",
"072330":"1",
"072992":"1",
"073034":"0",
"073069":"0",
"073077":"0",
"073085":"0",
"073107":"0",
"073115":"0",
"073131":"0",
"073204":"0",
"073212":"0",
"073220":"0",
"073239":"0",
"073247":"0",
"073255":"0",
"073263":"0",
"073271":"0",
"073280":"0",
"073298":"0",
"073301":"0",
"073310":"0",
"073999":"0",
"074014":"1",
"074022":"1",
"074030":"1",
"074049":"1",
"074080":"1",
"074081":"1",
"074090":"0",
"074111":"1",
"074120":"1",
"075010":"0",
"075029":"0",
"075037":"0",
"081015":"0",
"081023":"0",
"081031":"0",
"081040":"0",
"081058":"0",
"081066":"0",
"081074":"0",
"081082":"0",
"081104":"0",
"081112":"0",
"081120":"0",
"081139":"0",
"081147":"0",
"081155":"0",
"081163":"0",
"081171":"0",
"081180":"0",
"081198":"0",
"081201":"0",
"081210":"0",
"081228":"0",
"081236":"0",
"081244":"0",
"081252":"0",
"081260":"0",
"081279":"0",
"081287":"0",
"081295":"0",
"081309":"0",
"081317":"0",
"081325":"0",
"081333":"0",
"081341":"0",
"082011":"0",
"082038":"0",
"082046":"1",
"082054":"1",
"082062":"1",
"082070":"1",
"082076":"1",
"082089":"1",
"082097":"1",
"082100":"1",
"082119":"1",
"082127":"1",
"082135":"1",
"082143":"1",
"082151":"1",
"082160":"1",
"082178":"0",
"082186":"1",
"082194":"0",
"082216":"0",
"082224":"1",
"082232":"1",
"082240":"1",
"082259":"1",
"082267":"1",
"082275":"0",
"082283":"1",
"082291":"1",
"082305":"1",
"082313":"1",
"082321":"1",
"082330":"1",
"082992":"1",
"083034":"0",
"083069":"0",
"083077":"0",
"083085":"0",
"083107":"0",
"083115":"0",
"083131":"0",
"083204":"0",
"083212":"0",
"083220":"0",
"083239":"0",
"083247":"0",
"083255":"0",
"083263":"0",
"083271":"0",
"083280":"0",
"083298":"0",
"083301":"0",
"083310":"0",
"083999":"0",
"084014":"1",
"084022":"1",
"084030":"1",
"084049":"1",
"084080":"1",
"084081":"1",
"084090":"0",
"084111":"1",
"084120":"1",
"085010":"0",
"085029":"0",
"085037":"0",
"091015":"0",
"091023":"0",
"091031":"0",
"091040":"0",
"091058":"0",
"091066":"0",
"091074":"0",
"091082":"0",
"091104":"0",
"091112":"0",
"091120":"0",
"091139":"0",
"091147":"0",
"091155":"0",
"091163":"0",
"091171":"0",
"091180":"0",
"091198":"0",
"091201":"0",
"091210":"0",
"091228":"0",
"091236":"0",
"091244":"0",
"091252":"0",
"091260":"0",
"091279":"0",
"091287":"0",
"091295":"0",
"091309":"0",
"091317":"0",
"091325":"0",
"091333":"0",
"091341":"0",
"092011":"0",
"092038":"0",
"092046":"1",
"092054":"1",
"092062":"1",
"092070":"1",
"092076":"1",
"092089":"1",
"092097":"1",
"092100":"1",
"092119":"1",
"092127":"1",
"092135":"1",
"092143":"1",
"092151":"1",
"092160":"1",
"092178":"0",
"092186":"1",
"092194":"0",
"092216":"0",
"092224":"1",
"092232":"1",
"092240":"1",
"092259":"1",
"092267":"1",
"092275":"0",
"092283":"1",
"092291":"1",
"092305":"1",
"092313":"1",
"092321":"1",
"092330":"1",
"092992":"1",
"093034":"0",
"093069":"0",
"093077":"0",
"093085":"0",
"093107":"0",
"093115":"0",
"093131":"0",
"093204":"0",
"093212":"0",
"093220":"0",
"093239":"0",
"093247":"0",
"093255":"0",
"093263":"0",
"093271":"0",
"093280":"0",
"093298":"0",
"093301":"0",
"093310":"0",
"093999":"0",
"094014":"1",
"094022":"1",
"094030":"1",
"094049":"1",
"094080":"1",
"094081":"1",
"094090":"0",
"094111":"1",
"094120":"1",
"095010":"0",
"095029":"0",
"095037":"0",
"101015":"0",
"101023":"0",
"101031":"0",
"101040":"0",
"101058":"0",
"101066":"0",
"101074":"0",
"101082":"0",
"101104":"0",
"101112":"0",
"101120":"0",
"101139":"0",
"101147":"0",
"101155":"0",
"101163":"0",
"101171":"0",
"101180":"0",
"101198":"0",
"101201":"0",
"101210":"0",
"101228":"0",
"101236":"0",
"101244":"0",
"101252":"0",
"101260":"0",
"101279":"0",
"101287":"0",
"101295":"0",
"101309":"0",
"101317":"0",
"101325":"0",
"101333":"0",
"101341":"0",
"102011":"0",
"102038":"0",
"102046":"1",
"102054":"1",
"102062":"1",
"102070":"1",
"102076":"1",
"102089":"1",
"102097":"1",
"102100":"1",
"102119":"1",
"102127":"1",
"102135":"1",
"102143":"1",
"102151":"1",
"102160":"1",
"102178":"0",
"102186":"1",
"102194":"0",
"102216":"0",
"102224":"1",
"102232":"1",
"102240":"1",
"102259":"1",
"102267":"1",
"102275":"0",
"102283":"1",
"102291":"1",
"102305":"1",
"102313":"1",
"102321":"1",
"102330":"1",
"102992":"1",
"103034":"0",
"103069":"0",
"103077":"0",
"103085":"0",
"103107":"0",
"103115":"0",
"103131":"0",
"103204":"0",
"103212":"0",
"103220":"0",
"103239":"0",
"103247":"0",
"103255":"0",
"103263":"0",
"103271":"0",
"103280":"0",
"103298":"0",
"103301":"0",
"103310":"0",
"103999":"0",
"104014":"1",
"104022":"1",
"104030":"1",
"104049":"1",
"104080":"1",
"104081":"1",
"104090":"0",
"104111":"1",
"104120":"1",
"105010":"0",
"105029":"0",
"105037":"0",
"111015":"0",
"111023":"0",
"111031":"0",
"111040":"0",
"111058":"0",
"111066":"0",
"111074":"0",
"111082":"0",
"111104":"0",
"111112":"0",
"111120":"0",
"111139":"0",
"111147":"0",
"111155":"0",
"111163":"0",
"111171":"0",
"111180":"0",
"111198":"0",
"111201":"0",
"111210":"0",
"111228":"0",
"111236":"0",
"111244":"0",
"111252":"0",
"111260":"0",
"111279":"0",
"111287":"0",
"111295":"0",
"111309":"0",
"111317":"0",
"111325":"0",
"111333":"0",
"111341":"0",
"112011":"0",
"112038":"0",
"112046":"1",
"112054":"1",
"112062":"1",
"112070":"1",
"112076":"1",
"112089":"1",
"112097":"1",
"112100":"1",
"112119":"1",
"112127":"1",
"112135":"1",
"112143":"1",
"112151":"1",
"112160":"1",
"112178":"0",
"112186":"1",
"112194":"0",
"112216":"0",
"112224":"1",
"112232":"1",
"112240":"1",
"112259":"1",
"112267":"1",
"112275":"0",
"112283":"1",
"112291":"1",
"112305":"1",
"112313":"1",
"112321":"1",
"112330":"1",
"112992":"1",
"113034":"0",
"113069":"0",
"113077":"0",
"113085":"0",
"113107":"0",
"113115":"0",
"113131":"0",
"113204":"0",
"113212":"0",
"113220":"0",
"113239":"0",
"113247":"0",
"113255":"0",
"113263":"0",
"113271":"0",
"113280":"0",
"113298":"0",
"113301":"0",
"113310":"0",
"113999":"0",
"114014":"1",
"114022":"1",
"114030":"1",
"114049":"1",
"114080":"1",
"114081":"1",
"114090":"0",
"114111":"1",
"114120":"1",
"115010":"0",
"115029":"0",
"115037":"0",
"121015":"0",
"121023":"0",
"121031":"0",
"121040":"0",
"121058":"0",
"121066":"0",
"121074":"0",
"121082":"0",
"121104":"0",
"121112":"0",
"121120":"0",
"121139":"0",
"121147":"0",
"121155":"0",
"121163":"0",
"121171":"0",
"121180":"0",
"121198":"0",
"121201":"0",
"121210":"0",
"121228":"0",
"121236":"0",
"121244":"0",
"121252":"0",
"121260":"0",
"121279":"0",
"121287":"0",
"121295":"0",
"121309":"0",
"121317":"0",
"121325":"0",
"121333":"0",
"121341":"0",
"122011":"0",
"122038":"0",
"122046":"1",
"122054":"1",
"122062":"1",
"122070":"1",
"122076":"1",
"122089":"1",
"122097":"1",
"122100":"1",
"122119":"1",
"122127":"1",
"122135":"1",
"122143":"1",
"122151":"1",
"122160":"1",
"122178":"0",
"122186":"1",
"122194":"0",
"122216":"0",
"122224":"1",
"122232":"1",
"122240":"1",
"122259":"1",
"122267":"1",
"122275":"0",
"122283":"1",
"122291":"1",
"122305":"1",
"122313":"1",
"122321":"1",
"122330":"1",
"122992":"1",
"123034":"0",
"123069":"0",
"123077":"0",
"123085":"0",
"123107":"0",
"123115":"0",
"123131":"0",
"123204":"0",
"123212":"0",
"123220":"0",
"123239":"0",
"123247":"0",
"123255":"0",
"123263":"0",
"123271":"0",
"123280":"0",
"123298":"0",
"123301":"0",
"123310":"0",
"123999":"0",
"124014":"1",
"124022":"1",
"124030":"1",
"124049":"1",
"124080":"1",
"124081":"1",
"124090":"0",
"124111":"1",
"124120":"1",
"125010":"0",
"125029":"0",
"125037":"0",
"131015":"0",
"131023":"0",
"131031":"0",
"131040":"0",
"131058":"0",
"131066":"0",
"131074":"0",
"131082":"0",
"131104":"0",
"131112":"0",
"131120":"0",
"131139":"0",
"131147":"0",
"131155":"0",
"131163":"0",
"131171":"0",
"131180":"0",
"131198":"0",
"131201":"0",
"131210":"0",
"131228":"0",
"131236":"0",
"131244":"0",
"131252":"0",
"131260":"0",
"131279":"0",
"131287":"0",
"131295":"0",
"131309":"0",
"131317":"0",
"131325":"0",
"131333":"0",
"131341":"0",
"132011":"0",
"132038":"0",
"132046":"1",
"132054":"1",
"132062":"1",
"132070":"1",
"132076":"1",
"132089":"1",
"132097":"1",
"132100":"1",
"132119":"1",
"132127":"1",
"132135":"1",
"132143":"1",
"132151":"1",
"132160":"1",
"132178":"0",
"132186":"1",
"132194":"0",
"132216":"0",
"132224":"1",
"132232":"1",
"132240":"1",
"132259":"1",
"132267":"1",
"132275":"0",
"132283":"1",
"132291":"1",
"132305":"1",
"132313":"1",
"132321":"1",
"132330":"1",
"132992":"1",
"133034":"0",
"133069":"0",
"133077":"0",
"133085":"0",
"133107":"0",
"133115":"0",
"133131":"0",
"133204":"0",
"133212":"0",
"133220":"0",
"133239":"0",
"133247":"0",
"133255":"0",
"133263":"0",
"133271":"0",
"133280":"0",
"133298":"0",
"133301":"0",
"133310":"0",
"133999":"0",
"134014":"1",
"134022":"1",
"134030":"1",
"134049":"1",
"134080":"1",
"134081":"1",
"134090":"0",
"134111":"1",
"134120":"1",
"135010":"0",
"135029":"0",
"135037":"0",
"141015":"0",
"141023":"0",
"141031":"0",
"141040":"0",
"141058":"0",
"141066":"0",
"141074":"0",
"141082":"0",
"141104":"0",
"141112":"0",
"141120":"0",
"141139":"0",
"141147":"0",
"141155":"0",
"141163":"0",
"141171":"0",
"141180":"0",
"141198":"0",
"141201":"0",
"141210":"0",
"141228":"0",
"141236":"0",
"141244":"0",
"141252":"0",
"141260":"0",
"141279":"0",
"141287":"0",
"141295":"0",
"141309":"0",
"141317":"0",
"141325":"0",
"141333":"0",
"141341":"0",
"142011":"0",
"142038":"0",
"142046":"1",
"142054":"1",
"142062":"1",
"142070":"1",
"142076":"1",
"142089":"1",
"142097":"1",
"142100":"1",
"142119":"1",
"142127":"1",
"142135":"1",
"142143":"1",
"142151":"1",
"142160":"1",
"142178":"0",
"142186":"1",
"142194":"0",
"142216":"0",
"142224":"1",
"142232":"1",
"142240":"1",
"142259":"1",
"142267":"1",
"142275":"0",
"142283":"1",
"142291":"1",
"142305":"1",
"142313":"1",
"142321":"1",
"142330":"1",
"142992":"1",
"143034":"0",
"143069":"0",
"143077":"0",
"143085":"0",
"143107":"0",
"143115":"0",
"143131":"0",
"143204":"0",
"143212":"0",
"143220":"0",
"143239":"0",
"143247":"0",
"143255":"0",
"143263":"0",
"143271":"0",
"143280":"0",
"143298":"0",
"143301":"0",
"143310":"0",
"143999":"0",
"144014":"1",
"144022":"1",
"144030":"1",
"144049":"1",
"144080":"1",
"144081":"1",
"144090":"0",
"144111":"1",
"144120":"1",
"145010":"0",
"145029":"0",
"145037":"0",
"151015":"0",
"151023":"0",
"151031":"0",
"151040":"0",
"151058":"0",
"151066":"0",
"151074":"0",
"151082":"0",
"151104":"0",
"151112":"0",
"151120":"0",
"151139":"0",
"151147":"0",
"151155":"0",
"151163":"0",
"151171":"0",
"151180":"0",
"151198":"0",
"151201":"0",
"151210":"0",
"151228":"0",
"151236":"0",
"151244":"0",
"151252":"0",
"151260":"0",
"151279":"0",
"151287":"0",
"151295":"0",
"151309":"0",
"151317":"0",
"151325":"0",
"151333":"0",
"151341":"0",
"152011":"0",
"152038":"0",
"152046":"1",
"152054":"1",
"152062":"1",
"152070":"1",
"152076":"1",
"152089":"1",
"152097":"1",
"152100":"1",
"152119":"1",
"152127":"1",
"152135":"1",
"152143":"1",
"152151":"1",
"152160":"1",
"152178":"0",
"152186":"1",
"152194":"0",
"152216":"0",
"152224":"1",
"152232":"1",
"152240":"1",
"152259":"1",
"152267":"1",
"152275":"0",
"152283":"1",
"152291":"1",
"152305":"1",
"152313":"1",
"152321":"1",
"152330":"1",
"152992":"1",
"153034":"0",
"153069":"0",
"153077":"0",
"153085":"0",
"153107":"0",
"153115":"0",
"153131":"0",
"153204":"0",
"153212":"0",
"153220":"0",
"153239":"0",
"153247":"0",
"153255":"0",
"153263":"0",
"153271":"0",
"153280":"0",
"153298":"0",
"153301":"0",
"153310":"0",
"153999":"0",
"154014":"1",
"154022":"1",
"154030":"1",
"154049":"1",
"154080":"1",
"154081":"1",
"154090":"0",
"154111":"1",
"154120":"1",
"155010":"0",
"155029":"0",
"155037":"0",
"161015":"0",
"161023":"0",
"161031":"0",
"161040":"0",
"161058":"0",
"161066":"0",
"161074":"0",
"161082":"0",
"161104":"0",
"161112":"0",
"161120":"0",
"161139":"0",
"161147":"0",
"161155":"0",
"161163":"0",
"161171":"0",
"161180":"0",
"161198":"0",
"161201":"0",
"161210":"0",
"161228":"0",
"161236":"0",
"161244":"0",
"161252":"0",
"161260":"0",
"161279":"0",
"161287":"0",
"161295":"0",
"161309":"0",
"161317":"0",
"161325":"0",
"161333":"0",
"161341":"0",
"162011":"0",
"162038":"0",
"162046":"1",
"162054":"1",
"162062":"1",
"162070":"1",
"162076":"1",
"162089":"1",
"162097":"1",
"162100":"1",
"162119":"1",
"162127":"1",
"162135":"1",
"162143":"1",
"162151":"1",
"162160":"1",
"162178":"0",
"162186":"1",
"162194":"0",
"162216":"0",
"162224":"1",
"162232":"1",
"162240":"1",
"162259":"1",
"162267":"1",
"162275":"0",
"162283":"1",
"162291":"1",
"162305":"1",
"162313":"1",
"162321":"1",
"162330":"1",
"162992":"1",
"163034":"0",
"163069":"0",
"163077":"0",
"163085":"0",
"163107":"0",
"163115":"0",
"163131":"0",
"163204":"0",
"163212":"0",
"163220":"0",
"163239":"0",
"163247":"0",
"163255":"0",
"163263":"0",
"163271":"0",
"163280":"0",
"163298":"0",
"163301":"0",
"163310":"0",
"163999":"0",
"164014":"1",
"164022":"1",
"164030":"1",
"164049":"1",
"164080":"1",
"164081":"1",
"164090":"0",
"164111":"1",
"164120":"1",
"165010":"0",
"165029":"0",
"165037":"0",
"171015":"0",
"171023":"0",
"171031":"0",
"171040":"0",
"171058":"0",
"171066":"0",
"171074":"0",
"171082":"0",
"171104":"0",
"171112":"0",
"171120":"0",
"171139":"0",
"171147":"0",
"171155":"0",
"171163":"0",
"171171":"0",
"171180":"0",
"171198":"0",
"171201":"0",
"171210":"0",
"171228":"0",
"171236":"0",
"171244":"0",
"171252":"0",
"171260":"0",
"171279":"0",
"171287":"0",
"171295":"0",
"171309":"0",
"171317":"0",
"171325":"0",
"171333":"0",
"171341":"0",
"172011":"0",
"172038":"0",
"172046":"1",
"172054":"1",
"172062":"1",
"172070":"1",
"172076":"1",
"172089":"1",
"172097":"1",
"172100":"1",
"172119":"1",
"172127":"1",
"172135":"1",
"172143":"1",
"172151":"1",
"172160":"1",
"172178":"0",
"172186":"1",
"172194":"0",
"172216":"0",
"172224":"1",
"172232":"1",
"172240":"1",
"172259":"1",
"172267":"1",
"172275":"0",
"172283":"1",
"172291":"1",
"172305":"1",
"172313":"1",
"172321":"1",
"172330":"1",
"172992":"1",
"173034":"0",
"173069":"0",
"173077":"0",
"173085":"0",
"173107":"0",
"173115":"0",
"173131":"0",
"173204":"0",
"173212":"0",
"173220":"0",
"173239":"0",
"173247":"0",
"173255":"0",
"173263":"0",
"173271":"0",
"173280":"0",
"173298":"0",
"173301":"0",
"173310":"0",
"173999":"0",
"174014":"1",
"174022":"1",
"174030":"1",
"174049":"1",
"174080":"1",
"174081":"1",
"174090":"0",
"174111":"1",
"174120":"1",
"175010":"0",
"175029":"0",
"175037":"0",
"181015":"0",
"181023":"0",
"181031":"0",
"181040":"0",
"181058":"0",
"181066":"0",
"181074":"0",
"181082":"0",
"181104":"0",
"181112":"0",
"181120":"0",
"181139":"0",
"181147":"0",
"181155":"0",
"181163":"0",
"181171":"0",
"181180":"0",
"181198":"0",
"181201":"0",
"181210":"0",
"181228":"0",
"181236":"0",
"181244":"0",
"181252":"0",
"181260":"0",
"181279":"0",
"181287":"0",
"181295":"0",
"181309":"0",
"181317":"0",
"181325":"0",
"181333":"0",
"181341":"0",
"182011":"0",
"182038":"0",
"182046":"1",
"182054":"1",
"182062":"1",
"182070":"1",
"182076":"1",
"182089":"1",
"182097":"1",
"182100":"1",
"182119":"1",
"182127":"1",
"182135":"1",
"182143":"1",
"182151":"1",
"182160":"1",
"182178":"0",
"182186":"1",
"182194":"0",
"182216":"0",
"182224":"1",
"182232":"1",
"182240":"1",
"182259":"1",
"182267":"1",
"182275":"0",
"182283":"1",
"182291":"1",
"182305":"1",
"182313":"1",
"182321":"1",
"182330":"1",
"182992":"1",
"183034":"0",
"183069":"0",
"183077":"0",
"183085":"0",
"183107":"0",
"183115":"0",
"183131":"0",
"183204":"0",
"183212":"0",
"183220":"0",
"183239":"0",
"183247":"0",
"183255":"0",
"183263":"0",
"183271":"0",
"183280":"0",
"183298":"0",
"183301":"0",
"183310":"0",
"183999":"0",
"184014":"1",
"184022":"1",
"184030":"1",
"184049":"1",
"184080":"1",
"184081":"1",
"184090":"0",
"184111":"1",
"184120":"1",
"185010":"0",
"185029":"0",
"185037":"0",
"191015":"0",
"191023":"0",
"191031":"0",
"191040":"0",
"191058":"0",
"191066":"0",
"191074":"0",
"191082":"0",
"191104":"0",
"191112":"0",
"191120":"0",
"191139":"0",
"191147":"0",
"191155":"0",
"191163":"0",
"191171":"0",
"191180":"0",
"191198":"0",
"191201":"0",
"191210":"0",
"191228":"0",
"191236":"0",
"191244":"0",
"191252":"0",
"191260":"0",
"191279":"0",
"191287":"0",
"191295":"0",
"191309":"0",
"191317":"0",
"191325":"0",
"191333":"0",
"191341":"0",
"192011":"0",
"192038":"0",
"192046":"1",
"192054":"1",
"192062":"1",
"192070":"1",
"192076":"1",
"192089":"1",
"192097":"1",
"192100":"1",
"192119":"1",
"192127":"1",
"192135":"1",
"192143":"1",
"192151":"1",
"192160":"1",
"192178":"0",
"192186":"1",
"192194":"0",
"192216":"0",
"192224":"1",
"192232":"1",
"192240":"1",
"192259":"1",
"192267":"1",
"192275":"0",
"192283":"1",
"192291":"1",
"192305":"1",
"192313":"1",
"192321":"1",
"192330":"1",
"192992":"1",
"193034":"0",
"193069":"0",
"193077":"0",
"193085":"0",
"193107":"0",
"193115":"0",
"193131":"0",
"193204":"0",
"193212":"0",
"193220":"0",
"193239":"0",
"193247":"0",
"193255":"0",
"193263":"0",
"193271":"0",
"193280":"0",
"193298":"0",
"193301":"0",
"193310":"0",
"193999":"0",
"194014":"1",
"194022":"1",
"194030":"1",
"194049":"1",
"194080":"1",
"194081":"1",
"194090":"0",
"194111":"1",
"194120":"1",
"195010":"0",
"195029":"0",
"195037":"0",
"201015":"0",
"201023":"0",
"201031":"0",
"201040":"0",
"201058":"0",
"201066":"0",
"201074":"0",
"201082":"0",
"201104":"0",
"201112":"0",
"201120":"0",
"201139":"0",
"201147":"0",
"201155":"0",
"201163":"0",
"201171":"0",
"201180":"0",
"201198":"0",
"201201":"0",
"201210":"0",
"201228":"0",
"201236":"0",
"201244":"0",
"201252":"0",
"201260":"0",
"201279":"0",
"201287":"0",
"201295":"0",
"201309":"0",
"201317":"0",
"201325":"0",
"201333":"0",
"201341":"0",
"202011":"0",
"202038":"0",
"202046":"1",
"202054":"1",
"202062":"1",
"202070":"1",
"202076":"1",
"202089":"1",
"202097":"1",
"202100":"1",
"202119":"1",
"202127":"1",
"202135":"1",
"202143":"1",
"202151":"1",
"202160":"1",
"202178":"0",
"202186":"1",
"202194":"0",
"202216":"0",
"202224":"1",
"202232":"1",
"202240":"1",
"202259":"1",
"202267":"1",
"202275":"0",
"202283":"1",
"202291":"1",
"202305":"1",
"202313":"1",
"202321":"1",
"202330":"1",
"202992":"1",
"203034":"0",
"203069":"0",
"203077":"0",
"203085":"0",
"203107":"0",
"203115":"0",
"203131":"0",
"203204":"0",
"203212":"0",
"203220":"0",
"203239":"0",
"203247":"0",
"203255":"0",
"203263":"0",
"203271":"0",
"203280":"0",
"203298":"0",
"203301":"0",
"203310":"0",
"203999":"0",
"204014":"1",
"204022":"1",
"204030":"1",
"204049":"1",
"204080":"1",
"204081":"1",
"204090":"0",
"204111":"1",
"204120":"1",
"205010":"0",
"205029":"0",
"205037":"0",
"211015":"0",
"211023":"0",
"211031":"0",
"211040":"0",
"211058":"0",
"211066":"0",
"211074":"0",
"211082":"0",
"211104":"0",
"211112":"0",
"211120":"0",
"211139":"0",
"211147":"0",
"211155":"0",
"211163":"0",
"211171":"0",
"211180":"0",
"211198":"0",
"211201":"0",
"211210":"0",
"211228":"0",
"211236":"0",
"211244":"0",
"211252":"0",
"211260":"0",
"211279":"0",
"211287":"0",
"211295":"0",
"211309":"0",
"211317":"0",
"211325":"0",
"211333":"0",
"211341":"0",
"212011":"0",
"212038":"0",
"212046":"1",
"212054":"1",
"212062":"1",
"212070":"1",
"212076":"1",
"212089":"1",
"212097":"1",
"212100":"1",
"212119":"1",
"212127":"1",
"212135":"1",
"212143":"1",
"212151":"1",
"212160":"1",
"212178":"0",
"212186":"1",
"212194":"0",
"212216":"0",
"212224":"1",
"212232":"1",
"212240":"1",
"212259":"1",
"212267":"1",
"212275":"0",
"212283":"1",
"212291":"1",
"212305":"1",
"212313":"1",
"212321":"1",
"212330":"1",
"212992":"1",
"213034":"0",
"213069":"0",
"213077":"0",
"213085":"0",
"213107":"0",
"213115":"0",
"213131":"0",
"213204":"0",
"213212":"0",
"213220":"0",
"213239":"0",
"213247":"0",
"213255":"0",
"213263":"0",
"213271":"0",
"213280":"0",
"213298":"0",
"213301":"0",
"213310":"0",
"213999":"0",
"214014":"1",
"214022":"1",
"214030":"1",
"214049":"1",
"214080":"1",
"214081":"1",
"214090":"0",
"214111":"1",
"214120":"1",
"215010":"0",
"215029":"0",
"215037":"0",
"221015":"0",
"221023":"0",
"221031":"0",
"221040":"0",
"221058":"0",
"221066":"0",
"221074":"0",
"221082":"0",
"221104":"0",
"221112":"0",
"221120":"0",
"221139":"0",
"221147":"0",
"221155":"0",
"221163":"0",
"221171":"0",
"221180":"0",
"221198":"0",
"221201":"0",
"221210":"0",
"221228":"0",
"221236":"0",
"221244":"0",
"221252":"0",
"221260":"0",
"221279":"0",
"221287":"0",
"221295":"0",
"221309":"0",
"221317":"0",
"221325":"0",
"221333":"0",
"221341":"0",
"222011":"0",
"222038":"0",
"222046":"1",
"222054":"1",
"222062":"1",
"222070":"1",
"222076":"1",
"222089":"1",
"222097":"1",
"222100":"1",
"222119":"1",
"222127":"1",
"222135":"1",
"222143":"1",
"222151":"1",
"222160":"1",
"222178":"0",
"222186":"1",
"222194":"0",
"222216":"0",
"222224":"1",
"222232":"1",
"222240":"1",
"222259":"1",
"222267":"1",
"222275":"0",
"222283":"1",
"222291":"1",
"222305":"1",
"222313":"1",
"222321":"1",
"222330":"1",
"222992":"1",
"223034":"0",
"223069":"0",
"223077":"0",
"223085":"0",
"223107":"0",
"223115":"0",
"223131":"0",
"223204":"0",
"223212":"0",
"223220":"0",
"223239":"0",
"223247":"0",
"223255":"0",
"223263":"0",
"223271":"0",
"223280":"0",
"223298":"0",
"223301":"0",
"223310":"0",
"223999":"0",
"224014":"1",
"224022":"1",
"224030":"1",
"224049":"1",
"224080":"1",
"224081":"1",
"224090":"0",
"224111":"1",
"224120":"1",
"225010":"0",
"225029":"0",
"225037":"0",
"231015":"0",
"231023":"0",
"231031":"0",
"231040":"0",
"231058":"0",
"231066":"0",
"231074":"0",
"231082":"0",
"231104":"0",
"231112":"0",
"231120":"0",
"231139":"0",
"231147":"0",
"231155":"0",
"231163":"0",
"231171":"0",
"231180":"0",
"231198":"0",
"231201":"0",
"231210":"0",
"231228":"0",
"231236":"0",
"231244":"0",
"231252":"0",
"231260":"0",
"231279":"0",
"231287":"0",
"231295":"0",
"231309":"0",
"231317":"0",
"231325":"0",
"231333":"0",
"231341":"0",
"232011":"0",
"232038":"0",
"232046":"1",
"232054":"1",
"232062":"1",
"232070":"1",
"232076":"1",
"232089":"1",
"232097":"1",
"232100":"1",
"232119":"1",
"232127":"1",
"232135":"1",
"232143":"1",
"232151":"1",
"232160":"1",
"232178":"0",
"232186":"1",
"232194":"0",
"232216":"0",
"232224":"1",
"232232":"1",
"232240":"1",
"232259":"1",
"232267":"1",
"232275":"0",
"232283":"1",
"232291":"1",
"232305":"1",
"232313":"1",
"232321":"1",
"232330":"1",
"232992":"1",
"233034":"0",
"233069":"0",
"233077":"0",
"233085":"0",
"233107":"0",
"233115":"0",
"233131":"0",
"233204":"0",
"233212":"0",
"233220":"0",
"233239":"0",
"233247":"0",
"233255":"0",
"233263":"0",
"233271":"0",
"233280":"0",
"233298":"0",
"233301":"0",
"233310":"0",
"233999":"0",
"234014":"1",
"234022":"1",
"234030":"1",
"234049":"1",
"234080":"1",
"234081":"1",
"234090":"0",
"234111":"1",
"234120":"1",
"235010":"0",
"235029":"0",
"235037":"0",
"241015":"0",
"241023":"0",
"241031":"0",
"241040":"0",
"241058":"0",
"241066":"0",
"241074":"0",
"241082":"0",
"241104":"0",
"241112":"0",
"241120":"0",
"241139":"0",
"241147":"0",
"241155":"0",
"241163":"0",
"241171":"0",
"241180":"0",
"241198":"0",
"241201":"0",
"241210":"0",
"241228":"0",
"241236":"0",
"241244":"0",
"241252":"0",
"241260":"0",
"241279":"0",
"241287":"0",
"241295":"0",
"241309":"0",
"241317":"0",
"241325":"0",
"241333":"0",
"241341":"0",
"242011":"0",
"242038":"0",
"242046":"1",
"242054":"1",
"242062":"1",
"242070":"1",
"242076":"1",
"242089":"1",
"242097":"1",
"242100":"1",
"242119":"1",
"242127":"1",
"242135":"1",
"242143":"1",
"242151":"1",
"242160":"1",
"242178":"0",
"242186":"1",
"242194":"0",
"242216":"0",
"242224":"1",
"242232":"1",
"242240":"1",
"242259":"1",
"242267":"1",
"242275":"0",
"242283":"1",
"242291":"1",
"242305":"1",
"242313":"1",
"242321":"1",
"242330":"1",
"242992":"1",
"243034":"0",
"243069":"0",
"243077":"0",
"243085":"0",
"243107":"0",
"243115":"0",
"243131":"0",
"243204":"0",
"243212":"0",
"243220":"0",
"243239":"0",
"243247":"0",
"243255":"0",
"243263":"0",
"243271":"0",
"243280":"0",
"243298":"0",
"243301":"0",
"243310":"0",
"243999":"0",
"244014":"1",
"244022":"1",
"244030":"1",
"244049":"1",
"244080":"1",
"244081":"1",
"244090":"0",
"244111":"1",
"244120":"1",
"245010":"0",
"245029":"0",
"245037":"0",
"251015":"0",
"251023":"0",
"251031":"0",
"251040":"0",
"251058":"0",
"251066":"0",
"251074":"0",
"251082":"0",
"251104":"0",
"251112":"0",
"251120":"0",
"251139":"0",
"251147":"0",
"251155":"0",
"251163":"0",
"251171":"0",
"251180":"0",
"251198":"0",
"251201":"0",
"251210":"0",
"251228":"0",
"251236":"0",
"251244":"0",
"251252":"0",
"251260":"0",
"251279":"0",
"251287":"0",
"251295":"0",
"251309":"0",
"251317":"0",
"251325":"0",
"251333":"0",
"251341":"0",
"252011":"0",
"252038":"0",
"252046":"1",
"252054":"1",
"252062":"1",
"252070":"1",
"252076":"1",
"252089":"1",
"252097":"1",
"252100":"1",
"252119":"1",
"252127":"1",
"252135":"1",
"252143":"1",
"252151":"1",
"252160":"1",
"252178":"0",
"252186":"1",
"252194":"0",
"252216":"0",
"252224":"1",
"252232":"1",
"252240":"1",
"252259":"1",
"252267":"1",
"252275":"0",
"252283":"1",
"252291":"1",
"252305":"1",
"252313":"1",
"252321":"1",
"252330":"1",
"252992":"1",
"253034":"0",
"253069":"0",
"253077":"0",
"253085":"0",
"253107":"0",
"253115":"0",
"253131":"0",
"253204":"0",
"253212":"0",
"253220":"0",
"253239":"0",
"253247":"0",
"253255":"0",
"253263":"0",
"253271":"0",
"253280":"0",
"253298":"0",
"253301":"0",
"253310":"0",
"253999":"0",
"254014":"1",
"254022":"1",
"254030":"1",
"254049":"1",
"254080":"1",
"254081":"1",
"254090":"0",
"254111":"1",
"254120":"1",
"255010":"0",
"255029":"0",
"255037":"0",
"261015":"0",
"261023":"0",
"261031":"0",
"261040":"0",
"261058":"0",
"261066":"0",
"261074":"0",
"261082":"0",
"261104":"0",
"261112":"0",
"261120":"0",
"261139":"0",
"261147":"0",
"261155":"0",
"261163":"0",
"261171":"0",
"261180":"0",
"261198":"0",
"261201":"0",
"261210":"0",
"261228":"0",
"261236":"0",
"261244":"0",
"261252":"0",
"261260":"0",
"261279":"0",
"261287":"0",
"261295":"0",
"261309":"0",
"261317":"0",
"261325":"0",
"261333":"0",
"261341":"0",
"262011":"0",
"262038":"0",
"262046":"1",
"262054":"1",
"262062":"1",
"262070":"1",
"262076":"1",
"262089":"1",
"262097":"1",
"262100":"1",
"262119":"1",
"262127":"1",
"262135":"1",
"262143":"1",
"262151":"1",
"262160":"1",
"262178":"0",
"262186":"1",
"262194":"0",
"262216":"0",
"262224":"1",
"262232":"1",
"262240":"1",
"262259":"1",
"262267":"1",
"262275":"0",
"262283":"1",
"262291":"1",
"262305":"1",
"262313":"1",
"262321":"1",
"262330":"1",
"262992":"1",
"263034":"0",
"263069":"0",
"263077":"0",
"263085":"0",
"263107":"0",
"263115":"0",
"263131":"0",
"263204":"0",
"263212":"0",
"263220":"0",
"263239":"0",
"263247":"0",
"263255":"0",
"263263":"0",
"263271":"0",
"263280":"0",
"263298":"0",
"263301":"0",
"263310":"0",
"263999":"0",
"264014":"1",
"264022":"1",
"264030":"1",
"264049":"1",
"264080":"1",
"264081":"1",
"264090":"0",
"264111":"1",
"264120":"1",
"265010":"0",
"265029":"0",
"265037":"0",
"271015":"0",
"271023":"0",
"271031":"0",
"271040":"0",
"271058":"0",
"271066":"0",
"271074":"0",
"271082":"0",
"271104":"0",
"271112":"0",
"271120":"0",
"271139":"0",
"271147":"0",
"271155":"0",
"271163":"0",
"271171":"0",
"271180":"0",
"271198":"0",
"271201":"0",
"271210":"0",
"271228":"0",
"271236":"0",
"271244":"0",
"271252":"0",
"271260":"0",
"271279":"0",
"271287":"0",
"271295":"0",
"271309":"0",
"271317":"0",
"271325":"0",
"271333":"0",
"271341":"0",
"272011":"0",
"272038":"0",
"272046":"1",
"272054":"1",
"272062":"1",
"272070":"1",
"272076":"1",
"272089":"1",
"272097":"1",
"272100":"1",
"272119":"1",
"272127":"1",
"272135":"1",
"272143":"1",
"272151":"1",
"272160":"1",
"272178":"0",
"272186":"1",
"272194":"0",
"272216":"0",
"272224":"1",
"272232":"1",
"272240":"1",
"272259":"1",
"272267":"1",
"272275":"0",
"272283":"1",
"272291":"1",
"272305":"1",
"272313":"1",
"272321":"1",
"272330":"1",
"272992":"1",
"273034":"0",
"273069":"0",
"273077":"0",
"273085":"0",
"273107":"0",
"273115":"0",
"273131":"0",
"273204":"0",
"273212":"0",
"273220":"0",
"273239":"0",
"273247":"0",
"273255":"0",
"273263":"0",
"273271":"0",
"273280":"0",
"273298":"0",
"273301":"0",
"273310":"0",
"273999":"0",
"274014":"1",
"274022":"1",
"274030":"1",
"274049":"1",
"274080":"1",
"274081":"1",
"274090":"0",
"274111":"1",
"274120":"1",
"275010":"0",
"275029":"0",
"275037":"0",
"281015":"0",
"281023":"0",
"281031":"0",
"281040":"0",
"281058":"0",
"281066":"0",
"281074":"0",
"281082":"0",
"281104":"0",
"281112":"0",
"281120":"0",
"281139":"0",
"281147":"0",
"281155":"0",
"281163":"0",
"281171":"0",
"281180":"0",
"281198":"0",
"281201":"0",
"281210":"0",
"281228":"0",
"281236":"0",
"281244":"0",
"281252":"0",
"281260":"0",
"281279":"0",
"281287":"0",
"281295":"0",
"281309":"0",
"281317":"0",
"281325":"0",
"281333":"0",
"281341":"0",
"282011":"0",
"282038":"0",
"282046":"1",
"282054":"1",
"282062":"1",
"282070":"1",
"282076":"1",
"282089":"1",
"282097":"1",
"282100":"1",
"282119":"1",
"282127":"1",
"282135":"1",
"282143":"1",
"282151":"1",
"282160":"1",
"282178":"0",
"282186":"1",
"282194":"0",
"282216":"0",
"282224":"1",
"282232":"1",
"282240":"1",
"282259":"1",
"282267":"1",
"282275":"0",
"282283":"1",
"282291":"1",
"282305":"1",
"282313":"1",
"282321":"1",
"282330":"1",
"282992":"1",
"283034":"0",
"283069":"0",
"283077":"0",
"283085":"0",
"283107":"0",
"283115":"0",
"283131":"0",
"283204":"0",
"283212":"0",
"283220":"0",
"283239":"0",
"283247":"0",
"283255":"0",
"283263":"0",
"283271":"0",
"283280":"0",
"283298":"0",
"283301":"0",
"283310":"0",
"283999":"0",
"284014":"1",
"284022":"1",
"284030":"1",
"284049":"1",
"284080":"1",
"284081":"1",
"284090":"0",
"284111":"1",
"284120":"1",
"285010":"0",
"285029":"0",
"285037":"0",
"291015":"0",
"291023":"0",
"291031":"0",
"291040":"0",
"291058":"0",
"291066":"0",
"291074":"0",
"291082":"0",
"291104":"0",
"291112":"0",
"291120":"0",
"291139":"0",
"291147":"0",
"291155":"0",
"291163":"0",
"291171":"0",
"291180":"0",
"291198":"0",
"291201":"0",
"291210":"0",
"291228":"0",
"291236":"0",
"291244":"0",
"291252":"0",
"291260":"0",
"291279":"0",
"291287":"0",
"291295":"0",
"291309":"0",
"291317":"0",
"291325":"0",
"291333":"0",
"291341":"0",
"292011":"0",
"292038":"0",
"292046":"1",
"292054":"1",
"292062":"1",
"292070":"1",
"292076":"1",
"292089":"1",
"292097":"1",
"292100":"1",
"292119":"1",
"292127":"1",
"292135":"1",
"292143":"1",
"292151":"1",
"292160":"1",
"292178":"0",
"292186":"1",
"292194":"0",
"292216":"0",
"292224":"1",
"292232":"1",
"292240":"1",
"292259":"1",
"292267":"1",
"292275":"0",
"292283":"1",
"292291":"1",
"292305":"1",
"292313":"1",
"292321":"1",
"292330":"1",
"292992":"1",
"293034":"0",
"293069":"0",
"293077":"0",
"293085":"0",
"293107":"0",
"293115":"0",
"293131":"0",
"293204":"0",
"293212":"0",
"293220":"0",
"293239":"0",
"293247":"0",
"293255":"0",
"293263":"0",
"293271":"0",
"293280":"0",
"293298":"0",
"293301":"0",
"293310":"0",
"293999":"0",
"294014":"1",
"294022":"1",
"294030":"1",
"294049":"1",
"294080":"1",
"294081":"1",
"294090":"0",
"294111":"1",
"294120":"1",
"295010":"0",
"295029":"0",
"295037":"0",
"301015":"0",
"301023":"0",
"301031":"0",
"301040":"0",
"301058":"0",
"301066":"0",
"301074":"0",
"301082":"0",
"301104":"0",
"301112":"0",
"301120":"0",
"301139":"0",
"301147":"0",
"301155":"0",
"301163":"0",
"301171":"0",
"301180":"0",
"301198":"0",
"301201":"0",
"301210":"0",
"301228":"0",
"301236":"0",
"301244":"0",
"301252":"0",
"301260":"0",
"301279":"0",
"301287":"0",
"301295":"0",
"301309":"0",
"301317":"0",
"301325":"0",
"301333":"0",
"301341":"0",
"302011":"0",
"302038":"0",
"302046":"1",
"302054":"1",
"302062":"1",
"302070":"1",
"302076":"1",
"302089":"1",
"302097":"1",
"302100":"1",
"302119":"1",
"302127":"1",
"302135":"1",
"302143":"1",
"302151":"1",
"302160":"1",
"302178":"0",
"302186":"1",
"302194":"0",
"302216":"0",
"302224":"1",
"302232":"1",
"302240":"1",
"302259":"1",
"302267":"1",
"302275":"0",
"302283":"1",
"302291":"1",
"302305":"1",
"302313":"1",
"302321":"1",
"302330":"1",
"302992":"1",
"303034":"0",
"303069":"0",
"303077":"0",
"303085":"0",
"303107":"0",
"303115":"0",
"303131":"0",
"303204":"0",
"303212":"0",
"303220":"0",
"303239":"0",
"303247":"0",
"303255":"0",
"303263":"0",
"303271":"0",
"303280":"0",
"303298":"0",
"303301":"0",
"303310":"0",
"303999":"0",
"304014":"1",
"304022":"1",
"304030":"1",
"304049":"1",
"304080":"1",
"304081":"1",
"304090":"0",
"304111":"1",
"304120":"1",
"305010":"0",
"305029":"0",
"305037":"0",
"311015":"0",
"311023":"0",
"311031":"0",
"311040":"0",
"311058":"0",
"311066":"0",
"311074":"0",
"311082":"0",
"311104":"0",
"311112":"0",
"311120":"0",
"311139":"0",
"311147":"0",
"311155":"0",
"311163":"0",
"311171":"0",
"311180":"0",
"311198":"0",
"311201":"0",
"311210":"0",
"311228":"0",
"311236":"0",
"311244":"0",
"311252":"0",
"311260":"0",
"311279":"0",
"311287":"0",
"311295":"0",
"311309":"0",
"311317":"0",
"311325":"0",
"311333":"0",
"311341":"0",
"312011":"0",
"312038":"0",
"312046":"1",
"312054":"1",
"312062":"1",
"312070":"1",
"312076":"1",
"312089":"1",
"312097":"1",
"312100":"1",
"312119":"1",
"312127":"1",
"312135":"1",
"312143":"1",
"312151":"1",
"312160":"1",
"312178":"0",
"312186":"1",
"312194":"0",
"312216":"0",
"312224":"1",
"312232":"1",
"312240":"1",
"312259":"1",
"312267":"1",
"312275":"0",
"312283":"1",
"312291":"1",
"312305":"1",
"312313":"1",
"312321":"1",
"312330":"1",
"312992":"1",
"313034":"0",
"313069":"0",
"313077":"0",
"313085":"0",
"313107":"0",
"313115":"0",
"313131":"0",
"313204":"0",
"313212":"0",
"313220":"0",
"313239":"0",
"313247":"0",
"313255":"0",
"313263":"0",
"313271":"0",
"313280":"0",
"313298":"0",
"313301":"0",
"313310":"0",
"313999":"0",
"314014":"1",
"314022":"1",
"314030":"1",
"314049":"1",
"314080":"1",
"314081":"1",
"314090":"0",
"314111":"1",
"314120":"1",
"315010":"0",
"315029":"0",
"315037":"0",
"321015":"0",
"321023":"0",
"321031":"0",
"321040":"0",
"321058":"0",
"321066":"0",
"321074":"0",
"321082":"0",
"321104":"0",
"321112":"0",
"321120":"0",
"321139":"0",
"321147":"0",
"321155":"0",
"321163":"0",
"321171":"0",
"321180":"0",
"321198":"0",
"321201":"0",
"321210":"0",
"321228":"0",
"321236":"0",
"321244":"0",
"321252":"0",
"321260":"0",
"321279":"0",
"321287":"0",
"321295":"0",
"321309":"0",
"321317":"0",
"321325":"0",
"321333":"0",
"321341":"0",
"322011":"0",
"322038":"0",
"322046":"1",
"322054":"1",
"322062":"1",
"322070":"1",
"322076":"1",
"322089":"1",
"322097":"1",
"322100":"1",
"322119":"1",
"322127":"1",
"322135":"1",
"322143":"1",
"322151":"1",
"322160":"1",
"322178":"0",
"322186":"1",
"322194":"0",
"322216":"0",
"322224":"1",
"322232":"1",
"322240":"1",
"322259":"1",
"322267":"1",
"322275":"0",
"322283":"1",
"322291":"1",
"322305":"1",
"322313":"1",
"322321":"1",
"322330":"1",
"322992":"1",
"323034":"0",
"323069":"0",
"323077":"0",
"323085":"0",
"323107":"0",
"323115":"0",
"323131":"0",
"323204":"0",
"323212":"0",
"323220":"0",
"323239":"0",
"323247":"0",
"323255":"0",
"323263":"0",
"323271":"0",
"323280":"0",
"323298":"0",
"323301":"0",
"323310":"0",
"323999":"0",
"324014":"1",
"324022":"1",
"324030":"1",
"324049":"1",
"324080":"1",
"324081":"1",
"324090":"0",
"324111":"1",
"324120":"1",
"325010":"0",
"325029":"0",
"325037":"0",
"331015":"0",
"331023":"0",
"331031":"0",
"331040":"0",
"331058":"0",
"331066":"0",
"331074":"0",
"331082":"0",
"331104":"0",
"331112":"0",
"331120":"0",
"331139":"0",
"331147":"0",
"331155":"0",
"331163":"0",
"331171":"0",
"331180":"0",
"331198":"0",
"331201":"0",
"331210":"0",
"331228":"0",
"331236":"0",
"331244":"0",
"331252":"0",
"331260":"0",
"331279":"0",
"331287":"0",
"331295":"0",
"331309":"0",
"331317":"0",
"331325":"0",
"331333":"0",
"331341":"0",
"332011":"0",
"332038":"0",
"332046":"1",
"332054":"1",
"332062":"1",
"332070":"1",
"332076":"1",
"332089":"1",
"332097":"1",
"332100":"1",
"332119":"1",
"332127":"1",
"332135":"1",
"332143":"1",
"332151":"1",
"332160":"1",
"332178":"0",
"332186":"1",
"332194":"0",
"332216":"0",
"332224":"1",
"332232":"1",
"332240":"1",
"332259":"1",
"332267":"1",
"332275":"0",
"332283":"1",
"332291":"1",
"332305":"1",
"332313":"1",
"332321":"1",
"332330":"1",
"332992":"1",
"333034":"0",
"333069":"0",
"333077":"0",
"333085":"0",
"333107":"0",
"333115":"0",
"333131":"0",
"333204":"0",
"333212":"0",
"333220":"0",
"333239":"0",
"333247":"0",
"333255":"0",
"333263":"0",
"333271":"0",
"333280":"0",
"333298":"0",
"333301":"0",
"333310":"0",
"333999":"0",
"334014":"1",
"334022":"1",
"334030":"1",
"334049":"1",
"334080":"1",
"334081":"1",
"334090":"0",
"334111":"1",
"334120":"1",
"335010":"0",
"335029":"0",
"335037":"0",
"351015":"0",
"351023":"0",
"351031":"0",
"351040":"0",
"351058":"0",
"351066":"0",
"351074":"0",
"351082":"0",
"351104":"0",
"351112":"0",
"351120":"0",
"351139":"0",
"351147":"0",
"351155":"0",
"351163":"0",
"351171":"0",
"351180":"0",
"351198":"0",
"351201":"0",
"351210":"0",
"351228":"0",
"351236":"0",
"351244":"0",
"351252":"0",
"351260":"0",
"351279":"0",
"351287":"0",
"351295":"0",
"351309":"0",
"351317":"0",
"351325":"0",
"351333":"0",
"351341":"0",
"352011":"0",
"352038":"0",
"352046":"1",
"352054":"1",
"352062":"1",
"352070":"1",
"352076":"1",
"352089":"1",
"352097":"1",
"352100":"1",
"352119":"1",
"352127":"1",
"352135":"1",
"352143":"1",
"352151":"1",
"352160":"1",
"352178":"0",
"352186":"1",
"352194":"0",
"352216":"0",
"352224":"1",
"352232":"1",
"352240":"1",
"352259":"1",
"352267":"1",
"352275":"0",
"352283":"1",
"352291":"1",
"352305":"1",
"352313":"1",
"352321":"1",
"352330":"1",
"352992":"1",
"353034":"0",
"353069":"0",
"353077":"0",
"353085":"0",
"353107":"0",
"353115":"0",
"353131":"0",
"353204":"0",
"353212":"0",
"353220":"0",
"353239":"0",
"353247":"0",
"353255":"0",
"353263":"0",
"353271":"0",
"353280":"0",
"353298":"0",
"353301":"0",
"353310":"0",
"353999":"0",
"354014":"1",
"354022":"1",
"354030":"1",
"354049":"1",
"354080":"1",
"354081":"1",
"354090":"0",
"354111":"1",
"354120":"1",
"355010":"0",
"355029":"0",
"355037":"0",
"361015":"0",
"361023":"0",
"361031":"0",
"361040":"0",
"361058":"0",
"361066":"0",
"361074":"0",
"361082":"0",
"361104":"0",
"361112":"0",
"361120":"0",
"361139":"0",
"361147":"0",
"361155":"0",
"361163":"0",
"361171":"0",
"361180":"0",
"361198":"0",
"361201":"0",
"361210":"0",
"361228":"0",
"361236":"0",
"361244":"0",
"361252":"0",
"361260":"0",
"361279":"0",
"361287":"0",
"361295":"0",
"361309":"0",
"361317":"0",
"361325":"0",
"361333":"0",
"361341":"0",
"362011":"0",
"362038":"0",
"362046":"1",
"362054":"1",
"362062":"1",
"362070":"1",
"362076":"1",
"362089":"1",
"362097":"1",
"362100":"1",
"362119":"1",
"362127":"1",
"362135":"1",
"362143":"1",
"362151":"1",
"362160":"1",
"362178":"0",
"362186":"1",
"362194":"0",
"362216":"0",
"362224":"1",
"362232":"1",
"362240":"1",
"362259":"1",
"362267":"1",
"362275":"0",
"362283":"1",
"362291":"1",
"362305":"1",
"362313":"1",
"362321":"1",
"362330":"1",
"362992":"1",
"363034":"0",
"363069":"0",
"363077":"0",
"363085":"0",
"363107":"0",
"363115":"0",
"363131":"0",
"363204":"0",
"363212":"0",
"363220":"0",
"363239":"0",
"363247":"0",
"363255":"0",
"363263":"0",
"363271":"0",
"363280":"0",
"363298":"0",
"363301":"0",
"363310":"0",
"363999":"0",
"364014":"1",
"364022":"1",
"364030":"1",
"364049":"1",
"364080":"1",
"364081":"1",
"364090":"0",
"364111":"1",
"364120":"1",
"365010":"0",
"365029":"0",
"365037":"0",
"371015":"0",
"371023":"0",
"371031":"0",
"371040":"0",
"371058":"0",
"371066":"0",
"371074":"0",
"371082":"0",
"371104":"0",
"371112":"0",
"371120":"0",
"371139":"0",
"371147":"0",
"371155":"0",
"371163":"0",
"371171":"0",
"371180":"0",
"371198":"0",
"371201":"0",
"371210":"0",
"371228":"0",
"371236":"0",
"371244":"0",
"371252":"0",
"371260":"0",
"371279":"0",
"371287":"0",
"371295":"0",
"371309":"0",
"371317":"0",
"371325":"0",
"371333":"0",
"371341":"0",
"372011":"0",
"372038":"0",
"372046":"1",
"372054":"1",
"372062":"1",
"372070":"1",
"372076":"1",
"372089":"1",
"372097":"1",
"372100":"1",
"372119":"1",
"372127":"1",
"372135":"1",
"372143":"1",
"372151":"1",
"372160":"1",
"372178":"0",
"372186":"1",
"372194":"0",
"372216":"0",
"372224":"1",
"372232":"1",
"372240":"1",
"372259":"1",
"372267":"1",
"372275":"0",
"372283":"1",
"372291":"1",
"372305":"1",
"372313":"1",
"372321":"1",
"372330":"1",
"372992":"1",
"373034":"0",
"373069":"0",
"373077":"0",
"373085":"0",
"373107":"0",
"373115":"0",
"373131":"0",
"373204":"0",
"373212":"0",
"373220":"0",
"373239":"0",
"373247":"0",
"373255":"0",
"373263":"0",
"373271":"0",
"373280":"0",
"373298":"0",
"373301":"0",
"373310":"0",
"373999":"0",
"374014":"1",
"374022":"1",
"374030":"1",
"374049":"1",
"374080":"1",
"374081":"1",
"374090":"0",
"374111":"1",
"374120":"1",
"375010":"0",
"375029":"0",
"375037":"0",
"381015":"0",
"381023":"0",
"381031":"0",
"381040":"0",
"381058":"0",
"381066":"0",
"381074":"0",
"381082":"0",
"381104":"0",
"381112":"0",
"381120":"0",
"381139":"0",
"381147":"0",
"381155":"0",
"381163":"0",
"381171":"0",
"381180":"0",
"381198":"0",
"381201":"0",
"381210":"0",
"381228":"0",
"381236":"0",
"381244":"0",
"381252":"0",
"381260":"0",
"381279":"0",
"381287":"0",
"381295":"0",
"381309":"0",
"381317":"0",
"381325":"0",
"381333":"0",
"381341":"0",
"382011":"0",
"382038":"0",
"382046":"1",
"382054":"1",
"382062":"1",
"382070":"1",
"382076":"1",
"382089":"1",
"382097":"1",
"382100":"1",
"382119":"1",
"382127":"1",
"382135":"1",
"382143":"1",
"382151":"1",
"382160":"1",
"382178":"0",
"382186":"1",
"382194":"0",
"382216":"0",
"382224":"1",
"382232":"1",
"382240":"1",
"382259":"1",
"382267":"1",
"382275":"0",
"382283":"1",
"382291":"1",
"382305":"1",
"382313":"1",
"382321":"1",
"382330":"1",
"382992":"1",
"383034":"0",
"383069":"0",
"383077":"0",
"383085":"0",
"383107":"0",
"383115":"0",
"383131":"0",
"383204":"0",
"383212":"0",
"383220":"0",
"383239":"0",
"383247":"0",
"383255":"0",
"383263":"0",
"383271":"0",
"383280":"0",
"383298":"0",
"383301":"0",
"383310":"0",
"383999":"0",
"384014":"1",
"384022":"1",
"384030":"1",
"384049":"1",
"384080":"1",
"384081":"1",
"384090":"0",
"384111":"1",
"384120":"1",
"385010":"0",
"385029":"0",
"385037":"0",
"391015":"0",
"391023":"0",
"391031":"0",
"391040":"0",
"391058":"0",
"391066":"0",
"391074":"0",
"391082":"0",
"391104":"0",
"391112":"0",
"391120":"0",
"391139":"0",
"391147":"0",
"391155":"0",
"391163":"0",
"391171":"0",
"391180":"0",
"391198":"0",
"391201":"0",
"391210":"0",
"391228":"0",
"391236":"0",
"391244":"0",
"391252":"0",
"391260":"0",
"391279":"0",
"391287":"0",
"391295":"0",
"391309":"0",
"391317":"0",
"391325":"0",
"391333":"0",
"391341":"0",
"392011":"0",
"392038":"0",
"392046":"1",
"392054":"1",
"392062":"1",
"392070":"1",
"392076":"1",
"392089":"1",
"392097":"1",
"392100":"1",
"392119":"1",
"392127":"1",
"392135":"1",
"392143":"1",
"392151":"1",
"392160":"1",
"392178":"0",
"392186":"1",
"392194":"0",
"392216":"0",
"392224":"1",
"392232":"1",
"392240":"1",
"392259":"1",
"392267":"1",
"392275":"0",
"392283":"1",
"392291":"1",
"392305":"1",
"392313":"1",
"392321":"1",
"392330":"1",
"392992":"1",
"393034":"0",
"393069":"0",
"393077":"0",
"393085":"0",
"393107":"0",
"393115":"0",
"393131":"0",
"393204":"0",
"393212":"0",
"393220":"0",
"393239":"0",
"393247":"0",
"393255":"0",
"393263":"0",
"393271":"0",
"393280":"0",
"393298":"0",
"393301":"0",
"393310":"0",
"393999":"0",
"394014":"1",
"394022":"1",
"394030":"1",
"394049":"1",
"394080":"1",
"394081":"1",
"394090":"0",
"394111":"1",
"394120":"1",
"395010":"0",
"395029":"0",
"395037":"0",
"411015":"0",
"411023":"0",
"411031":"0",
"411040":"0",
"411058":"0",
"411066":"0",
"411074":"0",
"411082":"0",
"411104":"0",
"411112":"0",
"411120":"0",
"411139":"0",
"411147":"0",
"411155":"0",
"411163":"0",
"411171":"0",
"411180":"0",
"411198":"0",
"411201":"0",
"411210":"0",
"411228":"0",
"411236":"0",
"411244":"0",
"411252":"0",
"411260":"0",
"411279":"0",
"411287":"0",
"411295":"0",
"411309":"0",
"411317":"0",
"411325":"0",
"411333":"0",
"411341":"0",
"412011":"0",
"412038":"0",
"412046":"1",
"412054":"1",
"412062":"1",
"412070":"1",
"412076":"1",
"412089":"1",
"412097":"1",
"412100":"1",
"412119":"1",
"412127":"1",
"412135":"1",
"412143":"1",
"412151":"1",
"412160":"1",
"412178":"0",
"412186":"1",
"412194":"0",
"412216":"0",
"412224":"1",
"412232":"1",
"412240":"1",
"412259":"1",
"412267":"1",
"412275":"0",
"412283":"1",
"412291":"1",
"412305":"1",
"412313":"1",
"412321":"1",
"412330":"1",
"412992":"1",
"413034":"0",
"413069":"0",
"413077":"0",
"413085":"0",
"413107":"0",
"413115":"0",
"413131":"0",
"413204":"0",
"413212":"0",
"413220":"0",
"413239":"0",
"413247":"0",
"413255":"0",
"413263":"0",
"413271":"0",
"413280":"0",
"413298":"0",
"413301":"0",
"413310":"0",
"413999":"0",
"414014":"1",
"414022":"1",
"414030":"1",
"414049":"1",
"414080":"1",
"414081":"1",
"414090":"0",
"414111":"1",
"414120":"1",
"415010":"0",
"415029":"0",
"415037":"0",
"421015":"0",
"421023":"0",
"421031":"0",
"421040":"0",
"421058":"0",
"421066":"0",
"421074":"0",
"421082":"0",
"421104":"0",
"421112":"0",
"421120":"0",
"421139":"0",
"421147":"0",
"421155":"0",
"421163":"0",
"421171":"0",
"421180":"0",
"421198":"0",
"421201":"0",
"421210":"0",
"421228":"0",
"421236":"0",
"421244":"0",
"421252":"0",
"421260":"0",
"421279":"0",
"421287":"0",
"421295":"0",
"421309":"0",
"421317":"0",
"421325":"0",
"421333":"0",
"421341":"0",
"422011":"0",
"422038":"0",
"422046":"1",
"422054":"1",
"422062":"1",
"422070":"1",
"422076":"1",
"422089":"1",
"422097":"1",
"422100":"1",
"422119":"1",
"422127":"1",
"422135":"1",
"422143":"1",
"422151":"1",
"422160":"1",
"422178":"0",
"422186":"1",
"422194":"0",
"422216":"0",
"422224":"1",
"422232":"1",
"422240":"1",
"422259":"1",
"422267":"1",
"422275":"0",
"422283":"1",
"422291":"1",
"422305":"1",
"422313":"1",
"422321":"1",
"422330":"1",
"422992":"1",
"423034":"0",
"423069":"0",
"423077":"0",
"423085":"0",
"423107":"0",
"423115":"0",
"423131":"0",
"423204":"0",
"423212":"0",
"423220":"0",
"423239":"0",
"423247":"0",
"423255":"0",
"423263":"0",
"423271":"0",
"423280":"0",
"423298":"0",
"423301":"0",
"423310":"0",
"423999":"0",
"424014":"1",
"424022":"1",
"424030":"1",
"424049":"1",
"424080":"1",
"424081":"1",
"424090":"0",
"424111":"1",
"424120":"1",
"425010":"0",
"425029":"0",
"425037":"0",
"431015":"0",
"431023":"0",
"431031":"0",
"431040":"0",
"431058":"0",
"431066":"0",
"431074":"0",
"431082":"0",
"431104":"0",
"431112":"0",
"431120":"0",
"431139":"0",
"431147":"0",
"431155":"0",
"431163":"0",
"431171":"0",
"431180":"0",
"431198":"0",
"431201":"0",
"431210":"0",
"431228":"0",
"431236":"0",
"431244":"0",
"431252":"0",
"431260":"0",
"431279":"0",
"431287":"0",
"431295":"0",
"431309":"0",
"431317":"0",
"431325":"0",
"431333":"0",
"431341":"0",
"432011":"0",
"432038":"0",
"432046":"1",
"432054":"1",
"432062":"1",
"432070":"1",
"432076":"1",
"432089":"1",
"432097":"1",
"432100":"1",
"432119":"1",
"432127":"1",
"432135":"1",
"432143":"1",
"432151":"1",
"432160":"1",
"432178":"0",
"432186":"1",
"432194":"0",
"432216":"0",
"432224":"1",
"432232":"1",
"432240":"1",
"432259":"1",
"432267":"1",
"432275":"0",
"432283":"1",
"432291":"1",
"432305":"1",
"432313":"1",
"432321":"1",
"432330":"1",
"432992":"1",
"433034":"0",
"433069":"0",
"433077":"0",
"433085":"0",
"433107":"0",
"433115":"0",
"433131":"0",
"433204":"0",
"433212":"0",
"433220":"0",
"433239":"0",
"433247":"0",
"433255":"0",
"433263":"0",
"433271":"0",
"433280":"0",
"433298":"0",
"433301":"0",
"433310":"0",
"433999":"0",
"434014":"1",
"434022":"1",
"434030":"1",
"434049":"1",
"434080":"1",
"434081":"1",
"434090":"0",
"434111":"1",
"434120":"1",
"435010":"0",
"435029":"0",
"435037":"0",
"451015":"0",
"451023":"0",
"451031":"0",
"451040":"0",
"451058":"0",
"451066":"0",
"451074":"0",
"451082":"0",
"451104":"0",
"451112":"0",
"451120":"0",
"451139":"0",
"451147":"0",
"451155":"0",
"451163":"0",
"451171":"0",
"451180":"0",
"451198":"0",
"451201":"0",
"451210":"0",
"451228":"0",
"451236":"0",
"451244":"0",
"451252":"0",
"451260":"0",
"451279":"0",
"451287":"0",
"451295":"0",
"451309":"0",
"451317":"0",
"451325":"0",
"451333":"0",
"451341":"0",
"452011":"0",
"452038":"0",
"452046":"1",
"452054":"1",
"452062":"1",
"452070":"1",
"452076":"1",
"452089":"1",
"452097":"1",
"452100":"1",
"452119":"1",
"452127":"1",
"452135":"1",
"452143":"1",
"452151":"1",
"452160":"1",
"452178":"0",
"452186":"1",
"452194":"0",
"452216":"0",
"452224":"1",
"452232":"1",
"452240":"1",
"452259":"1",
"452267":"1",
"452275":"0",
"452283":"1",
"452291":"1",
"452305":"1",
"452313":"1",
"452321":"1",
"452330":"1",
"452992":"1",
"453034":"0",
"453069":"0",
"453077":"0",
"453085":"0",
"453107":"0",
"453115":"0",
"453131":"0",
"453204":"0",
"453212":"0",
"453220":"0",
"453239":"0",
"453247":"0",
"453255":"0",
"453263":"0",
"453271":"0",
"453280":"0",
"453298":"0",
"453301":"0",
"453310":"0",
"453999":"0",
"454014":"1",
"454022":"1",
"454030":"1",
"454049":"1",
"454080":"1",
"454081":"1",
"454090":"0",
"454111":"1",
"454120":"1",
"455010":"0",
"455029":"0",
"455037":"0",
"461015":"0",
"461023":"0",
"461031":"0",
"461040":"0",
"461058":"0",
"461066":"0",
"461074":"0",
"461082":"0",
"461104":"0",
"461112":"0",
"461120":"0",
"461139":"0",
"461147":"0",
"461155":"0",
"461163":"0",
"461171":"0",
"461180":"0",
"461198":"0",
"461201":"0",
"461210":"0",
"461228":"0",
"461236":"0",
"461244":"0",
"461252":"0",
"461260":"0",
"461279":"0",
"461287":"0",
"461295":"0",
"461309":"0",
"461317":"0",
"461325":"0",
"461333":"0",
"461341":"0",
"462011":"0",
"462038":"0",
"462046":"1",
"462054":"1",
"462062":"1",
"462070":"1",
"462076":"1",
"462089":"1",
"462097":"1",
"462100":"1",
"462119":"1",
"462127":"1",
"462135":"1",
"462143":"1",
"462151":"1",
"462160":"1",
"462178":"0",
"462186":"1",
"462194":"0",
"462216":"0",
"462224":"1",
"462232":"1",
"462240":"1",
"462259":"1",
"462267":"1",
"462275":"0",
"462283":"1",
"462291":"1",
"462305":"1",
"462313":"1",
"462321":"1",
"462330":"1",
"462992":"1",
"463034":"0",
"463069":"0",
"463077":"0",
"463085":"0",
"463107":"0",
"463115":"0",
"463131":"0",
"463204":"0",
"463212":"0",
"463220":"0",
"463239":"0",
"463247":"0",
"463255":"0",
"463263":"0",
"463271":"0",
"463280":"0",
"463298":"0",
"463301":"0",
"463310":"0",
"463999":"0",
"464014":"1",
"464022":"1",
"464030":"1",
"464049":"1",
"464080":"1",
"464081":"1",
"464090":"0",
"464111":"1",
"464120":"1",
"465010":"0",
"465029":"0",
"465037":"0",
"471015":"0",
"471023":"0",
"471031":"0",
"471040":"0",
"471058":"0",
"471066":"0",
"471074":"0",
"471082":"0",
"471104":"0",
"471112":"0",
"471120":"0",
"471139":"0",
"471147":"0",
"471155":"0",
"471163":"0",
"471171":"0",
"471180":"0",
"471198":"0",
"471201":"0",
"471210":"0",
"471228":"0",
"471236":"0",
"471244":"0",
"471252":"0",
"471260":"0",
"471279":"0",
"471287":"0",
"471295":"0",
"471309":"0",
"471317":"0",
"471325":"0",
"471333":"0",
"471341":"0",
"472011":"0",
"472038":"0",
"472046":"1",
"472054":"1",
"472062":"1",
"472070":"1",
"472076":"1",
"472089":"1",
"472097":"1",
"472100":"1",
"472119":"1",
"472127":"1",
"472135":"1",
"472143":"1",
"472151":"1",
"472160":"1",
"472178":"0",
"472186":"1",
"472194":"0",
"472216":"0",
"472224":"1",
"472232":"1",
"472240":"1",
"472259":"1",
"472267":"1",
"472275":"0",
"472283":"1",
"472291":"1",
"472305":"1",
"472313":"1",
"472321":"1",
"472330":"1",
"472992":"1",
"473034":"0",
"473069":"0",
"473077":"0",
"473085":"0",
"473107":"0",
"473115":"0",
"473131":"0",
"473204":"0",
"473212":"0",
"473220":"0",
"473239":"0",
"473247":"0",
"473255":"0",
"473263":"0",
"473271":"0",
"473280":"0",
"473298":"0",
"473301":"0",
"473310":"0",
"473999":"0",
"474014":"1",
"474022":"1",
"474030":"1",
"474049":"1",
"474080":"1",
"474081":"1",
"474090":"0",
"474111":"1",
"474120":"1",
"475010":"0",
"475029":"0",
"475037":"0",
"491015":"0",
"491023":"0",
"491031":"0",
"491040":"0",
"491058":"0",
"491066":"0",
"491074":"0",
"491082":"0",
"491104":"0",
"491112":"0",
"491120":"0",
"491139":"0",
"491147":"0",
"491155":"0",
"491163":"0",
"491171":"0",
"491180":"0",
"491198":"0",
"491201":"0",
"491210":"0",
"491228":"0",
"491236":"0",
"491244":"0",
"491252":"0",
"491260":"0",
"491279":"0",
"491287":"0",
"491295":"0",
"491309":"0",
"491317":"0",
"491325":"0",
"491333":"0",
"491341":"0",
"492011":"0",
"492038":"0",
"492046":"1",
"492054":"1",
"492062":"1",
"492070":"1",
"492076":"1",
"492089":"1",
"492097":"1",
"492100":"1",
"492119":"1",
"492127":"1",
"492135":"1",
"492143":"1",
"492151":"1",
"492160":"1",
"492178":"0",
"492186":"1",
"492194":"0",
"492216":"0",
"492224":"1",
"492232":"1",
"492240":"1",
"492259":"1",
"492267":"1",
"492275":"0",
"492283":"1",
"492291":"1",
"492305":"1",
"492313":"1",
"492321":"1",
"492330":"1",
"492992":"1",
"493034":"0",
"493069":"0",
"493077":"0",
"493085":"0",
"493107":"0",
"493115":"0",
"493131":"0",
"493204":"0",
"493212":"0",
"493220":"0",
"493239":"0",
"493247":"0",
"493255":"0",
"493263":"0",
"493271":"0",
"493280":"0",
"493298":"0",
"493301":"0",
"493310":"0",
"493999":"0",
"494014":"1",
"494022":"1",
"494030":"1",
"494049":"1",
"494080":"1",
"494081":"1",
"494090":"0",
"494111":"1",
"494120":"1",
"495010":"0",
"495029":"0",
"495037":"0",
"501015":"0",
"501023":"0",
"501031":"0",
"501040":"0",
"501058":"0",
"501066":"0",
"501074":"0",
"501082":"0",
"501104":"0",
"501112":"0",
"501120":"0",
"501139":"0",
"501147":"0",
"501155":"0",
"501163":"0",
"501171":"0",
"501180":"0",
"501198":"0",
"501201":"0",
"501210":"0",
"501228":"0",
"501236":"0",
"501244":"0",
"501252":"0",
"501260":"0",
"501279":"0",
"501287":"0",
"501295":"0",
"501309":"0",
"501317":"0",
"501325":"0",
"501333":"0",
"501341":"0",
"502011":"0",
"502038":"0",
"502046":"1",
"502054":"1",
"502062":"1",
"502070":"1",
"502076":"1",
"502089":"1",
"502097":"1",
"502100":"1",
"502119":"1",
"502127":"1",
"502135":"1",
"502143":"1",
"502151":"1",
"502160":"1",
"502178":"0",
"502186":"1",
"502194":"0",
"502216":"0",
"502224":"1",
"502232":"1",
"502240":"1",
"502259":"1",
"502267":"1",
"502275":"0",
"502283":"1",
"502291":"1",
"502305":"1",
"502313":"1",
"502321":"1",
"502330":"1",
"502992":"1",
"503034":"0",
"503069":"0",
"503077":"0",
"503085":"0",
"503107":"0",
"503115":"0",
"503131":"0",
"503204":"0",
"503212":"0",
"503220":"0",
"503239":"0",
"503247":"0",
"503255":"0",
"503263":"0",
"503271":"0",
"503280":"0",
"503298":"0",
"503301":"0",
"503310":"0",
"503999":"0",
"504014":"1",
"504022":"1",
"504030":"1",
"504049":"1",
"504080":"1",
"504081":"1",
"504090":"0",
"504111":"1",
"504120":"1",
"505010":"0",
"505029":"0",
"505037":"0",
"511015":"0",
"511023":"0",
"511031":"0",
"511040":"0",
"511058":"0",
"511066":"0",
"511074":"0",
"511082":"0",
"511104":"0",
"511112":"0",
"511120":"0",
"511139":"0",
"511147":"0",
"511155":"0",
"511163":"0",
"511171":"0",
"511180":"0",
"511198":"0",
"511201":"0",
"511210":"0",
"511228":"0",
"511236":"0",
"511244":"0",
"511252":"0",
"511260":"0",
"511279":"0",
"511287":"0",
"511295":"0",
"511309":"0",
"511317":"0",
"511325":"0",
"511333":"0",
"511341":"0",
"512011":"0",
"512038":"0",
"512046":"1",
"512054":"1",
"512062":"1",
"512070":"1",
"512076":"1",
"512089":"1",
"512097":"1",
"512100":"1",
"512119":"1",
"512127":"1",
"512135":"1",
"512143":"1",
"512151":"1",
"512160":"1",
"512178":"0",
"512186":"1",
"512194":"0",
"512216":"0",
"512224":"1",
"512232":"1",
"512240":"1",
"512259":"1",
"512267":"1",
"512275":"0",
"512283":"1",
"512291":"1",
"512305":"1",
"512313":"1",
"512321":"1",
"512330":"1",
"512992":"1",
"513034":"0",
"513069":"0",
"513077":"0",
"513085":"0",
"513107":"0",
"513115":"0",
"513131":"0",
"513204":"0",
"513212":"0",
"513220":"0",
"513239":"0",
"513247":"0",
"513255":"0",
"513263":"0",
"513271":"0",
"513280":"0",
"513298":"0",
"513301":"0",
"513310":"0",
"513999":"0",
"514014":"1",
"514022":"1",
"514030":"1",
"514049":"1",
"514080":"1",
"514081":"1",
"514090":"0",
"514111":"1",
"514120":"1",
"515010":"0",
"515029":"0",
"515037":"0",
"521015":"0",
"521023":"0",
"521031":"0",
"521040":"0",
"521058":"0",
"521066":"0",
"521074":"0",
"521082":"0",
"521104":"0",
"521112":"0",
"521120":"0",
"521139":"0",
"521147":"0",
"521155":"0",
"521163":"0",
"521171":"0",
"521180":"0",
"521198":"0",
"521201":"0",
"521210":"0",
"521228":"0",
"521236":"0",
"521244":"0",
"521252":"0",
"521260":"0",
"521279":"0",
"521287":"0",
"521295":"0",
"521309":"0",
"521317":"0",
"521325":"0",
"521333":"0",
"521341":"0",
"522011":"0",
"522038":"0",
"522046":"1",
"522054":"1",
"522062":"1",
"522070":"1",
"522076":"1",
"522089":"1",
"522097":"1",
"522100":"1",
"522119":"1",
"522127":"1",
"522135":"1",
"522143":"1",
"522151":"1",
"522160":"1",
"522178":"0",
"522186":"1",
"522194":"0",
"522216":"0",
"522224":"1",
"522232":"1",
"522240":"1",
"522259":"1",
"522267":"1",
"522275":"0",
"522283":"1",
"522291":"1",
"522305":"1",
"522313":"1",
"522321":"1",
"522330":"1",
"522992":"1",
"523034":"0",
"523069":"0",
"523077":"0",
"523085":"0",
"523107":"0",
"523115":"0",
"523131":"0",
"523204":"0",
"523212":"0",
"523220":"0",
"523239":"0",
"523247":"0",
"523255":"0",
"523263":"0",
"523271":"0",
"523280":"0",
"523298":"0",
"523301":"0",
"523310":"0",
"523999":"0",
"524014":"1",
"524022":"1",
"524030":"1",
"524049":"1",
"524080":"1",
"524081":"1",
"524090":"0",
"524111":"1",
"524120":"1",
"525010":"0",
"525029":"0",
"525037":"0",
"531015":"0",
"531023":"0",
"531031":"0",
"531040":"0",
"531058":"0",
"531066":"0",
"531074":"0",
"531082":"0",
"531104":"0",
"531112":"0",
"531120":"0",
"531139":"0",
"531147":"0",
"531155":"0",
"531163":"0",
"531171":"0",
"531180":"0",
"531198":"0",
"531201":"0",
"531210":"0",
"531228":"0",
"531236":"0",
"531244":"0",
"531252":"0",
"531260":"0",
"531279":"0",
"531287":"0",
"531295":"0",
"531309":"0",
"531317":"0",
"531325":"0",
"531333":"0",
"531341":"0",
"532011":"0",
"532038":"0",
"532046":"1",
"532054":"1",
"532062":"1",
"532070":"1",
"532076":"1",
"532089":"1",
"532097":"1",
"532100":"1",
"532119":"1",
"532127":"1",
"532135":"1",
"532143":"1",
"532151":"1",
"532160":"1",
"532178":"0",
"532186":"1",
"532194":"0",
"532216":"0",
"532224":"1",
"532232":"1",
"532240":"1",
"532259":"1",
"532267":"1",
"532275":"0",
"532283":"1",
"532291":"1",
"532305":"1",
"532313":"1",
"532321":"1",
"532330":"1",
"532992":"1",
"533034":"0",
"533069":"0",
"533077":"0",
"533085":"0",
"533107":"0",
"533115":"0",
"533131":"0",
"533204":"0",
"533212":"0",
"533220":"0",
"533239":"0",
"533247":"0",
"533255":"0",
"533263":"0",
"533271":"0",
"533280":"0",
"533298":"0",
"533301":"0",
"533310":"0",
"533999":"0",
"534014":"1",
"534022":"1",
"534030":"1",
"534049":"1",
"534080":"1",
"534081":"1",
"534090":"0",
"534111":"1",
"534120":"1",
"535010":"0",
"535029":"0",
"535037":"0",
"551015":"0",
"551023":"0",
"551031":"0",
"551040":"0",
"551058":"0",
"551066":"0",
"551074":"0",
"551082":"0",
"551104":"0",
"551112":"0",
"551120":"0",
"551139":"0",
"551147":"0",
"551155":"0",
"551163":"0",
"551171":"0",
"551180":"0",
"551198":"0",
"551201":"0",
"551210":"0",
"551228":"0",
"551236":"0",
"551244":"0",
"551252":"0",
"551260":"0",
"551279":"0",
"551287":"0",
"551295":"0",
"551309":"0",
"551317":"0",
"551325":"0",
"551333":"0",
"551341":"0",
"552011":"0",
"552038":"0",
"552046":"1",
"552054":"1",
"552062":"1",
"552070":"1",
"552076":"1",
"552089":"1",
"552097":"1",
"552100":"1",
"552119":"1",
"552127":"1",
"552135":"1",
"552143":"1",
"552151":"1",
"552160":"1",
"552178":"0",
"552186":"1",
"552194":"0",
"552216":"0",
"552224":"1",
"552232":"1",
"552240":"1",
"552259":"1",
"552267":"1",
"552275":"0",
"552283":"1",
"552291":"1",
"552305":"1",
"552313":"1",
"552321":"1",
"552330":"1",
"552992":"1",
"553034":"0",
"553069":"0",
"553077":"0",
"553085":"0",
"553107":"0",
"553115":"0",
"553131":"0",
"553204":"0",
"553212":"0",
"553220":"0",
"553239":"0",
"553247":"0",
"553255":"0",
"553263":"0",
"553271":"0",
"553280":"0",
"553298":"0",
"553301":"0",
"553310":"0",
"553999":"0",
"554014":"1",
"554022":"1",
"554030":"1",
"554049":"1",
"554080":"1",
"554081":"1",
"554090":"0",
"554111":"1",
"554120":"1",
"555010":"0",
"555029":"0",
"555037":"0",
"561015":"0",
"561023":"0",
"561031":"0",
"561040":"0",
"561058":"0",
"561066":"0",
"561074":"0",
"561082":"0",
"561104":"0",
"561112":"0",
"561120":"0",
"561139":"0",
"561147":"0",
"561155":"0",
"561163":"0",
"561171":"0",
"561180":"0",
"561198":"0",
"561201":"0",
"561210":"0",
"561228":"0",
"561236":"0",
"561244":"0",
"561252":"0",
"561260":"0",
"561279":"0",
"561287":"0",
"561295":"0",
"561309":"0",
"561317":"0",
"561325":"0",
"561333":"0",
"561341":"0",
"562011":"0",
"562038":"0",
"562046":"1",
"562054":"1",
"562062":"1",
"562070":"1",
"562076":"1",
"562089":"1",
"562097":"1",
"562100":"1",
"562119":"1",
"562127":"1",
"562135":"1",
"562143":"1",
"562151":"1",
"562160":"1",
"562178":"0",
"562186":"1",
"562194":"0",
"562216":"0",
"562224":"1",
"562232":"1",
"562240":"1",
"562259":"1",
"562267":"1",
"562275":"0",
"562283":"1",
"562291":"1",
"562305":"1",
"562313":"1",
"562321":"1",
"562330":"1",
"562992":"1",
"563034":"0",
"563069":"0",
"563077":"0",
"563085":"0",
"563107":"0",
"563115":"0",
"563131":"0",
"563204":"0",
"563212":"0",
"563220":"0",
"563239":"0",
"563247":"0",
"563255":"0",
"563263":"0",
"563271":"0",
"563280":"0",
"563298":"0",
"563301":"0",
"563310":"0",
"563999":"0",
"564014":"1",
"564022":"1",
"564030":"1",
"564049":"1",
"564080":"1",
"564081":"1",
"564090":"0",
"564111":"1",
"564120":"1",
"565010":"0",
"565029":"0",
"565037":"0",
"581015":"0",
"581023":"0",
"581031":"0",
"581040":"0",
"581058":"0",
"581066":"0",
"581074":"0",
"581082":"0",
"581104":"0",
"581112":"0",
"581120":"0",
"581139":"0",
"581147":"0",
"581155":"0",
"581163":"0",
"581171":"0",
"581180":"0",
"581198":"0",
"581201":"0",
"581210":"0",
"581228":"0",
"581236":"0",
"581244":"0",
"581252":"0",
"581260":"0",
"581279":"0",
"581287":"0",
"581295":"0",
"581309":"0",
"581317":"0",
"581325":"0",
"581333":"0",
"581341":"0",
"582011":"0",
"582038":"0",
"582046":"1",
"582054":"1",
"582062":"1",
"582070":"1",
"582076":"1",
"582089":"1",
"582097":"1",
"582100":"1",
"582119":"1",
"582127":"1",
"582135":"1",
"582143":"1",
"582151":"1",
"582160":"1",
"582178":"0",
"582186":"1",
"582194":"0",
"582216":"0",
"582224":"1",
"582232":"1",
"582240":"1",
"582259":"1",
"582267":"1",
"582275":"0",
"582283":"1",
"582291":"1",
"582305":"1",
"582313":"1",
"582321":"1",
"582330":"1",
"582992":"1",
"583034":"0",
"583069":"0",
"583077":"0",
"583085":"0",
"583107":"0",
"583115":"0",
"583131":"0",
"583204":"0",
"583212":"0",
"583220":"0",
"583239":"0",
"583247":"0",
"583255":"0",
"583263":"0",
"583271":"0",
"583280":"0",
"583298":"0",
"583301":"0",
"583310":"0",
"583999":"0",
"584014":"1",
"584022":"1",
"584030":"1",
"584049":"1",
"584080":"1",
"584081":"1",
"584090":"0",
"584111":"1",
"584120":"1",
"585010":"0",
"585029":"0",
"585037":"0",
"591015":"0",
"591023":"0",
"591031":"0",
"591040":"0",
"591058":"0",
"591066":"0",
"591074":"0",
"591082":"0",
"591104":"0",
"591112":"0",
"591120":"0",
"591139":"0",
"591147":"0",
"591155":"0",
"591163":"0",
"591171":"0",
"591180":"0",
"591198":"0",
"591201":"0",
"591210":"0",
"591228":"0",
"591236":"0",
"591244":"0",
"591252":"0",
"591260":"0",
"591279":"0",
"591287":"0",
"591295":"0",
"591309":"0",
"591317":"0",
"591325":"0",
"591333":"0",
"591341":"0",
"592011":"0",
"592038":"0",
"592046":"1",
"592054":"1",
"592062":"1",
"592070":"1",
"592076":"1",
"592089":"1",
"592097":"1",
"592100":"1",
"592119":"1",
"592127":"1",
"592135":"1",
"592143":"1",
"592151":"1",
"592160":"1",
"592178":"0",
"592186":"1",
"592194":"0",
"592216":"0",
"592224":"1",
"592232":"1",
"592240":"1",
"592259":"1",
"592267":"1",
"592275":"0",
"592283":"1",
"592291":"1",
"592305":"1",
"592313":"1",
"592321":"1",
"592330":"1",
"592992":"1",
"593034":"0",
"593069":"0",
"593077":"0",
"593085":"0",
"593107":"0",
"593115":"0",
"593131":"0",
"593204":"0",
"593212":"0",
"593220":"0",
"593239":"0",
"593247":"0",
"593255":"0",
"593263":"0",
"593271":"0",
"593280":"0",
"593298":"0",
"593301":"0",
"593310":"0",
"593999":"0",
"594014":"1",
"594022":"1",
"594030":"1",
"594049":"1",
"594080":"1",
"594081":"1",
"594090":"0",
"594111":"1",
"594120":"1",
"595010":"0",
"595029":"0",
"595037":"0",
"601015":"0",
"601023":"0",
"601031":"0",
"601040":"0",
"601058":"0",
"601066":"0",
"601074":"0",
"601082":"0",
"601104":"0",
"601112":"0",
"601120":"0",
"601139":"0",
"601147":"0",
"601155":"0",
"601163":"0",
"601171":"0",
"601180":"0",
"601198":"0",
"601201":"0",
"601210":"0",
"601228":"0",
"601236":"0",
"601244":"0",
"601252":"0",
"601260":"0",
"601279":"0",
"601287":"0",
"601295":"0",
"601309":"0",
"601317":"0",
"601325":"0",
"601333":"0",
"601341":"0",
"602011":"0",
"602038":"0",
"602046":"1",
"602054":"1",
"602062":"1",
"602070":"1",
"602076":"1",
"602089":"1",
"602097":"1",
"602100":"1",
"602119":"1",
"602127":"1",
"602135":"1",
"602143":"1",
"602151":"1",
"602160":"1",
"602178":"0",
"602186":"1",
"602194":"0",
"602216":"0",
"602224":"1",
"602232":"1",
"602240":"1",
"602259":"1",
"602267":"1",
"602275":"0",
"602283":"1",
"602291":"1",
"602305":"1",
"602313":"1",
"602321":"1",
"602330":"1",
"602992":"1",
"603034":"0",
"603069":"0",
"603077":"0",
"603085":"0",
"603107":"0",
"603115":"0",
"603131":"0",
"603204":"0",
"603212":"0",
"603220":"0",
"603239":"0",
"603247":"0",
"603255":"0",
"603263":"0",
"603271":"0",
"603280":"0",
"603298":"0",
"603301":"0",
"603310":"0",
"603999":"0",
"604014":"1",
"604022":"1",
"604030":"1",
"604049":"1",
"604080":"1",
"604081":"1",
"604090":"0",
"604111":"1",
"604120":"1",
"605010":"0",
"605029":"0",
"605037":"0",
"611015":"0",
"611023":"0",
"611031":"0",
"611040":"0",
"611058":"0",
"611066":"0",
"611074":"0",
"611082":"0",
"611104":"0",
"611112":"0",
"611120":"0",
"611139":"0",
"611147":"0",
"611155":"0",
"611163":"0",
"611171":"0",
"611180":"0",
"611198":"0",
"611201":"0",
"611210":"0",
"611228":"0",
"611236":"0",
"611244":"0",
"611252":"0",
"611260":"0",
"611279":"0",
"611287":"0",
"611295":"0",
"611309":"0",
"611317":"0",
"611325":"0",
"611333":"0",
"611341":"0",
"612011":"0",
"612038":"0",
"612046":"1",
"612054":"1",
"612062":"1",
"612070":"1",
"612076":"1",
"612089":"1",
"612097":"1",
"612100":"1",
"612119":"1",
"612127":"1",
"612135":"1",
"612143":"1",
"612151":"1",
"612160":"1",
"612178":"0",
"612186":"1",
"612194":"0",
"612216":"0",
"612224":"1",
"612232":"1",
"612240":"1",
"612259":"1",
"612267":"1",
"612275":"0",
"612283":"1",
"612291":"1",
"612305":"1",
"612313":"1",
"612321":"1",
"612330":"1",
"612992":"1",
"613034":"0",
"613069":"0",
"613077":"0",
"613085":"0",
"613107":"0",
"613115":"0",
"613131":"0",
"613204":"0",
"613212":"0",
"613220":"0",
"613239":"0",
"613247":"0",
"613255":"0",
"613263":"0",
"613271":"0",
"613280":"0",
"613298":"0",
"613301":"0",
"613310":"0",
"613999":"0",
"614014":"1",
"614022":"1",
"614030":"1",
"614049":"1",
"614080":"1",
"614081":"1",
"614090":"0",
"614111":"1",
"614120":"1",
"615010":"0",
"615029":"0",
"615037":"0",
"621015":"0",
"621023":"0",
"621031":"0",
"621040":"0",
"621058":"0",
"621066":"0",
"621074":"0",
"621082":"0",
"621104":"0",
"621112":"0",
"621120":"0",
"621139":"0",
"621147":"0",
"621155":"0",
"621163":"0",
"621171":"0",
"621180":"0",
"621198":"0",
"621201":"0",
"621210":"0",
"621228":"0",
"621236":"0",
"621244":"0",
"621252":"0",
"621260":"0",
"621279":"0",
"621287":"0",
"621295":"0",
"621309":"0",
"621317":"0",
"621325":"0",
"621333":"0",
"621341":"0",
"622011":"0",
"622038":"0",
"622046":"1",
"622054":"1",
"622062":"1",
"622070":"1",
"622076":"1",
"622089":"1",
"622097":"1",
"622100":"1",
"622119":"1",
"622127":"1",
"622135":"1",
"622143":"1",
"622151":"1",
"622160":"1",
"622178":"0",
"622186":"1",
"622194":"0",
"622216":"0",
"622224":"1",
"622232":"1",
"622240":"1",
"622259":"1",
"622267":"1",
"622275":"0",
"622283":"1",
"622291":"1",
"622305":"1",
"622313":"1",
"622321":"1",
"622330":"1",
"622992":"1",
"623034":"0",
"623069":"0",
"623077":"0",
"623085":"0",
"623107":"0",
"623115":"0",
"623131":"0",
"623204":"0",
"623212":"0",
"623220":"0",
"623239":"0",
"623247":"0",
"623255":"0",
"623263":"0",
"623271":"0",
"623280":"0",
"623298":"0",
"623301":"0",
"623310":"0",
"623999":"0",
"624014":"1",
"624022":"1",
"624030":"1",
"624049":"1",
"624080":"1",
"624081":"1",
"624090":"0",
"624111":"1",
"624120":"1",
"625010":"0",
"625029":"0",
"625037":"0",
"631015":"0",
"631023":"0",
"631031":"0",
"631040":"0",
"631058":"0",
"631066":"0",
"631074":"0",
"631082":"0",
"631104":"0",
"631112":"0",
"631120":"0",
"631139":"0",
"631147":"0",
"631155":"0",
"631163":"0",
"631171":"0",
"631180":"0",
"631198":"0",
"631201":"0",
"631210":"0",
"631228":"0",
"631236":"0",
"631244":"0",
"631252":"0",
"631260":"0",
"631279":"0",
"631287":"0",
"631295":"0",
"631309":"0",
"631317":"0",
"631325":"0",
"631333":"0",
"631341":"0",
"632011":"0",
"632038":"0",
"632046":"1",
"632054":"1",
"632062":"1",
"632070":"1",
"632076":"1",
"632089":"1",
"632097":"1",
"632100":"1",
"632119":"1",
"632127":"1",
"632135":"1",
"632143":"1",
"632151":"1",
"632160":"1",
"632178":"0",
"632186":"1",
"632194":"0",
"632216":"0",
"632224":"1",
"632232":"1",
"632240":"1",
"632259":"1",
"632267":"1",
"632275":"0",
"632283":"1",
"632291":"1",
"632305":"1",
"632313":"1",
"632321":"1",
"632330":"1",
"632992":"1",
"633034":"0",
"633069":"0",
"633077":"0",
"633085":"0",
"633107":"0",
"633115":"0",
"633131":"0",
"633204":"0",
"633212":"0",
"633220":"0",
"633239":"0",
"633247":"0",
"633255":"0",
"633263":"0",
"633271":"0",
"633280":"0",
"633298":"0",
"633301":"0",
"633310":"0",
"633999":"0",
"634014":"1",
"634022":"1",
"634030":"1",
"634049":"1",
"634080":"1",
"634081":"1",
"634090":"0",
"634111":"1",
"634120":"1",
"635010":"0",
"635029":"0",
"635037":"0",
"641015":"0",
"641023":"0",
"641031":"0",
"641040":"0",
"641058":"0",
"641066":"0",
"641074":"0",
"641082":"0",
"641104":"0",
"641112":"0",
"641120":"0",
"641139":"0",
"641147":"0",
"641155":"0",
"641163":"0",
"641171":"0",
"641180":"0",
"641198":"0",
"641201":"0",
"641210":"0",
"641228":"0",
"641236":"0",
"641244":"0",
"641252":"0",
"641260":"0",
"641279":"0",
"641287":"0",
"641295":"0",
"641309":"0",
"641317":"0",
"641325":"0",
"641333":"0",
"641341":"0",
"642011":"0",
"642038":"0",
"642046":"1",
"642054":"1",
"642062":"1",
"642070":"1",
"642076":"1",
"642089":"1",
"642097":"1",
"642100":"1",
"642119":"1",
"642127":"1",
"642135":"1",
"642143":"1",
"642151":"1",
"642160":"1",
"642178":"0",
"642186":"1",
"642194":"0",
"642216":"0",
"642224":"1",
"642232":"1",
"642240":"1",
"642259":"1",
"642267":"1",
"642275":"0",
"642283":"1",
"642291":"1",
"642305":"1",
"642313":"1",
"642321":"1",
"642330":"1",
"642992":"1",
"643034":"0",
"643069":"0",
"643077":"0",
"643085":"0",
"643107":"0",
"643115":"0",
"643131":"0",
"643204":"0",
"643212":"0",
"643220":"0",
"643239":"0",
"643247":"0",
"643255":"0",
"643263":"0",
"643271":"0",
"643280":"0",
"643298":"0",
"643301":"0",
"643310":"0",
"643999":"0",
"644014":"1",
"644022":"1",
"644030":"1",
"644049":"1",
"644080":"1",
"644081":"1",
"644090":"0",
"644111":"1",
"644120":"1",
"645010":"0",
"645029":"0",
"645037":"0",
"651015":"0",
"651023":"0",
"651031":"0",
"651040":"0",
"651058":"0",
"651066":"0",
"651074":"0",
"651082":"0",
"651104":"0",
"651112":"0",
"651120":"0",
"651139":"0",
"651147":"0",
"651155":"0",
"651163":"0",
"651171":"0",
"651180":"0",
"651198":"0",
"651201":"0",
"651210":"0",
"651228":"0",
"651236":"0",
"651244":"0",
"651252":"0",
"651260":"0",
"651279":"0",
"651287":"0",
"651295":"0",
"651309":"0",
"651317":"0",
"651325":"0",
"651333":"0",
"651341":"0",
"652011":"0",
"652038":"0",
"652046":"1",
"652054":"1",
"652062":"1",
"652070":"1",
"652076":"1",
"652089":"1",
"652097":"1",
"652100":"1",
"652119":"1",
"652127":"1",
"652135":"1",
"652143":"1",
"652151":"1",
"652160":"1",
"652178":"0",
"652186":"1",
"652194":"0",
"652216":"0",
"652224":"1",
"652232":"1",
"652240":"1",
"652259":"1",
"652267":"1",
"652275":"0",
"652283":"1",
"652291":"1",
"652305":"1",
"652313":"1",
"652321":"1",
"652330":"1",
"652992":"1",
"653034":"0",
"653069":"0",
"653077":"0",
"653085":"0",
"653107":"0",
"653115":"0",
"653131":"0",
"653204":"0",
"653212":"0",
"653220":"0",
"653239":"0",
"653247":"0",
"653255":"0",
"653263":"0",
"653271":"0",
"653280":"0",
"653298":"0",
"653301":"0",
"653310":"0",
"653999":"0",
"654014":"1",
"654022":"1",
"654030":"1",
"654049":"1",
"654080":"1",
"654081":"1",
"654090":"0",
"654111":"1",
"654120":"1",
"655010":"0",
"655029":"0",
"655037":"0",
"661015":"0",
"661023":"0",
"661031":"0",
"661040":"0",
"661058":"0",
"661066":"0",
"661074":"0",
"661082":"0",
"661104":"0",
"661112":"0",
"661120":"0",
"661139":"0",
"661147":"0",
"661155":"0",
"661163":"0",
"661171":"0",
"661180":"0",
"661198":"0",
"661201":"0",
"661210":"0",
"661228":"0",
"661236":"0",
"661244":"0",
"661252":"0",
"661260":"0",
"661279":"0",
"661287":"0",
"661295":"0",
"661309":"0",
"661317":"0",
"661325":"0",
"661333":"0",
"661341":"0",
"662011":"0",
"662038":"0",
"662046":"1",
"662054":"1",
"662062":"1",
"662070":"1",
"662076":"1",
"662089":"1",
"662097":"1",
"662100":"1",
"662119":"1",
"662127":"1",
"662135":"1",
"662143":"1",
"662151":"1",
"662160":"1",
"662178":"0",
"662186":"1",
"662194":"0",
"662216":"0",
"662224":"1",
"662232":"1",
"662240":"1",
"662259":"1",
"662267":"1",
"662275":"0",
"662283":"1",
"662291":"1",
"662305":"1",
"662313":"1",
"662321":"1",
"662330":"1",
"662992":"1",
"663034":"0",
"663069":"0",
"663077":"0",
"663085":"0",
"663107":"0",
"663115":"0",
"663131":"0",
"663204":"0",
"663212":"0",
"663220":"0",
"663239":"0",
"663247":"0",
"663255":"0",
"663263":"0",
"663271":"0",
"663280":"0",
"663298":"0",
"663301":"0",
"663310":"0",
"663999":"0",
"664014":"1",
"664022":"1",
"664030":"1",
"664049":"1",
"664080":"1",
"664081":"1",
"664090":"0",
"664111":"1",
"664120":"1",
"665010":"0",
"665029":"0",
"665037":"0",
"681015":"0",
"681023":"0",
"681031":"0",
"681040":"0",
"681058":"0",
"681066":"0",
"681074":"0",
"681082":"0",
"681104":"0",
"681112":"0",
"681120":"0",
"681139":"0",
"681147":"0",
"681155":"0",
"681163":"0",
"681171":"0",
"681180":"0",
"681198":"0",
"681201":"0",
"681210":"0",
"681228":"0",
"681236":"0",
"681244":"0",
"681252":"0",
"681260":"0",
"681279":"0",
"681287":"0",
"681295":"0",
"681309":"0",
"681317":"0",
"681325":"0",
"681333":"0",
"681341":"0",
"682011":"0",
"682038":"0",
"682046":"1",
"682054":"1",
"682062":"1",
"682070":"1",
"682076":"1",
"682089":"1",
"682097":"1",
"682100":"1",
"682119":"1",
"682127":"1",
"682135":"1",
"682143":"1",
"682151":"1",
"682160":"1",
"682178":"0",
"682186":"1",
"682194":"0",
"682216":"0",
"682224":"1",
"682232":"1",
"682240":"1",
"682259":"1",
"682267":"1",
"682275":"0",
"682283":"1",
"682291":"1",
"682305":"1",
"682313":"1",
"682321":"1",
"682330":"1",
"682992":"1",
"683034":"0",
"683069":"0",
"683077":"0",
"683085":"0",
"683107":"0",
"683115":"0",
"683131":"0",
"683204":"0",
"683212":"0",
"683220":"0",
"683239":"0",
"683247":"0",
"683255":"0",
"683263":"0",
"683271":"0",
"683280":"0",
"683298":"0",
"683301":"0",
"683310":"0",
"683999":"0",
"684014":"1",
"684022":"1",
"684030":"1",
"684049":"1",
"684080":"1",
"684081":"1",
"684090":"0",
"684111":"1",
"684120":"1",
"685010":"0",
"685029":"0",
"685037":"0",
"691015":"0",
"691023":"0",
"691031":"0",
"691040":"0",
"691058":"0",
"691066":"0",
"691074":"0",
"691082":"0",
"691104":"0",
"691112":"0",
"691120":"0",
"691139":"0",
"691147":"0",
"691155":"0",
"691163":"0",
"691171":"0",
"691180":"0",
"691198":"0",
"691201":"0",
"691210":"0",
"691228":"0",
"691236":"0",
"691244":"0",
"691252":"0",
"691260":"0",
"691279":"0",
"691287":"0",
"691295":"0",
"691309":"0",
"691317":"0",
"691325":"0",
"691333":"0",
"691341":"0",
"692011":"0",
"692038":"0",
"692046":"1",
"692054":"1",
"692062":"1",
"692070":"1",
"692076":"1",
"692089":"1",
"692097":"1",
"692100":"1",
"692119":"1",
"692127":"1",
"692135":"1",
"692143":"1",
"692151":"1",
"692160":"1",
"692178":"0",
"692186":"1",
"692194":"0",
"692216":"0",
"692224":"1",
"692232":"1",
"692240":"1",
"692259":"1",
"692267":"1",
"692275":"0",
"692283":"1",
"692291":"1",
"692305":"1",
"692313":"1",
"692321":"1",
"692330":"1",
"692992":"1",
"693034":"0",
"693069":"0",
"693077":"0",
"693085":"0",
"693107":"0",
"693115":"0",
"693131":"0",
"693204":"0",
"693212":"0",
"693220":"0",
"693239":"0",
"693247":"0",
"693255":"0",
"693263":"0",
"693271":"0",
"693280":"0",
"693298":"0",
"693301":"0",
"693310":"0",
"693999":"0",
"694014":"1",
"694022":"1",
"694030":"1",
"694049":"1",
"694080":"1",
"694081":"1",
"694090":"0",
"694111":"1",
"694120":"1",
"695010":"0",
"695029":"0",
"695037":"0",
"701015":"0",
"701023":"0",
"701031":"0",
"701040":"0",
"701058":"0",
"701066":"0",
"701074":"0",
"701082":"0",
"701104":"0",
"701112":"0",
"701120":"0",
"701139":"0",
"701147":"0",
"701155":"0",
"701163":"0",
"701171":"0",
"701180":"0",
"701198":"0",
"701201":"0",
"701210":"0",
"701228":"0",
"701236":"0",
"701244":"0",
"701252":"0",
"701260":"0",
"701279":"0",
"701287":"0",
"701295":"0",
"701309":"0",
"701317":"0",
"701325":"0",
"701333":"0",
"701341":"0",
"702011":"0",
"702038":"0",
"702046":"1",
"702054":"1",
"702062":"1",
"702070":"1",
"702076":"1",
"702089":"1",
"702097":"1",
"702100":"1",
"702119":"1",
"702127":"1",
"702135":"1",
"702143":"1",
"702151":"1",
"702160":"1",
"702178":"0",
"702186":"1",
"702194":"0",
"702216":"0",
"702224":"1",
"702232":"1",
"702240":"1",
"702259":"1",
"702267":"1",
"702275":"0",
"702283":"1",
"702291":"1",
"702305":"1",
"702313":"1",
"702321":"1",
"702330":"1",
"702992":"1",
"703034":"0",
"703069":"0",
"703077":"0",
"703085":"0",
"703107":"0",
"703115":"0",
"703131":"0",
"703204":"0",
"703212":"0",
"703220":"0",
"703239":"0",
"703247":"0",
"703255":"0",
"703263":"0",
"703271":"0",
"703280":"0",
"703298":"0",
"703301":"0",
"703310":"0",
"703999":"0",
"704014":"1",
"704022":"1",
"704030":"1",
"704049":"1",
"704080":"1",
"704081":"1",
"704090":"0",
"704111":"1",
"704120":"1",
"705010":"0",
"705029":"0",
"705037":"0",
"711015":"0",
"711023":"0",
"711031":"0",
"711040":"0",
"711058":"0",
"711066":"0",
"711074":"0",
"711082":"0",
"711104":"0",
"711112":"0",
"711120":"0",
"711139":"0",
"711147":"0",
"711155":"0",
"711163":"0",
"711171":"0",
"711180":"0",
"711198":"0",
"711201":"0",
"711210":"0",
"711228":"0",
"711236":"0",
"711244":"0",
"711252":"0",
"711260":"0",
"711279":"0",
"711287":"0",
"711295":"0",
"711309":"0",
"711317":"0",
"711325":"0",
"711333":"0",
"711341":"0",
"712011":"0",
"712038":"0",
"712046":"1",
"712054":"1",
"712062":"1",
"712070":"1",
"712076":"1",
"712089":"1",
"712097":"1",
"712100":"1",
"712119":"1",
"712127":"1",
"712135":"1",
"712143":"1",
"712151":"1",
"712160":"1",
"712178":"0",
"712186":"1",
"712194":"0",
"712216":"0",
"712224":"1",
"712232":"1",
"712240":"1",
"712259":"1",
"712267":"1",
"712275":"0",
"712283":"1",
"712291":"1",
"712305":"1",
"712313":"1",
"712321":"1",
"712330":"1",
"712992":"1",
"713034":"0",
"713069":"0",
"713077":"0",
"713085":"0",
"713107":"0",
"713115":"0",
"713131":"0",
"713204":"0",
"713212":"0",
"713220":"0",
"713239":"0",
"713247":"0",
"713255":"0",
"713263":"0",
"713271":"0",
"713280":"0",
"713298":"0",
"713301":"0",
"713310":"0",
"713999":"0",
"714014":"1",
"714022":"1",
"714030":"1",
"714049":"1",
"714080":"1",
"714081":"1",
"714090":"0",
"714111":"1",
"714120":"1",
"715010":"0",
"715029":"0",
"715037":"0",
"721015":"0",
"721023":"0",
"721031":"0",
"721040":"0",
"721058":"0",
"721066":"0",
"721074":"0",
"721082":"0",
"721104":"0",
"721112":"0",
"721120":"0",
"721139":"0",
"721147":"0",
"721155":"0",
"721163":"0",
"721171":"0",
"721180":"0",
"721198":"0",
"721201":"0",
"721210":"0",
"721228":"0",
"721236":"0",
"721244":"0",
"721252":"0",
"721260":"0",
"721279":"0",
"721287":"0",
"721295":"0",
"721309":"0",
"721317":"0",
"721325":"0",
"721333":"0",
"721341":"0",
"722011":"0",
"722038":"0",
"722046":"1",
"722054":"1",
"722062":"1",
"722070":"1",
"722076":"1",
"722089":"1",
"722097":"1",
"722100":"1",
"722119":"1",
"722127":"1",
"722135":"1",
"722143":"1",
"722151":"1",
"722160":"1",
"722178":"0",
"722186":"1",
"722194":"0",
"722216":"0",
"722224":"1",
"722232":"1",
"722240":"1",
"722259":"1",
"722267":"1",
"722275":"0",
"722283":"1",
"722291":"1",
"722305":"1",
"722313":"1",
"722321":"1",
"722330":"1",
"722992":"1",
"723034":"0",
"723069":"0",
"723077":"0",
"723085":"0",
"723107":"0",
"723115":"0",
"723131":"0",
"723204":"0",
"723212":"0",
"723220":"0",
"723239":"0",
"723247":"0",
"723255":"0",
"723263":"0",
"723271":"0",
"723280":"0",
"723298":"0",
"723301":"0",
"723310":"0",
"723999":"0",
"724014":"1",
"724022":"1",
"724030":"1",
"724049":"1",
"724080":"1",
"724081":"1",
"724090":"0",
"724111":"1",
"724120":"1",
"725010":"0",
"725029":"0",
"725037":"0",
"731015":"0",
"731023":"0",
"731031":"0",
"731040":"0",
"731058":"0",
"731066":"0",
"731074":"0",
"731082":"0",
"731104":"0",
"731112":"0",
"731120":"0",
"731139":"0",
"731147":"0",
"731155":"0",
"731163":"0",
"731171":"0",
"731180":"0",
"731198":"0",
"731201":"0",
"731210":"0",
"731228":"0",
"731236":"0",
"731244":"0",
"731252":"0",
"731260":"0",
"731279":"0",
"731287":"0",
"731295":"0",
"731309":"0",
"731317":"0",
"731325":"0",
"731333":"0",
"731341":"0",
"732011":"0",
"732038":"0",
"732046":"1",
"732054":"1",
"732062":"1",
"732070":"1",
"732076":"1",
"732089":"1",
"732097":"1",
"732100":"1",
"732119":"1",
"732127":"1",
"732135":"1",
"732143":"1",
"732151":"1",
"732160":"1",
"732178":"0",
"732186":"1",
"732194":"0",
"732216":"0",
"732224":"1",
"732232":"1",
"732240":"1",
"732259":"1",
"732267":"1",
"732275":"0",
"732283":"1",
"732291":"1",
"732305":"1",
"732313":"1",
"732321":"1",
"732330":"1",
"732992":"1",
"733034":"0",
"733069":"0",
"733077":"0",
"733085":"0",
"733107":"0",
"733115":"0",
"733131":"0",
"733204":"0",
"733212":"0",
"733220":"0",
"733239":"0",
"733247":"0",
"733255":"0",
"733263":"0",
"733271":"0",
"733280":"0",
"733298":"0",
"733301":"0",
"733310":"0",
"733999":"0",
"734014":"1",
"734022":"1",
"734030":"1",
"734049":"1",
"734080":"1",
"734081":"1",
"734090":"0",
"734111":"1",
"734120":"1",
"735010":"0",
"735029":"0",
"735037":"0",
"741015":"0",
"741023":"0",
"741031":"0",
"741040":"0",
"741058":"0",
"741066":"0",
"741074":"0",
"741082":"0",
"741104":"0",
"741112":"0",
"741120":"0",
"741139":"0",
"741147":"0",
"741155":"0",
"741163":"0",
"741171":"0",
"741180":"0",
"741198":"0",
"741201":"0",
"741210":"0",
"741228":"0",
"741236":"0",
"741244":"0",
"741252":"0",
"741260":"0",
"741279":"0",
"741287":"0",
"741295":"0",
"741309":"0",
"741317":"0",
"741325":"0",
"741333":"0",
"741341":"0",
"742011":"0",
"742038":"0",
"742046":"1",
"742054":"1",
"742062":"1",
"742070":"1",
"742076":"1",
"742089":"1",
"742097":"1",
"742100":"1",
"742119":"1",
"742127":"1",
"742135":"1",
"742143":"1",
"742151":"1",
"742160":"1",
"742178":"0",
"742186":"1",
"742194":"0",
"742216":"0",
"742224":"1",
"742232":"1",
"742240":"1",
"742259":"1",
"742267":"1",
"742275":"0",
"742283":"1",
"742291":"1",
"742305":"1",
"742313":"1",
"742321":"1",
"742330":"1",
"742992":"1",
"743034":"0",
"743069":"0",
"743077":"0",
"743085":"0",
"743107":"0",
"743115":"0",
"743131":"0",
"743204":"0",
"743212":"0",
"743220":"0",
"743239":"0",
"743247":"0",
"743255":"0",
"743263":"0",
"743271":"0",
"743280":"0",
"743298":"0",
"743301":"0",
"743310":"0",
"743999":"0",
"744014":"1",
"744022":"1",
"744030":"1",
"744049":"1",
"744080":"1",
"744081":"1",
"744090":"0",
"744111":"1",
"744120":"1",
"745010":"0",
"745029":"0",
"745037":"0",
"751015":"0",
"751023":"0",
"751031":"0",
"751040":"0",
"751058":"0",
"751066":"0",
"751074":"0",
"751082":"0",
"751104":"0",
"751112":"0",
"751120":"0",
"751139":"0",
"751147":"0",
"751155":"0",
"751163":"0",
"751171":"0",
"751180":"0",
"751198":"0",
"751201":"0",
"751210":"0",
"751228":"0",
"751236":"0",
"751244":"0",
"751252":"0",
"751260":"0",
"751279":"0",
"751287":"0",
"751295":"0",
"751309":"0",
"751317":"0",
"751325":"0",
"751333":"0",
"751341":"0",
"752011":"0",
"752038":"0",
"752046":"1",
"752054":"1",
"752062":"1",
"752070":"1",
"752076":"1",
"752089":"1",
"752097":"1",
"752100":"1",
"752119":"1",
"752127":"1",
"752135":"1",
"752143":"1",
"752151":"1",
"752160":"1",
"752178":"0",
"752186":"1",
"752194":"0",
"752216":"0",
"752224":"1",
"752232":"1",
"752240":"1",
"752259":"1",
"752267":"1",
"752275":"0",
"752283":"1",
"752291":"1",
"752305":"1",
"752313":"1",
"752321":"1",
"752330":"1",
"752992":"1",
"753034":"0",
"753069":"0",
"753077":"0",
"753085":"0",
"753107":"0",
"753115":"0",
"753131":"0",
"753204":"0",
"753212":"0",
"753220":"0",
"753239":"0",
"753247":"0",
"753255":"0",
"753263":"0",
"753271":"0",
"753280":"0",
"753298":"0",
"753301":"0",
"753310":"0",
"753999":"0",
"754014":"1",
"754022":"1",
"754030":"1",
"754049":"1",
"754080":"1",
"754081":"1",
"754090":"0",
"754111":"1",
"754120":"1",
"755010":"0",
"755029":"0",
"755037":"0",
"771015":"0",
"771023":"0",
"771031":"0",
"771040":"0",
"771058":"0",
"771066":"0",
"771074":"0",
"771082":"0",
"771104":"0",
"771112":"0",
"771120":"0",
"771139":"0",
"771147":"0",
"771155":"0",
"771163":"0",
"771171":"0",
"771180":"0",
"771198":"0",
"771201":"0",
"771210":"0",
"771228":"0",
"771236":"0",
"771244":"0",
"771252":"0",
"771260":"0",
"771279":"0",
"771287":"0",
"771295":"0",
"771309":"0",
"771317":"0",
"771325":"0",
"771333":"0",
"771341":"0",
"772011":"0",
"772038":"0",
"772046":"1",
"772054":"1",
"772062":"1",
"772070":"1",
"772076":"1",
"772089":"1",
"772097":"1",
"772100":"1",
"772119":"1",
"772127":"1",
"772135":"1",
"772143":"1",
"772151":"1",
"772160":"1",
"772178":"0",
"772186":"1",
"772194":"0",
"772216":"0",
"772224":"1",
"772232":"1",
"772240":"1",
"772259":"1",
"772267":"1",
"772275":"0",
"772283":"1",
"772291":"1",
"772305":"1",
"772313":"1",
"772321":"1",
"772330":"1",
"772992":"1",
"773034":"0",
"773069":"0",
"773077":"0",
"773085":"0",
"773107":"0",
"773115":"0",
"773131":"0",
"773204":"0",
"773212":"0",
"773220":"0",
"773239":"0",
"773247":"0",
"773255":"0",
"773263":"0",
"773271":"0",
"773280":"0",
"773298":"0",
"773301":"0",
"773310":"0",
"773999":"0",
"774014":"1",
"774022":"1",
"774030":"1",
"774049":"1",
"774080":"1",
"774081":"1",
"774090":"0",
"774111":"1",
"774120":"1",
"775010":"0",
"775029":"0",
"775037":"0",
"781015":"0",
"781023":"0",
"781031":"0",
"781040":"0",
"781058":"0",
"781066":"0",
"781074":"0",
"781082":"0",
"781104":"0",
"781112":"0",
"781120":"0",
"781139":"0",
"781147":"0",
"781155":"0",
"781163":"0",
"781171":"0",
"781180":"0",
"781198":"0",
"781201":"0",
"781210":"0",
"781228":"0",
"781236":"0",
"781244":"0",
"781252":"0",
"781260":"0",
"781279":"0",
"781287":"0",
"781295":"0",
"781309":"0",
"781317":"0",
"781325":"0",
"781333":"0",
"781341":"0",
"782011":"0",
"782038":"0",
"782046":"1",
"782054":"1",
"782062":"1",
"782070":"1",
"782076":"1",
"782089":"1",
"782097":"1",
"782100":"1",
"782119":"1",
"782127":"1",
"782135":"1",
"782143":"1",
"782151":"1",
"782160":"1",
"782178":"0",
"782186":"1",
"782194":"0",
"782216":"0",
"782224":"1",
"782232":"1",
"782240":"1",
"782259":"1",
"782267":"1",
"782275":"0",
"782283":"1",
"782291":"1",
"782305":"1",
"782313":"1",
"782321":"1",
"782330":"1",
"782992":"1",
"783034":"0",
"783069":"0",
"783077":"0",
"783085":"0",
"783107":"0",
"783115":"0",
"783131":"0",
"783204":"0",
"783212":"0",
"783220":"0",
"783239":"0",
"783247":"0",
"783255":"0",
"783263":"0",
"783271":"0",
"783280":"0",
"783298":"0",
"783301":"0",
"783310":"0",
"783999":"0",
"784014":"1",
"784022":"1",
"784030":"1",
"784049":"1",
"784080":"1",
"784081":"1",
"784090":"0",
"784111":"1",
"784120":"1",
"785010":"0",
"785029":"0",
"785037":"0",
"791015":"0",
"791023":"0",
"791031":"0",
"791040":"0",
"791058":"0",
"791066":"0",
"791074":"0",
"791082":"0",
"791104":"0",
"791112":"0",
"791120":"0",
"791139":"0",
"791147":"0",
"791155":"0",
"791163":"0",
"791171":"0",
"791180":"0",
"791198":"0",
"791201":"0",
"791210":"0",
"791228":"0",
"791236":"0",
"791244":"0",
"791252":"0",
"791260":"0",
"791279":"0",
"791287":"0",
"791295":"0",
"791309":"0",
"791317":"0",
"791325":"0",
"791333":"0",
"791341":"0",
"792011":"0",
"792038":"0",
"792046":"1",
"792054":"1",
"792062":"1",
"792070":"1",
"792076":"1",
"792089":"1",
"792097":"1",
"792100":"1",
"792119":"1",
"792127":"1",
"792135":"1",
"792143":"1",
"792151":"1",
"792160":"1",
"792178":"0",
"792186":"1",
"792194":"0",
"792216":"0",
"792224":"1",
"792232":"1",
"792240":"1",
"792259":"1",
"792267":"1",
"792275":"0",
"792283":"1",
"792291":"1",
"792305":"1",
"792313":"1",
"792321":"1",
"792330":"1",
"792992":"1",
"793034":"0",
"793069":"0",
"793077":"0",
"793085":"0",
"793107":"0",
"793115":"0",
"793131":"0",
"793204":"0",
"793212":"0",
"793220":"0",
"793239":"0",
"793247":"0",
"793255":"0",
"793263":"0",
"793271":"0",
"793280":"0",
"793298":"0",
"793301":"0",
"793310":"0",
"793999":"0",
"794014":"1",
"794022":"1",
"794030":"1",
"794049":"1",
"794080":"1",
"794081":"1",
"794090":"0",
"794111":"1",
"794120":"1",
"795010":"0",
"795029":"0",
"795037":"0",
"801015":"0",
"801023":"0",
"801031":"0",
"801040":"0",
"801058":"0",
"801066":"0",
"801074":"0",
"801082":"0",
"801104":"0",
"801112":"0",
"801120":"0",
"801139":"0",
"801147":"0",
"801155":"0",
"801163":"0",
"801171":"0",
"801180":"0",
"801198":"0",
"801201":"0",
"801210":"0",
"801228":"0",
"801236":"0",
"801244":"0",
"801252":"0",
"801260":"0",
"801279":"0",
"801287":"0",
"801295":"0",
"801309":"0",
"801317":"0",
"801325":"0",
"801333":"0",
"801341":"0",
"802011":"0",
"802038":"0",
"802046":"1",
"802054":"1",
"802062":"1",
"802070":"1",
"802076":"1",
"802089":"1",
"802097":"1",
"802100":"1",
"802119":"1",
"802127":"1",
"802135":"1",
"802143":"1",
"802151":"1",
"802160":"1",
"802178":"0",
"802186":"1",
"802194":"0",
"802216":"0",
"802224":"1",
"802232":"1",
"802240":"1",
"802259":"1",
"802267":"1",
"802275":"0",
"802283":"1",
"802291":"1",
"802305":"1",
"802313":"1",
"802321":"1",
"802330":"1",
"802992":"1",
"803034":"0",
"803069":"0",
"803077":"0",
"803085":"0",
"803107":"0",
"803115":"0",
"803131":"0",
"803204":"0",
"803212":"0",
"803220":"0",
"803239":"0",
"803247":"0",
"803255":"0",
"803263":"0",
"803271":"0",
"803280":"0",
"803298":"0",
"803301":"0",
"803310":"0",
"803999":"0",
"804014":"1",
"804022":"1",
"804030":"1",
"804049":"1",
"804080":"1",
"804081":"1",
"804090":"0",
"804111":"1",
"804120":"1",
"805010":"0",
"805029":"0",
"805037":"0",
"811015":"0",
"811023":"0",
"811031":"0",
"811040":"0",
"811058":"0",
"811066":"0",
"811074":"0",
"811082":"0",
"811104":"0",
"811112":"0",
"811120":"0",
"811139":"0",
"811147":"0",
"811155":"0",
"811163":"0",
"811171":"0",
"811180":"0",
"811198":"0",
"811201":"0",
"811210":"0",
"811228":"0",
"811236":"0",
"811244":"0",
"811252":"0",
"811260":"0",
"811279":"0",
"811287":"0",
"811295":"0",
"811309":"0",
"811317":"0",
"811325":"0",
"811333":"0",
"811341":"0",
"812011":"0",
"812038":"0",
"812046":"1",
"812054":"1",
"812062":"1",
"812070":"1",
"812076":"1",
"812089":"1",
"812097":"1",
"812100":"1",
"812119":"1",
"812127":"1",
"812135":"1",
"812143":"1",
"812151":"1",
"812160":"1",
"812178":"0",
"812186":"1",
"812194":"0",
"812216":"0",
"812224":"1",
"812232":"1",
"812240":"1",
"812259":"1",
"812267":"1",
"812275":"0",
"812283":"1",
"812291":"1",
"812305":"1",
"812313":"1",
"812321":"1",
"812330":"1",
"812992":"1",
"813034":"0",
"813069":"0",
"813077":"0",
"813085":"0",
"813107":"0",
"813115":"0",
"813131":"0",
"813204":"0",
"813212":"0",
"813220":"0",
"813239":"0",
"813247":"0",
"813255":"0",
"813263":"0",
"813271":"0",
"813280":"0",
"813298":"0",
"813301":"0",
"813310":"0",
"813999":"0",
"814014":"1",
"814022":"1",
"814030":"1",
"814049":"1",
"814080":"1",
"814081":"1",
"814090":"0",
"814111":"1",
"814120":"1",
"815010":"0",
"815029":"0",
"815037":"0",
"821015":"0",
"821023":"0",
"821031":"0",
"821040":"0",
"821058":"0",
"821066":"0",
"821074":"0",
"821082":"0",
"821104":"0",
"821112":"0",
"821120":"0",
"821139":"0",
"821147":"0",
"821155":"0",
"821163":"0",
"821171":"0",
"821180":"0",
"821198":"0",
"821201":"0",
"821210":"0",
"821228":"0",
"821236":"0",
"821244":"0",
"821252":"0",
"821260":"0",
"821279":"0",
"821287":"0",
"821295":"0",
"821309":"0",
"821317":"0",
"821325":"0",
"821333":"0",
"821341":"0",
"822011":"0",
"822038":"0",
"822046":"1",
"822054":"1",
"822062":"1",
"822070":"1",
"822076":"1",
"822089":"1",
"822097":"1",
"822100":"1",
"822119":"1",
"822127":"1",
"822135":"1",
"822143":"1",
"822151":"1",
"822160":"1",
"822178":"0",
"822186":"1",
"822194":"0",
"822216":"0",
"822224":"1",
"822232":"1",
"822240":"1",
"822259":"1",
"822267":"1",
"822275":"0",
"822283":"1",
"822291":"1",
"822305":"1",
"822313":"1",
"822321":"1",
"822330":"1",
"822992":"1",
"823034":"0",
"823069":"0",
"823077":"0",
"823085":"0",
"823107":"0",
"823115":"0",
"823131":"0",
"823204":"0",
"823212":"0",
"823220":"0",
"823239":"0",
"823247":"0",
"823255":"0",
"823263":"0",
"823271":"0",
"823280":"0",
"823298":"0",
"823301":"0",
"823310":"0",
"823999":"0",
"824014":"1",
"824022":"1",
"824030":"1",
"824049":"1",
"824080":"1",
"824081":"1",
"824090":"0",
"824111":"1",
"824120":"1",
"825010":"0",
"825029":"0",
"825037":"0",
"841015":"0",
"841023":"0",
"841031":"0",
"841040":"0",
"841058":"0",
"841066":"0",
"841074":"0",
"841082":"0",
"841104":"0",
"841112":"0",
"841120":"0",
"841139":"0",
"841147":"0",
"841155":"0",
"841163":"0",
"841171":"0",
"841180":"0",
"841198":"0",
"841201":"0",
"841210":"0",
"841228":"0",
"841236":"0",
"841244":"0",
"841252":"0",
"841260":"0",
"841279":"0",
"841287":"0",
"841295":"0",
"841309":"0",
"841317":"0",
"841325":"0",
"841333":"0",
"841341":"0",
"842011":"0",
"842038":"0",
"842046":"0",
"842054":"0",
"842062":"0",
"842070":"0",
"842076":"0",
"842089":"0",
"842097":"0",
"842100":"0",
"842119":"0",
"842127":"0",
"842135":"0",
"842143":"0",
"842151":"0",
"842160":"0",
"842178":"0",
"842186":"0",
"842194":"0",
"842216":"0",
"842224":"0",
"842232":"0",
"842240":"0",
"842259":"0",
"842267":"0",
"842275":"0",
"842283":"0",
"842291":"0",
"842305":"0",
"842313":"0",
"842321":"0",
"842330":"0",
"842992":"0",
"843034":"0",
"843069":"0",
"843077":"0",
"843085":"0",
"843107":"0",
"843115":"0",
"843131":"0",
"843204":"0",
"843212":"0",
"843220":"0",
"843239":"0",
"843247":"0",
"843255":"0",
"843263":"0",
"843271":"0",
"843280":"0",
"843298":"0",
"843301":"0",
"843310":"0",
"843999":"0",
"844014":"0",
"844022":"0",
"844030":"0",
"844049":"0",
"844080":"0",
"844081":"0",
"844090":"0",
"844111":"0",
"844120":"0",
"845010":"0",
"845029":"0",
"845037":"0",
"851015":"0",
"851023":"0",
"851031":"0",
"851040":"0",
"851058":"0",
"851066":"0",
"851074":"0",
"851082":"0",
"851104":"0",
"851112":"0",
"851120":"0",
"851139":"0",
"851147":"0",
"851155":"0",
"851163":"0",
"851171":"0",
"851180":"0",
"851198":"0",
"851201":"0",
"851210":"0",
"851228":"0",
"851236":"0",
"851244":"0",
"851252":"0",
"851260":"0",
"851279":"0",
"851287":"0",
"851295":"0",
"851309":"0",
"851317":"0",
"851325":"0",
"851333":"0",
"851341":"0",
"852011":"0",
"852038":"0",
"852046":"1",
"852054":"1",
"852062":"1",
"852070":"1",
"852076":"1",
"852089":"1",
"852097":"1",
"852100":"1",
"852119":"1",
"852127":"1",
"852135":"1",
"852143":"1",
"852151":"1",
"852160":"1",
"852178":"0",
"852186":"1",
"852194":"0",
"852216":"0",
"852224":"1",
"852232":"1",
"852240":"1",
"852259":"1",
"852267":"1",
"852275":"0",
"852283":"1",
"852291":"1",
"852305":"1",
"852313":"1",
"852321":"1",
"852330":"1",
"852992":"1",
"853034":"0",
"853069":"0",
"853077":"0",
"853085":"0",
"853107":"0",
"853115":"0",
"853131":"0",
"853204":"0",
"853212":"0",
"853220":"0",
"853239":"0",
"853247":"0",
"853255":"0",
"853263":"0",
"853271":"0",
"853280":"0",
"853298":"0",
"853301":"0",
"853310":"0",
"853999":"0",
"854014":"1",
"854022":"1",
"854030":"1",
"854049":"1",
"854080":"1",
"854081":"1",
"854090":"0",
"854111":"1",
"854120":"1",
"855010":"0",
"855029":"0",
"855037":"0",
"861015":"0",
"861023":"0",
"861031":"0",
"861040":"0",
"861058":"0",
"861066":"0",
"861074":"0",
"861082":"0",
"861104":"0",
"861112":"0",
"861120":"0",
"861139":"0",
"861147":"0",
"861155":"0",
"861163":"0",
"861171":"0",
"861180":"0",
"861198":"0",
"861201":"0",
"861210":"0",
"861228":"0",
"861236":"0",
"861244":"0",
"861252":"0",
"861260":"0",
"861279":"0",
"861287":"0",
"861295":"0",
"861309":"0",
"861317":"0",
"861325":"0",
"861333":"0",
"861341":"0",
"862011":"0",
"862038":"0",
"862046":"1",
"862054":"1",
"862062":"1",
"862070":"1",
"862076":"1",
"862089":"1",
"862097":"1",
"862100":"1",
"862119":"1",
"862127":"1",
"862135":"1",
"862143":"1",
"862151":"1",
"862160":"1",
"862178":"0",
"862186":"1",
"862194":"0",
"862216":"0",
"862224":"1",
"862232":"1",
"862240":"1",
"862259":"1",
"862267":"1",
"862275":"0",
"862283":"1",
"862291":"1",
"862305":"1",
"862313":"1",
"862321":"1",
"862330":"1",
"862992":"1",
"863034":"0",
"863069":"0",
"863077":"0",
"863085":"0",
"863107":"0",
"863115":"0",
"863131":"0",
"863204":"0",
"863212":"0",
"863220":"0",
"863239":"0",
"863247":"0",
"863255":"0",
"863263":"0",
"863271":"0",
"863280":"0",
"863298":"0",
"863301":"0",
"863310":"0",
"863999":"0",
"864014":"1",
"864022":"1",
"864030":"1",
"864049":"1",
"864080":"1",
"864081":"1",
"864090":"0",
"864111":"1",
"864120":"1",
"865010":"0",
"865029":"0",
"865037":"0",
"871015":"0",
"871023":"0",
"871031":"0",
"871040":"0",
"871058":"0",
"871066":"0",
"871074":"0",
"871082":"0",
"871104":"0",
"871112":"0",
"871120":"0",
"871139":"0",
"871147":"0",
"871155":"0",
"871163":"0",
"871171":"0",
"871180":"0",
"871198":"0",
"871201":"0",
"871210":"0",
"871228":"0",
"871236":"0",
"871244":"0",
"871252":"0",
"871260":"0",
"871279":"0",
"871287":"0",
"871295":"0",
"871309":"0",
"871317":"0",
"871325":"0",
"871333":"0",
"871341":"0",
"872011":"0",
"872038":"0",
"872046":"1",
"872054":"1",
"872062":"1",
"872070":"1",
"872076":"1",
"872089":"1",
"872097":"1",
"872100":"1",
"872119":"1",
"872127":"1",
"872135":"1",
"872143":"1",
"872151":"1",
"872160":"1",
"872178":"0",
"872186":"1",
"872194":"0",
"872216":"0",
"872224":"1",
"872232":"1",
"872240":"1",
"872259":"1",
"872267":"1",
"872275":"0",
"872283":"1",
"872291":"1",
"872305":"1",
"872313":"1",
"872321":"1",
"872330":"1",
"872992":"1",
"873034":"0",
"873069":"0",
"873077":"0",
"873085":"0",
"873107":"0",
"873115":"0",
"873131":"0",
"873204":"0",
"873212":"0",
"873220":"0",
"873239":"0",
"873247":"0",
"873255":"0",
"873263":"0",
"873271":"0",
"873280":"0",
"873298":"0",
"873301":"0",
"873310":"0",
"873999":"0",
"874014":"1",
"874022":"1",
"874030":"1",
"874049":"1",
"874080":"1",
"874081":"1",
"874090":"0",
"874111":"1",
"874120":"1",
"875010":"0",
"875029":"0",
"875037":"0",
"881015":"0",
"881023":"0",
"881031":"0",
"881040":"0",
"881058":"0",
"881066":"0",
"881074":"0",
"881082":"0",
"881104":"0",
"881112":"0",
"881120":"0",
"881139":"0",
"881147":"0",
"881155":"0",
"881163":"0",
"881171":"0",
"881180":"0",
"881198":"0",
"881201":"0",
"881210":"0",
"881228":"0",
"881236":"0",
"881244":"0",
"881252":"0",
"881260":"0",
"881279":"0",
"881287":"0",
"881295":"0",
"881309":"0",
"881317":"0",
"881325":"0",
"881333":"0",
"881341":"0",
"882011":"0",
"882038":"0",
"882046":"1",
"882054":"1",
"882062":"1",
"882070":"1",
"882076":"1",
"882089":"1",
"882097":"1",
"882100":"1",
"882119":"1",
"882127":"1",
"882135":"1",
"882143":"1",
"882151":"1",
"882160":"1",
"882178":"0",
"882186":"1",
"882194":"0",
"882216":"0",
"882224":"1",
"882232":"1",
"882240":"1",
"882259":"1",
"882267":"1",
"882275":"0",
"882283":"1",
"882291":"1",
"882305":"1",
"882313":"1",
"882321":"1",
"882330":"1",
"882992":"1",
"883034":"0",
"883069":"0",
"883077":"0",
"883085":"0",
"883107":"0",
"883115":"0",
"883131":"0",
"883204":"0",
"883212":"0",
"883220":"0",
"883239":"0",
"883247":"0",
"883255":"0",
"883263":"0",
"883271":"0",
"883280":"0",
"883298":"0",
"883301":"0",
"883310":"0",
"883999":"0",
"884014":"1",
"884022":"1",
"884030":"1",
"884049":"1",
"884080":"1",
"884081":"1",
"884090":"0",
"884111":"1",
"884120":"1",
"885010":"0",
"885029":"0",
"885037":"0",
"901015":"0",
"901023":"0",
"901031":"0",
"901040":"0",
"901058":"0",
"901066":"0",
"901074":"0",
"901082":"0",
"901104":"0",
"901112":"0",
"901120":"0",
"901139":"0",
"901147":"0",
"901155":"0",
"901163":"0",
"901171":"0",
"901180":"0",
"901198":"0",
"901201":"0",
"901210":"0",
"901228":"0",
"901236":"0",
"901244":"0",
"901252":"0",
"901260":"0",
"901279":"0",
"901287":"0",
"901295":"0",
"901309":"0",
"901317":"0",
"901325":"0",
"901333":"0",
"901341":"0",
"902011":"0",
"902038":"0",
"902046":"1",
"902054":"1",
"902062":"1",
"902070":"1",
"902076":"1",
"902089":"1",
"902097":"1",
"902100":"1",
"902119":"1",
"902127":"1",
"902135":"1",
"902143":"1",
"902151":"1",
"902160":"1",
"902178":"0",
"902186":"1",
"902194":"0",
"902216":"0",
"902224":"1",
"902232":"1",
"902240":"1",
"902259":"1",
"902267":"1",
"902275":"0",
"902283":"1",
"902291":"1",
"902305":"1",
"902313":"1",
"902321":"1",
"902330":"1",
"902992":"1",
"903034":"0",
"903069":"0",
"903077":"0",
"903085":"0",
"903107":"0",
"903115":"0",
"903131":"0",
"903204":"0",
"903212":"0",
"903220":"0",
"903239":"0",
"903247":"0",
"903255":"0",
"903263":"0",
"903271":"0",
"903280":"0",
"903298":"0",
"903301":"0",
"903310":"0",
"903999":"0",
"904014":"1",
"904022":"1",
"904030":"1",
"904049":"1",
"904080":"1",
"904081":"1",
"904090":"0",
"904111":"1",
"904120":"1",
"905010":"0",
"905029":"0",
"905037":"0",
"911015":"0",
"911023":"0",
"911031":"0",
"911040":"0",
"911058":"0",
"911066":"0",
"911074":"0",
"911082":"0",
"911104":"0",
"911112":"0",
"911120":"0",
"911139":"0",
"911147":"0",
"911155":"0",
"911163":"0",
"911171":"0",
"911180":"0",
"911198":"0",
"911201":"0",
"911210":"0",
"911228":"0",
"911236":"0",
"911244":"0",
"911252":"0",
"911260":"0",
"911279":"0",
"911287":"0",
"911295":"0",
"911309":"0",
"911317":"0",
"911325":"0",
"911333":"0",
"911341":"0",
"912011":"0",
"912038":"0",
"912046":"1",
"912054":"1",
"912062":"1",
"912070":"1",
"912076":"1",
"912089":"1",
"912097":"1",
"912100":"1",
"912119":"1",
"912127":"1",
"912135":"1",
"912143":"1",
"912151":"1",
"912160":"1",
"912178":"0",
"912186":"1",
"912194":"0",
"912216":"0",
"912224":"1",
"912232":"1",
"912240":"1",
"912259":"1",
"912267":"1",
"912275":"0",
"912283":"1",
"912291":"1",
"912305":"1",
"912313":"1",
"912321":"1",
"912330":"1",
"912992":"1",
"913034":"0",
"913069":"0",
"913077":"0",
"913085":"0",
"913107":"0",
"913115":"0",
"913131":"0",
"913204":"0",
"913212":"0",
"913220":"0",
"913239":"0",
"913247":"0",
"913255":"0",
"913263":"0",
"913271":"0",
"913280":"0",
"913298":"0",
"913301":"0",
"913310":"0",
"913999":"0",
"914014":"1",
"914022":"1",
"914030":"1",
"914049":"1",
"914080":"1",
"914081":"1",
"914090":"0",
"914111":"1",
"914120":"1",
"915010":"0",
"915029":"0",
"915037":"0",
"921015":"0",
"921023":"0",
"921031":"0",
"921040":"0",
"921058":"0",
"921066":"0",
"921074":"0",
"921082":"0",
"921104":"0",
"921112":"0",
"921120":"0",
"921139":"0",
"921147":"0",
"921155":"0",
"921163":"0",
"921171":"0",
"921180":"0",
"921198":"0",
"921201":"0",
"921210":"0",
"921228":"0",
"921236":"0",
"921244":"0",
"921252":"0",
"921260":"0",
"921279":"0",
"921287":"0",
"921295":"0",
"921309":"0",
"921317":"0",
"921325":"0",
"921333":"0",
"921341":"0",
"922011":"0",
"922038":"0",
"922046":"1",
"922054":"1",
"922062":"1",
"922070":"1",
"922076":"1",
"922089":"1",
"922097":"1",
"922100":"1",
"922119":"1",
"922127":"1",
"922135":"1",
"922143":"1",
"922151":"1",
"922160":"1",
"922178":"0",
"922186":"1",
"922194":"0",
"922216":"0",
"922224":"1",
"922232":"1",
"922240":"1",
"922259":"1",
"922267":"1",
"922275":"0",
"922283":"1",
"922291":"1",
"922305":"1",
"922313":"1",
"922321":"1",
"922330":"1",
"922992":"1",
"923034":"0",
"923069":"0",
"923077":"0",
"923085":"0",
"923107":"0",
"923115":"0",
"923131":"0",
"923204":"0",
"923212":"0",
"923220":"0",
"923239":"0",
"923247":"0",
"923255":"0",
"923263":"0",
"923271":"0",
"923280":"0",
"923298":"0",
"923301":"0",
"923310":"0",
"923999":"0",
"924014":"1",
"924022":"1",
"924030":"1",
"924049":"1",
"924080":"1",
"924081":"1",
"924090":"0",
"924111":"1",
"924120":"1",
"925010":"0",
"925029":"0",
"925037":"0",
"931015":"0",
"931023":"0",
"931031":"0",
"931040":"0",
"931058":"0",
"931066":"0",
"931074":"0",
"931082":"0",
"931104":"0",
"931112":"0",
"931120":"0",
"931139":"0",
"931147":"0",
"931155":"0",
"931163":"0",
"931171":"0",
"931180":"0",
"931198":"0",
"931201":"0",
"931210":"0",
"931228":"0",
"931236":"0",
"931244":"0",
"931252":"0",
"931260":"0",
"931279":"0",
"931287":"0",
"931295":"0",
"931309":"0",
"931317":"0",
"931325":"0",
"931333":"0",
"931341":"0",
"932011":"0",
"932038":"0",
"932046":"1",
"932054":"1",
"932062":"1",
"932070":"1",
"932076":"1",
"932089":"1",
"932097":"1",
"932100":"1",
"932119":"1",
"932127":"1",
"932135":"1",
"932143":"1",
"932151":"1",
"932160":"1",
"932178":"0",
"932186":"1",
"932194":"0",
"932216":"0",
"932224":"1",
"932232":"1",
"932240":"1",
"932259":"1",
"932267":"1",
"932275":"0",
"932283":"1",
"932291":"1",
"932305":"1",
"932313":"1",
"932321":"1",
"932330":"1",
"932992":"1",
"933034":"0",
"933069":"0",
"933077":"0",
"933085":"0",
"933107":"0",
"933115":"0",
"933131":"0",
"933204":"0",
"933212":"0",
"933220":"0",
"933239":"0",
"933247":"0",
"933255":"0",
"933263":"0",
"933271":"0",
"933280":"0",
"933298":"0",
"933301":"0",
"933310":"0",
"933999":"0",
"934014":"1",
"934022":"1",
"934030":"1",
"934049":"1",
"934080":"1",
"934081":"1",
"934090":"0",
"934111":"1",
"934120":"1",
"935010":"0",
"935029":"0",
"935037":"0",
"941015":"0",
"941023":"0",
"941031":"0",
"941040":"0",
"941058":"0",
"941066":"0",
"941074":"0",
"941082":"0",
"941104":"0",
"941112":"0",
"941120":"0",
"941139":"0",
"941147":"0",
"941155":"0",
"941163":"0",
"941171":"0",
"941180":"0",
"941198":"0",
"941201":"0",
"941210":"0",
"941228":"0",
"941236":"0",
"941244":"0",
"941252":"0",
"941260":"0",
"941279":"0",
"941287":"0",
"941295":"0",
"941309":"0",
"941317":"0",
"941325":"0",
"941333":"0",
"941341":"0",
"942011":"0",
"942038":"0",
"942046":"0",
"942054":"0",
"942062":"0",
"942070":"0",
"942076":"0",
"942089":"0",
"942097":"0",
"942100":"0",
"942119":"0",
"942127":"0",
"942135":"0",
"942143":"0",
"942151":"0",
"942160":"0",
"942178":"0",
"942186":"0",
"942194":"0",
"942216":"0",
"942224":"0",
"942232":"0",
"942240":"0",
"942259":"0",
"942267":"0",
"942275":"0",
"942283":"0",
"942291":"0",
"942305":"0",
"942313":"0",
"942321":"0",
"942330":"0",
"942992":"0",
"943034":"0",
"943069":"0",
"943077":"0",
"943085":"0",
"943107":"0",
"943115":"0",
"943131":"0",
"943204":"0",
"943212":"0",
"943220":"0",
"943239":"0",
"943247":"0",
"943255":"0",
"943263":"0",
"943271":"0",
"943280":"0",
"943298":"0",
"943301":"0",
"943310":"0",
"943999":"0",
"944014":"0",
"944022":"0",
"944030":"0",
"944049":"0",
"944080":"0",
"944081":"0",
"944090":"0",
"944111":"0",
"944120":"0",
"945010":"0",
"945029":"0",
"945037":"0",
"951015":"0",
"951023":"0",
"951031":"0",
"951040":"0",
"951058":"0",
"951066":"0",
"951074":"0",
"951082":"0",
"951104":"0",
"951112":"0",
"951120":"0",
"951139":"0",
"951147":"0",
"951155":"0",
"951163":"0",
"951171":"0",
"951180":"0",
"951198":"0",
"951201":"0",
"951210":"0",
"951228":"0",
"951236":"0",
"951244":"0",
"951252":"0",
"951260":"0",
"951279":"0",
"951287":"0",
"951295":"0",
"951309":"0",
"951317":"0",
"951325":"0",
"951333":"0",
"951341":"0",
"952011":"0",
"952038":"0",
"952046":"1",
"952054":"1",
"952062":"1",
"952070":"1",
"952076":"1",
"952089":"1",
"952097":"1",
"952100":"1",
"952119":"1",
"952127":"1",
"952135":"1",
"952143":"1",
"952151":"1",
"952160":"1",
"952178":"0",
"952186":"1",
"952194":"0",
"952216":"0",
"952224":"1",
"952232":"1",
"952240":"1",
"952259":"1",
"952267":"1",
"952275":"0",
"952283":"1",
"952291":"1",
"952305":"1",
"952313":"1",
"952321":"1",
"952330":"1",
"952992":"1",
"953034":"0",
"953069":"0",
"953077":"0",
"953085":"0",
"953107":"0",
"953115":"0",
"953131":"0",
"953204":"0",
"953212":"0",
"953220":"0",
"953239":"0",
"953247":"0",
"953255":"0",
"953263":"0",
"953271":"0",
"953280":"0",
"953298":"0",
"953301":"0",
"953310":"0",
"953999":"0",
"954014":"1",
"954022":"1",
"954030":"1",
"954049":"1",
"954080":"1",
"954081":"1",
"954090":"0",
"954111":"1",
"954120":"1",
"955010":"0",
"955029":"0",
"955037":"0",
"961015":"0",
"961023":"0",
"961031":"0",
"961040":"0",
"961058":"0",
"961066":"0",
"961074":"0",
"961082":"0",
"961104":"0",
"961112":"0",
"961120":"0",
"961139":"0",
"961147":"0",
"961155":"0",
"961163":"0",
"961171":"0",
"961180":"0",
"961198":"0",
"961201":"0",
"961210":"0",
"961228":"0",
"961236":"0",
"961244":"0",
"961252":"0",
"961260":"0",
"961279":"0",
"961287":"0",
"961295":"0",
"961309":"0",
"961317":"0",
"961325":"0",
"961333":"0",
"961341":"0",
"962011":"0",
"962038":"0",
"962046":"1",
"962054":"1",
"962062":"1",
"962070":"1",
"962076":"1",
"962089":"1",
"962097":"1",
"962100":"1",
"962119":"1",
"962127":"1",
"962135":"1",
"962143":"1",
"962151":"1",
"962160":"1",
"962178":"0",
"962186":"1",
"962194":"0",
"962216":"0",
"962224":"1",
"962232":"1",
"962240":"1",
"962259":"1",
"962267":"1",
"962275":"0",
"962283":"1",
"962291":"1",
"962305":"1",
"962313":"1",
"962321":"1",
"962330":"1",
"962992":"1",
"963034":"0",
"963069":"0",
"963077":"0",
"963085":"0",
"963107":"0",
"963115":"0",
"963131":"0",
"963204":"0",
"963212":"0",
"963220":"0",
"963239":"0",
"963247":"0",
"963255":"0",
"963263":"0",
"963271":"0",
"963280":"0",
"963298":"0",
"963301":"0",
"963310":"0",
"963999":"0",
"964014":"1",
"964022":"1",
"964030":"1",
"964049":"1",
"964080":"1",
"964081":"1",
"964090":"0",
"964111":"1",
"964120":"1",
"965010":"0",
"965029":"0",
"965037":"0",
"971015":"0",
"971023":"0",
"971031":"0",
"971040":"0",
"971058":"0",
"971066":"0",
"971074":"0",
"971082":"0",
"971104":"0",
"971112":"0",
"971120":"0",
"971139":"0",
"971147":"0",
"971155":"0",
"971163":"0",
"971171":"0",
"971180":"0",
"971198":"0",
"971201":"0",
"971210":"0",
"971228":"0",
"971236":"0",
"971244":"0",
"971252":"0",
"971260":"0",
"971279":"0",
"971287":"0",
"971295":"0",
"971309":"0",
"971317":"0",
"971325":"0",
"971333":"0",
"971341":"0",
"972011":"0",
"972038":"0",
"972046":"1",
"972054":"1",
"972062":"1",
"972070":"1",
"972076":"1",
"972089":"1",
"972097":"1",
"972100":"1",
"972119":"1",
"972127":"1",
"972135":"1",
"972143":"1",
"972151":"1",
"972160":"1",
"972178":"0",
"972186":"1",
"972194":"0",
"972216":"0",
"972224":"1",
"972232":"1",
"972240":"1",
"972259":"1",
"972267":"1",
"972275":"0",
"972283":"1",
"972291":"1",
"972305":"1",
"972313":"1",
"972321":"1",
"972330":"1",
"972992":"1",
"973034":"0",
"973069":"0",
"973077":"0",
"973085":"0",
"973107":"0",
"973115":"0",
"973131":"0",
"973204":"0",
"973212":"0",
"973220":"0",
"973239":"0",
"973247":"0",
"973255":"0",
"973263":"0",
"973271":"0",
"973280":"0",
"973298":"0",
"973301":"0",
"973310":"0",
"973999":"0",
"974014":"1",
"974022":"1",
"974030":"1",
"974049":"1",
"974080":"1",
"974081":"1",
"974090":"0",
"974111":"1",
"974120":"1",
"975010":"0",
"975029":"0",
"975037":"0",
"991015":"0",
"991023":"0",
"991031":"0",
"991040":"0",
"991058":"0",
"991066":"0",
"991074":"0",
"991082":"0",
"991104":"0",
"991112":"0",
"991120":"0",
"991139":"0",
"991147":"0",
"991155":"0",
"991163":"0",
"991171":"0",
"991180":"0",
"991198":"0",
"991201":"0",
"991210":"0",
"991228":"0",
"991236":"0",
"991244":"0",
"991252":"0",
"991260":"0",
"991279":"0",
"991287":"0",
"991295":"0",
"991309":"0",
"991317":"0",
"991325":"0",
"991333":"0",
"991341":"0",
"992011":"0",
"992038":"0",
"992046":"0",
"992054":"0",
"992062":"0",
"992070":"0",
"992076":"0",
"992089":"0",
"992097":"0",
"992100":"0",
"992119":"0",
"992127":"0",
"992135":"0",
"992143":"0",
"992151":"0",
"992160":"0",
"992178":"0",
"992186":"0",
"992194":"0",
"992216":"0",
"992224":"0",
"992232":"0",
"992240":"0",
"992259":"0",
"992267":"0",
"992275":"0",
"992283":"0",
"992291":"0",
"992305":"0",
"992313":"0",
"992321":"0",
"992330":"0",
"992992":"0",
"993034":"0",
"993069":"0",
"993077":"0",
"993085":"0",
"993107":"0",
"993115":"0",
"993131":"0",
"993204":"0",
"993212":"0",
"993220":"0",
"993239":"0",
"993247":"0",
"993255":"0",
"993263":"0",
"993271":"0",
"993280":"0",
"993298":"0",
"993301":"0",
"993310":"0",
"993999":"0",
"994014":"0",
"994022":"0",
"994030":"0",
"994049":"0",
"994080":"0",
"994081":"0",
"994090":"0",
"994111":"0",
"994120":"0",
"995010":"0",
"995029":"0",
"995037":"0",
"843123":"0", ##adicionei depois 
"865002":"0", ##adicionei depois
"013123":"0", ##adicionei depois
"943123":"0", ##adicionei depois
"414065":"0", ##adicionei depois
"014065":"0", ##adicionei depois
"434065":"0", ##adicionei depois
"433042":"0", ##adicionei depois
"823123":"0", ##adicionei depois
"973123":"0", ##adicionei depois
"109999":"1", ##adicionei depois
"949999":"0", ##adicionei depois
"729999":"1", ##adicionei depois
"949999":"0", ##adicionei depois
"469999":"1", ##adicionei depois
"629999":"1", ##adicionei depois
"859999":"1", ##adicionei depois
"939999":"1", ##adicionei depois
"479999":"1", ##adicionei depois
"329999":"1", ##adicionei depois
"974057":"0", ##adicionei depois
"813026":"0", ##adicionei depois
"943130":"0", ##adicionei depois
"424065":"0", ##adicionei depois
"943026":"0", ##adicionei depois
"943042":"0", ##adicionei depois
"943026":"0", ##adicionei depois
"943026":"0", ##adicionei depois
"943026":"0", ##adicionei depois
"943050":"0", ##adicionei depois
"863026":"0", ##adicionei depois
"015002":"0", ##adicionei depois
"943042":"0", ##adicionei depois
"943042":"0", ##adicionei depois
"943042":"0", ##adicionei depois
"933042":"0", ##adicionei depois
"933042":"0", ##adicionei depois
"013026":"0", ##adicionei depois
"863042":"0", ##adicionei depois
"863018":"0", ##adicionei depois
"843026":"0", ##adicionei depois
"713026":"0", ##adicionei depois
"474057":"0", ##adicionei depois
"453026":"0", ##adicionei depois
"434995":"0", ##adicionei depois
"424065":"0", ##adicionei depois
"975002":"0"  ##adicionei depois
}

# ":"0", ##adicionei depoisClassificação do IBGE que converte a cnae 1 para setor de ativ. econômica do ibge. Como a RAIS ESTAB. não tem setor ibge, vamos adicionar para criar o porte sebrae, que usa o setor ibge para sua padronização.    
dpara_cnae1_setoribge = {
"10006":"Extrativa Mineral",
"11100":"Extrativa Mineral",
"11207":"Extrativa Mineral",
"13102":"Extrativa Mineral",
"13218":"Extrativa Mineral",
"13226":"Extrativa Mineral",
"13234":"Extrativa Mineral",
"13242":"Extrativa Mineral",
"13250":"Extrativa Mineral",
"13293":"Extrativa Mineral",
"14109":"Extrativa Mineral",
"14214":"Extrativa Mineral",
"14222":"Extrativa Mineral",
"14290":"Extrativa Mineral",
"23302":"Indústria de Transformação",
"26115":"Indústria de Transformação",
"26123":"Indústria de Transformação",
"26190":"Indústria de Transformação",
"26204":"Indústria de Transformação",
"26301":"Indústria de Transformação",
"26417":"Indústria de Transformação",
"26425":"Indústria de Transformação",
"26492":"Indústria de Transformação",
"26913":"Indústria de Transformação",
"26921":"Indústria de Transformação",
"26999":"Indústria de Transformação",
"27111":"Indústria de Transformação",
"27120":"Indústria de Transformação",
"27138":"Indústria de Transformação",
"27146":"Indústria de Transformação",
"27219":"Indústria de Transformação",
"27227":"Indústria de Transformação",
"27235":"Indústria de Transformação",
"27243":"Indústria de Transformação",
"27251":"Indústria de Transformação",
"27260":"Indústria de Transformação",
"27294":"Indústria de Transformação",
"27316":"Indústria de Transformação",
"27391":"Indústria de Transformação",
"27413":"Indústria de Transformação",
"27421":"Indústria de Transformação",
"27499":"Indústria de Transformação",
"27510":"Indústria de Transformação",
"27529":"Indústria de Transformação",
"28118":"Indústria de Transformação",
"28126":"Indústria de Transformação",
"28134":"Indústria de Transformação",
"28215":"Indústria de Transformação",
"28223":"Indústria de Transformação",
"28312":"Indústria de Transformação",
"28320":"Indústria de Transformação",
"28339":"Indústria de Transformação",
"28347":"Indústria de Transformação",
"28398":"Indústria de Transformação",
"28410":"Indústria de Transformação",
"28428":"Indústria de Transformação",
"28436":"Indústria de Transformação",
"28819":"Indústria de Transformação",
"28827":"Indústria de Transformação",
"28916":"Indústria de Transformação",
"28924":"Indústria de Transformação",
"28932":"Indústria de Transformação",
"28991":"Indústria de Transformação",
"37109":"Indústria de Transformação",
"29114":"Indústria de Transformação",
"29122":"Indústria de Transformação",
"29130":"Indústria de Transformação",
"29149":"Indústria de Transformação",
"29157":"Indústria de Transformação",
"29211":"Indústria de Transformação",
"29220":"Indústria de Transformação",
"29238":"Indústria de Transformação",
"29246":"Indústria de Transformação",
"29254":"Indústria de Transformação",
"29297":"Indústria de Transformação",
"29319":"Indústria de Transformação",
"29327":"Indústria de Transformação",
"29408":"Indústria de Transformação",
"29513":"Indústria de Transformação",
"29521":"Indústria de Transformação",
"29530":"Indústria de Transformação",
"29548":"Indústria de Transformação",
"29610":"Indústria de Transformação",
"29629":"Indústria de Transformação",
"29637":"Indústria de Transformação",
"29645":"Indústria de Transformação",
"29653":"Indústria de Transformação",
"29696":"Indústria de Transformação",
"29718":"Indústria de Transformação",
"29726":"Indústria de Transformação",
"29815":"Indústria de Transformação",
"29890":"Indústria de Transformação",
"29912":"Indústria de Transformação",
"29920":"Indústria de Transformação",
"29939":"Indústria de Transformação",
"29947":"Indústria de Transformação",
"29955":"Indústria de Transformação",
"29963":"Indústria de Transformação",
"30112":"Indústria de Transformação",
"30120":"Indústria de Transformação",
"30210":"Indústria de Transformação",
"30228":"Indústria de Transformação",
"33502":"Indústria de Transformação",
"31119":"Indústria de Transformação",
"31127":"Indústria de Transformação",
"31135":"Indústria de Transformação",
"31216":"Indústria de Transformação",
"31224":"Indústria de Transformação",
"31305":"Indústria de Transformação",
"31410":"Indústria de Transformação",
"31429":"Indústria de Transformação",
"31518":"Indústria de Transformação",
"31526":"Indústria de Transformação",
"31607":"Indústria de Transformação",
"31810":"Indústria de Transformação",
"31828":"Indústria de Transformação",
"31917":"Indústria de Transformação",
"31925":"Indústria de Transformação",
"31992":"Indústria de Transformação",
"32107":"Indústria de Transformação",
"32212":"Indústria de Transformação",
"32220":"Indústria de Transformação",
"32301":"Indústria de Transformação",
"32905":"Indústria de Transformação",
"34100":"Indústria de Transformação",
"34207":"Indústria de Transformação",
"34312":"Indústria de Transformação",
"34320":"Indústria de Transformação",
"34398":"Indústria de Transformação",
"34410":"Indústria de Transformação",
"34428":"Indústria de Transformação",
"34436":"Indústria de Transformação",
"34444":"Indústria de Transformação",
"34495":"Indústria de Transformação",
"34509":"Indústria de Transformação",
"35114":"Indústria de Transformação",
"35122":"Indústria de Transformação",
"35211":"Indústria de Transformação",
"35220":"Indústria de Transformação",
"35238":"Indústria de Transformação",
"35319":"Indústria de Transformação",
"35327":"Indústria de Transformação",
"35912":"Indústria de Transformação",
"35920":"Indústria de Transformação",
"35998":"Indústria de Transformação",
"20109":"Indústria de Transformação",
"20214":"Indústria de Transformação",
"20222":"Indústria de Transformação",
"20230":"Indústria de Transformação",
"20290":"Indústria de Transformação",
"36110":"Indústria de Transformação",
"36129":"Indústria de Transformação",
"36145":"Indústria de Transformação",
"36978":"Indústria de Transformação",
"21105":"Indústria de Transformação",
"21210":"Indústria de Transformação",
"21229":"Indústria de Transformação",
"21318":"Indústria de Transformação",
"21326":"Indústria de Transformação",
"21415":"Indústria de Transformação",
"21423":"Indústria de Transformação",
"21490":"Indústria de Transformação",
"22110":"Indústria de Transformação",
"22128":"Indústria de Transformação",
"22136":"Indústria de Transformação",
"22144":"Indústria de Transformação",
"22152":"Indústria de Transformação",
"22160":"Indústria de Transformação",
"22179":"Indústria de Transformação",
"22187":"Indústria de Transformação",
"22195":"Indústria de Transformação",
"22217":"Indústria de Transformação",
"22225":"Indústria de Transformação",
"22292":"Indústria de Transformação",
"16004":"Indústria de Transformação",
"19100":"Indústria de Transformação",
"19216":"Indústria de Transformação",
"19291":"Indústria de Transformação",
"22314":"Indústria de Transformação",
"22322":"Indústria de Transformação",
"22330":"Indústria de Transformação",
"22349":"Indústria de Transformação",
"25119":"Indústria de Transformação",
"25127":"Indústria de Transformação",
"25194":"Indústria de Transformação",
"33103":"Indústria de Transformação",
"33200":"Indústria de Transformação",
"33308":"Indústria de Transformação",
"33405":"Indústria de Transformação",
"33502":"Indústria de Transformação",
"33910":"Indústria de Transformação",
"33928":"Indústria de Transformação",
"33936":"Indústria de Transformação",
"33944":"Indústria de Transformação",
"36919":"Indústria de Transformação",
"36927":"Indústria de Transformação",
"36935":"Indústria de Transformação",
"36943":"Indústria de Transformação",
"36951":"Indústria de Transformação",
"37206":"Indústria de Transformação",
"92118":"Indústria de Transformação",
"23108":"Indústria de Transformação",
"23205":"Indústria de Transformação",
"23213":"Indústria de Transformação",
"23299":"Indústria de Transformação",
"24112":"Indústria de Transformação",
"24120":"Indústria de Transformação",
"24139":"Indústria de Transformação",
"24147":"Indústria de Transformação",
"24198":"Indústria de Transformação",
"24210":"Indústria de Transformação",
"24228":"Indústria de Transformação",
"24295":"Indústria de Transformação",
"24317":"Indústria de Transformação",
"24325":"Indústria de Transformação",
"24333":"Indústria de Transformação",
"24414":"Indústria de Transformação",
"24422":"Indústria de Transformação",
"24511":"Indústria de Transformação",
"24520":"Indústria de Transformação",
"24538":"Indústria de Transformação",
"24546":"Indústria de Transformação",
"24619":"Indústria de Transformação",
"24627":"Indústria de Transformação",
"24635":"Indústria de Transformação",
"24694":"Indústria de Transformação",
"24716":"Indústria de Transformação",
"24724":"Indústria de Transformação",
"24732":"Indústria de Transformação",
"24813":"Indústria de Transformação",
"24821":"Indústria de Transformação",
"24830":"Indústria de Transformação",
"24910":"Indústria de Transformação",
"24929":"Indústria de Transformação",
"24937":"Indústria de Transformação",
"24945":"Indústria de Transformação",
"24953":"Indústria de Transformação",
"24961":"Indústria de Transformação",
"24996":"Indústria de Transformação",
"25216":"Indústria de Transformação",
"25224":"Indústria de Transformação",
"25291":"Indústria de Transformação",
"36994":"Indústria de Transformação",
"17116":"Indústria de Transformação",
"17191":"Indústria de Transformação",
"17213":"Indústria de Transformação",
"17221":"Indústria de Transformação",
"17230":"Indústria de Transformação",
"17248":"Indústria de Transformação",
"17310":"Indústria de Transformação",
"17329":"Indústria de Transformação",
"17337":"Indústria de Transformação",
"17418":"Indústria de Transformação",
"17493":"Indústria de Transformação",
"17507":"Indústria de Transformação",
"17612":"Indústria de Transformação",
"17620":"Indústria de Transformação",
"17639":"Indústria de Transformação",
"17647":"Indústria de Transformação",
"17698":"Indústria de Transformação",
"17710":"Indústria de Transformação",
"17728":"Indústria de Transformação",
"17795":"Indústria de Transformação",
"18112":"Indústria de Transformação",
"18120":"Indústria de Transformação",
"18139":"Indústria de Transformação",
"18210":"Indústria de Transformação",
"18228":"Indústria de Transformação",
"36960":"Indústria de Transformação",
"19313":"Indústria de Transformação",
"19321":"Indústria de Transformação",
"19330":"Indústria de Transformação",
"19399":"Indústria de Transformação",
"15113":"Indústria de Transformação",
"15121":"Indústria de Transformação",
"15130":"Indústria de Transformação",
"15148":"Indústria de Transformação",
"15210":"Indústria de Transformação",
"15229":"Indústria de Transformação",
"15237":"Indústria de Transformação",
"15318":"Indústria de Transformação",
"15326":"Indústria de Transformação",
"15334":"Indústria de Transformação",
"15415":"Indústria de Transformação",
"15423":"Indústria de Transformação",
"15431":"Indústria de Transformação",
"15512":"Indústria de Transformação",
"15520":"Indústria de Transformação",
"15539":"Indústria de Transformação",
"15547":"Indústria de Transformação",
"15555":"Indústria de Transformação",
"15563":"Indústria de Transformação",
"15598":"Indústria de Transformação",
"15610":"Indústria de Transformação",
"15628":"Indústria de Transformação",
"15717":"Indústria de Transformação",
"15725":"Indústria de Transformação",
"15814":"Indústria de Transformação",
"15822":"Indústria de Transformação",
"15830":"Indústria de Transformação",
"15849":"Indústria de Transformação",
"15857":"Indústria de Transformação",
"15865":"Indústria de Transformação",
"15890":"Indústria de Transformação",
"15911":"Indústria de Transformação",
"15920":"Indústria de Transformação",
"15938":"Indústria de Transformação",
"15946":"Indústria de Transformação",
"15954":"Indústria de Transformação",
"23400":"Indústria de Transformação",
"55247":"Indústria de Transformação",
"40100":"Serviços Industriais de Utilidade Pública",
"40118":"Serviços Industriais de Utilidade Pública",
"40126":"Serviços Industriais de Utilidade Pública",
"40134":"Serviços Industriais de Utilidade Pública",
"40142":"Serviços Industriais de Utilidade Pública",
"40207":"Serviços Industriais de Utilidade Pública",
"40304":"Serviços Industriais de Utilidade Pública",
"41009":"Serviços Industriais de Utilidade Pública",
"90000":"Serviços Industriais de Utilidade Pública",
"45110":"Construção Civil",
"45128":"Construção Civil",
"45136":"Construção Civil",
"45217":"Construção Civil",
"45225":"Construção Civil",
"45233":"Construção Civil",
"45241":"Construção Civil",
"45250":"Construção Civil",
"45292":"Construção Civil",
"45314":"Construção Civil",
"45322":"Construção Civil",
"45330":"Construção Civil",
"45349":"Construção Civil",
"45411":"Construção Civil",
"45420":"Construção Civil",
"45438":"Construção Civil",
"45497":"Construção Civil",
"45500":"Construção Civil",
"45519":"Construção Civil",
"45527":"Construção Civil",
"45594":"Construção Civil",
"45608":"Construção Civil",
"50105":"Comércio",
"50202":"Comércio",
"50300":"Comércio",
"50415":"Comércio",
"50423":"Comércio",
"50504":"Comércio",
"52116":"Comércio",
"52124":"Comércio",
"52132":"Comércio",
"52140":"Comércio",
"52159":"Comércio",
"52213":"Comércio",
"52221":"Comércio",
"52230":"Comércio",
"52248":"Comércio",
"52299":"Comércio",
"52310":"Comércio",
"52329":"Comércio",
"52337":"Comércio",
"52418":"Comércio",
"52426":"Comércio",
"52434":"Comércio",
"52442":"Comércio",
"52450":"Comércio",
"52469":"Comércio",
"52477":"Comércio",
"52493":"Comércio",
"52507":"Comércio",
"52612":"Comércio",
"52620":"Comércio",
"52698":"Comércio",
"52710":"Comércio",
"52728":"Comércio",
"52795":"Comércio",
"51110":"Comércio",
"51128":"Comércio",
"51136":"Comércio",
"51144":"Comércio",
"51152":"Comércio",
"51160":"Comércio",
"51179":"Comércio",
"51187":"Comércio",
"51195":"Comércio",
"51217":"Comércio",
"51225":"Comércio",
"51314":"Comércio",
"51322":"Comércio",
"51330":"Comércio",
"51349":"Comércio",
"51357":"Comércio",
"51365":"Comércio",
"51373":"Comércio",
"51390":"Comércio",
"51411":"Comércio",
"51420":"Comércio",
"51438":"Comércio",
"51446":"Comércio",
"51454":"Comércio",
"51462":"Comércio",
"51470":"Comércio",
"51497":"Comércio",
"51519":"Comércio",
"51527":"Comércio",
"51535":"Comércio",
"51543":"Comércio",
"51551":"Comércio",
"51594":"Comércio",
"51616":"Comércio",
"51624":"Comércio",
"51632":"Comércio",
"51640":"Comércio",
"51659":"Comércio",
"51691":"Comércio",
"51918":"Comércio",
"51926":"Comércio",
"65102":"Serviços",
"65218":"Serviços",
"65226":"Serviços",
"65234":"Serviços",
"65242":"Serviços",
"65315":"Serviços",
"65323":"Serviços",
"65331":"Serviços",
"65340":"Serviços",
"65358":"Serviços",
"65404":"Serviços",
"65510":"Serviços",
"65595":"Serviços",
"65919":"Serviços",
"65927":"Serviços",
"65935":"Serviços",
"65994":"Serviços",
"66117":"Serviços",
"66125":"Serviços",
"66133":"Serviços",
"66214":"Serviços",
"66222":"Serviços",
"66303":"Serviços",
"67113":"Serviços",
"67121":"Serviços",
"67199":"Serviços",
"67202":"Serviços",
"2135":"Serviços",
"22314":"Serviços",
"22322":"Serviços",
"22330":"Serviços",
"22349":"Serviços",
"51110":"Serviços",
"51128":"Serviços",
"51136":"Serviços",
"51144":"Serviços",
"51152":"Serviços",
"51160":"Serviços",
"51179":"Serviços",
"51187":"Serviços",
"51195":"Serviços",
"63118":"Serviços",
"63126":"Serviços",
"63215":"Serviços",
"63231":"Serviços",
"63304":"Serviços",
"63401":"Serviços",
"67113":"Serviços",
"67121":"Serviços",
"67199":"Serviços",
"67202":"Serviços",
"70106":"Serviços",
"70203":"Serviços",
"70319":"Serviços",
"70327":"Serviços",
"70408":"Serviços",
"71102":"Serviços",
"71218":"Serviços",
"71226":"Serviços",
"71234":"Serviços",
"71315":"Serviços",
"71323":"Serviços",
"71331":"Serviços",
"71390":"Serviços",
"71404":"Serviços",
"72109":"Serviços",
"72206":"Serviços",
"72214":"Serviços",
"72290":"Serviços",
"72303":"Serviços",
"72400":"Serviços",
"72508":"Serviços",
"72907":"Serviços",
"73105":"Serviços",
"73202":"Serviços",
"74110":"Serviços",
"74128":"Serviços",
"74136":"Serviços",
"74144":"Serviços",
"74152":"Serviços",
"74160":"Serviços",
"74209":"Serviços",
"74306":"Serviços",
"74403":"Serviços",
"74500":"Serviços",
"74608":"Serviços",
"74705":"Serviços",
"74918":"Serviços",
"74926":"Serviços",
"74993":"Serviços",
"92401":"Serviços",
"99007":"Serviços",
"60100":"Serviços",
"60216":"Serviços",
"60224":"Serviços",
"60232":"Serviços",
"60240":"Serviços",
"60259":"Serviços",
"60267":"Serviços",
"60275":"Serviços",
"60283":"Serviços",
"60291":"Serviços",
"60305":"Serviços",
"61115":"Serviços",
"61123":"Serviços",
"61212":"Serviços",
"61220":"Serviços",
"61239":"Serviços",
"62103":"Serviços",
"62200":"Serviços",
"62308":"Serviços",
"63118":"Serviços",
"63126":"Serviços",
"63215":"Serviços",
"63223":"Serviços",
"63231":"Serviços",
"63304":"Serviços",
"63401":"Serviços",
"64114":"Serviços",
"64122":"Serviços",
"64203":"Serviços",
"31895":"Serviços",
"36137":"Serviços",
"50202":"Serviços",
"50423":"Serviços",
"52710":"Serviços",
"52728":"Serviços",
"52795":"Serviços",
"55115":"Serviços",
"55123":"Serviços",
"55131":"Serviços",
"55190":"Serviços",
"55212":"Serviços",
"55220":"Serviços",
"55239":"Serviços",
"55247":"Serviços",
"55298":"Serviços",
"66214":"Serviços",
"66222":"Serviços",
"70408":"Serviços",
"71404":"Serviços",
"74705":"Serviços",
"74918":"Serviços",
#"75302":"Serviços", #RETIREI E MANTIVE COMO ADM PUBLICA (SERV DE SEGURIDADE SOCIAL)
"85316":"Serviços",
"85324":"Serviços",
"90000":"Serviços",
"91111":"Serviços",
"91120":"Serviços",
"91200":"Serviços",
"91910":"Serviços",
"91928":"Serviços",
"91995":"Serviços",
"92118":"Serviços",
"92126":"Serviços",
"92134":"Serviços",
"92215":"Serviços",
"92223":"Serviços",
"92312":"Serviços",
"92320":"Serviços",
"92398":"Serviços",
"92401":"Serviços",
"92517":"Serviços",
"92525":"Serviços",
"92533":"Serviços",
"92614":"Serviços",
"92622":"Serviços",
"93017":"Serviços",
"93025":"Serviços",
"93033":"Serviços",
"93041":"Serviços",
"93092":"Serviços",
"95001":"Serviços",
"85111":"Serviços",
"85120":"Serviços",
"85138":"Serviços",
"85146":"Serviços",
"85154":"Serviços",
"85162":"Serviços",
"85200":"Serviços",
"80110":"Serviços",
"80128":"Serviços",
"80136":"Serviços",
"80144":"Serviços",
"80152":"Serviços",
"80209":"Serviços",
"80217":"Serviços",
"80225":"Serviços",
"80306":"Serviços",
"80314":"Serviços",
"80322":"Serviços",
"80330":"Serviços",
"80918":"Serviços",
"80926":"Serviços",
"80934":"Serviços",
"80942":"Serviços",
"80950":"Serviços",
"80969":"Serviços",
"80977":"Serviços",
"80993":"Serviços",
"75116":"Administração Pública",
"75124":"Administração Pública",
"75132":"Administração Pública",
"75140":"Administração Pública",
"75213":"Administração Pública",
"75221":"Administração Pública",
"75230":"Administração Pública",
"75248":"Administração Pública",
"75256":"Administração Pública",
"75302":"Administração Pública",
"01112":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01120":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01139":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01147":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01155":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01198":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01210":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01228":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01317":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01325":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01333":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01341":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01392":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01414":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01422":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01430":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01449":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01457":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01465":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01503":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01619":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01627":"Agropecuária, Extração Vegetal, Caça e Pesca",
"01708":"Agropecuária, Extração Vegetal, Caça e Pesca",
"02119":"Agropecuária, Extração Vegetal, Caça e Pesca",
"02127":"Agropecuária, Extração Vegetal, Caça e Pesca",
"05118":"Agropecuária, Extração Vegetal, Caça e Pesca",
"05126":"Agropecuária, Extração Vegetal, Caça e Pesca",
"02135": "Serviços", ##adicionei depois
"-1":"-1"
}

# Classificação do Sebrae sobre o porte da empresa com base no setor do IBGE
pad_sebrae_porte = {
"Agropecuária, Extração Vegetal, Caça e Pesca1":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca10":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca2":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca4":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca7":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca5":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca8":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca3":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca6":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca9":"M/G",
"Comércio1":"Micro",
"Comércio10":"Grande",
"Comércio2":"Micro",
"Comércio4":"Pequena",
"Comércio7":"Grande",
"Comércio5":"Pequena",
"Comércio8":"Grande",
"Comércio3":"Micro",
"Comércio6":"Média",
"Comércio9":"Grande",
"Construção Civil1":"Micro",
"Construção Civil10":"Grande",
"Construção Civil2":"Micro",
"Construção Civil4":"Micro",
"Construção Civil7":"Média",
"Construção Civil5":"Pequena",
"Construção Civil8":"Média",
"Construção Civil3":"Micro",
"Construção Civil6":"Pequena",
"Construção Civil9":"Grande",
"Extrativa Mineral1":"Micro",
"Extrativa Mineral10":"Grande",
"Extrativa Mineral2":"Micro",
"Extrativa Mineral4":"Micro",
"Extrativa Mineral7":"Média",
"Extrativa Mineral5":"Pequena",
"Extrativa Mineral8":"Média",
"Extrativa Mineral3":"Micro",
"Extrativa Mineral6":"Pequena",
"Extrativa Mineral9":"Grande",
"Indústria de Transformação1":"Micro",
"Indústria de Transformação10":"Grande",
"Indústria de Transformação2":"Micro",
"Indústria de Transformação4":"Micro",
"Indústria de Transformação7":"Média",
"Indústria de Transformação5":"Pequena",
"Indústria de Transformação8":"Média",
"Indústria de Transformação3":"Micro",
"Indústria de Transformação6":"Pequena",
"Indústria de Transformação9":"Grande",
"Serviços Industriais de Utilidade Pública1":"Micro",
"Serviços Industriais de Utilidade Pública10":"Grande",
"Serviços Industriais de Utilidade Pública2":"Micro",
"Serviços Industriais de Utilidade Pública4":"Micro",
"Serviços Industriais de Utilidade Pública7":"Média",
"Serviços Industriais de Utilidade Pública5":"Pequena",
"Serviços Industriais de Utilidade Pública8":"Média",
"Serviços Industriais de Utilidade Pública3":"Micro",
"Serviços Industriais de Utilidade Pública6":"Pequena",
"Serviços Industriais de Utilidade Pública9":"Grande",
"Serviços1":"Micro",
"Serviços10":"Grande",
"Serviços2":"Micro",
"Serviços4":"Pequena",
"Serviços7":"Grande",
"Serviços5":"Pequena",
"Serviços8":"Grande",
"Serviços3":"Micro",
"Serviços6":"Média",
"Serviços9":"Grande",
"Administração Pública1":"Não se aplica",
"Administração Pública10":"Não se aplica",
"Administração Pública2":"Não se aplica",
"Administração Pública4":"Não se aplica",
"Administração Pública7":"Não se aplica",
"Administração Pública5":"Não se aplica",
"Administração Pública8":"Não se aplica",
"Administração Pública3":"Não se aplica",
"Administração Pública6":"Não se aplica",
"Administração Pública9":"Não se aplica",
"-11":"-1",
"-12":"-1",
"-13":"-1",
"-14":"-1",
"-15":"-1",
"-16":"-1",
"-17":"-1",
"-18":"-1",
"-19":"-1",
"-110":"-1",
"-1-1":"-1",
"Administração Pública-1":"Não se aplica",
"Serviços Industriais de Utilidade Pública-1":"-1",
"Serviços-1":"-1",
"Extrativa Mineral-1":"-1",
"Indústria de Transformação-1":"-1",
"Comércio-1":"-1",
"Construção Civil-1":"-1",
"Agropecuária, Extração Vegetal, Caça e Pesca-1":"-1"
}

# Classificação do Sebrae sobre o setor da empresa com base na CNAE 2 SUBCLASSE
pad_sebrae_setor = {
"111301":"Agropecuária",
"111302":"Agropecuária",
"111303":"Agropecuária",
"111399":"Agropecuária",
"112101":"Agropecuária",
"112102":"Agropecuária",
"112199":"Agropecuária",
"113000":"Agropecuária",
"114800":"Agropecuária",
"115600":"Agropecuária",
"116401":"Agropecuária",
"116402":"Agropecuária",
"116403":"Agropecuária",
"116499":"Agropecuária",
"119901":"Agropecuária",
"119902":"Agropecuária",
"119903":"Agropecuária",
"119904":"Agropecuária",
"119905":"Agropecuária",
"119906":"Agropecuária",
"119907":"Agropecuária",
"119908":"Agropecuária",
"119909":"Agropecuária",
"119999":"Agropecuária",
"121101":"Agropecuária",
"121102":"Agropecuária",
"122900":"Agropecuária",
"131800":"Agropecuária",
"132600":"Agropecuária",
"133401":"Agropecuária",
"133402":"Agropecuária",
"133403":"Agropecuária",
"133404":"Agropecuária",
"133405":"Agropecuária",
"133406":"Agropecuária",
"133407":"Agropecuária",
"133408":"Agropecuária",
"133409":"Agropecuária",
"133410":"Agropecuária",
"133411":"Agropecuária",
"133499":"Agropecuária",
"134200":"Agropecuária",
"135100":"Agropecuária",
"139301":"Agropecuária",
"139302":"Agropecuária",
"139303":"Agropecuária",
"139304":"Agropecuária",
"139305":"Agropecuária",
"139306":"Agropecuária",
"139399":"Agropecuária",
"141501":"Agropecuária",
"141502":"Agropecuária",
"142300":"Agropecuária",
"151201":"Agropecuária",
"151202":"Agropecuária",
"151203":"Agropecuária",
"152101":"Agropecuária",
"152102":"Agropecuária",
"152103":"Agropecuária",
"153901":"Agropecuária",
"153902":"Agropecuária",
"154700":"Agropecuária",
"155501":"Agropecuária",
"155502":"Agropecuária",
"155503":"Agropecuária",
"155504":"Agropecuária",
"155505":"Agropecuária",
"159801":"Agropecuária",
"159802":"Agropecuária",
"159803":"Agropecuária",
"159804":"Agropecuária",
"159899":"Agropecuária",
"161001":"Agropecuária",
"161002":"Agropecuária",
"161003":"Agropecuária",
"161099":"Agropecuária",
"162801":"Agropecuária",
"162802":"Agropecuária",
"162803":"Agropecuária",
"162899":"Agropecuária",
"163600":"Agropecuária",
"170900":"Agropecuária",
"210101":"Agropecuária",
"210102":"Agropecuária",
"210103":"Agropecuária",
"210104":"Agropecuária",
"210105":"Agropecuária",
"210106":"Agropecuária",
"210107":"Agropecuária",
"210108":"Agropecuária",
"210109":"Agropecuária",
"210199":"Agropecuária",
"220901":"Agropecuária",
"220902":"Agropecuária",
"220903":"Agropecuária",
"220904":"Agropecuária",
"220905":"Agropecuária",
"220906":"Agropecuária",
"220999":"Agropecuária",
"230600":"Serviços",
"311601":"Agropecuária",
"311602":"Agropecuária",
"311603":"Agropecuária",
"311604":"Agropecuária",
"312401":"Agropecuária",
"312402":"Agropecuária",
"312403":"Agropecuária",
"312404":"Agropecuária",
"321301":"Agropecuária",
"321302":"Agropecuária",
"321303":"Agropecuária",
"321304":"Agropecuária",
"321305":"Agropecuária",
"321399":"Agropecuária",
"322101":"Agropecuária",
"322102":"Agropecuária",
"322103":"Agropecuária",
"322104":"Agropecuária",
"322105":"Agropecuária",
"322106":"Agropecuária",
"322107":"Agropecuária",
"322199":"Agropecuária",
"500301":"Indústria",
"500302":"Indústria",
"600001":"Indústria",
"600002":"Indústria",
"600003":"Indústria",
"710301":"Indústria",
"710302":"Indústria",
"721901":"Indústria",
"721902":"Indústria",
"722701":"Indústria",
"722702":"Indústria",
"723501":"Indústria",
"723502":"Indústria",
"724301":"Indústria",
"724302":"Indústria",
"725100":"Indústria",
"729401":"Indústria",
"729402":"Indústria",
"729403":"Indústria",
"729404":"Indústria",
"729405":"Indústria",
"810001":"Indústria",
"810002":"Indústria",
"810003":"Indústria",
"810004":"Indústria",
"810005":"Indústria",
"810006":"Indústria",
"810007":"Indústria",
"810008":"Indústria",
"810009":"Indústria",
"810010":"Indústria",
"810099":"Indústria",
"891600":"Indústria",
"892401":"Indústria",
"892402":"Indústria",
"892403":"Indústria",
"893200":"Indústria",
"899101":"Indústria",
"899102":"Indústria",
"899103":"Indústria",
"899199":"Indústria",
"910600":"Indústria",
"990401":"Indústria",
"990402":"Indústria",
"990403":"Indústria",
"1011201":"Indústria",
"1011202":"Indústria",
"1011203":"Indústria",
"1011204":"Indústria",
"1011205":"Indústria",
"1012101":"Indústria",
"1012102":"Indústria",
"1012103":"Indústria",
"1012104":"Indústria",
"1013901":"Indústria",
"1013902":"Indústria",
"1020101":"Indústria",
"1020102":"Indústria",
"1031700":"Indústria",
"1032501":"Indústria",
"1032599":"Indústria",
"1033301":"Indústria",
"1033302":"Indústria",
"1041400":"Indústria",
"1042200":"Indústria",
"1043100":"Indústria",
"1051100":"Indústria",
"1052000":"Indústria",
"1053800":"Indústria",
"1061901":"Indústria",
"1061902":"Indústria",
"1062700":"Indústria",
"1063500":"Indústria",
"1064300":"Indústria",
"1065101":"Indústria",
"1065102":"Indústria",
"1065103":"Indústria",
"1066000":"Indústria",
"1069400":"Indústria",
"1071600":"Indústria",
"1072401":"Indústria",
"1072402":"Indústria",
"1081301":"Indústria",
"1081302":"Indústria",
"1082100":"Indústria",
"1091100":"Indústria",
"1091101":"Indústria",
"1091102":"Indústria",
"1092900":"Indústria",
"1093701":"Indústria",
"1093702":"Indústria",
"1094500":"Indústria",
"1095300":"Indústria",
"1096100":"Indústria",
"1099601":"Indústria",
"1099602":"Indústria",
"1099603":"Indústria",
"1099604":"Indústria",
"1099605":"Indústria",
"1099606":"Indústria",
"1099607":"Indústria",
"1099699":"Indústria",
"1111901":"Indústria",
"1111902":"Indústria",
"1112700":"Indústria",
"1113501":"Indústria",
"1113502":"Indústria",
"1121600":"Indústria",
"1122401":"Indústria",
"1122402":"Indústria",
"1122403":"Indústria",
"1122404":"Indústria",
"1122499":"Indústria",
"1210700":"Indústria",
"1220401":"Indústria",
"1220402":"Indústria",
"1220403":"Indústria",
"1220499":"Indústria",
"1311100":"Indústria",
"1312000":"Indústria",
"1313800":"Indústria",
"1314600":"Indústria",
"1321900":"Indústria",
"1322700":"Indústria",
"1323500":"Indústria",
"1330800":"Indústria",
"1340501":"Indústria",
"1340502":"Indústria",
"1340599":"Indústria",
"1351100":"Indústria",
"1352900":"Indústria",
"1353700":"Indústria",
"1354500":"Indústria",
"1359600":"Indústria",
"1411801":"Indústria",
"1411802":"Indústria",
"1412601":"Indústria",
"1412602":"Indústria",
"1412603":"Indústria",
"1413401":"Indústria",
"1413402":"Indústria",
"1413403":"Indústria",
"1414200":"Indústria",
"1421500":"Indústria",
"1422300":"Indústria",
"1510600":"Indústria",
"1521100":"Indústria",
"1529700":"Indústria",
"1531901":"Indústria",
"1531902":"Indústria",
"1532700":"Indústria",
"1533500":"Indústria",
"1539400":"Indústria",
"1540800":"Indústria",
"1610201":"Indústria",
"1610202":"Indústria",
"1610203":"Indústria",
"1610204":"Indústria",
"1610205":"Indústria",
"1621800":"Indústria",
"1622601":"Indústria",
"1622602":"Indústria",
"1622699":"Indústria",
"1623400":"Indústria",
"1629301":"Indústria",
"1629302":"Indústria",
"1710900":"Indústria",
"1721400":"Indústria",
"1722200":"Indústria",
"1731100":"Indústria",
"1732000":"Indústria",
"1733800":"Indústria",
"1741901":"Indústria",
"1741902":"Indústria",
"1742701":"Indústria",
"1742702":"Indústria",
"1742799":"Indústria",
"1749400":"Indústria",
"1811301":"Indústria",
"1811302":"Indústria",
"1812100":"Indústria",
"1813001":"Indústria",
"1813099":"Indústria",
"1821100":"Indústria",
"1822900":"Indústria",
"1822901":"Indústria",
"1822999":"Indústria",
"1830001":"Indústria",
"1830002":"Indústria",
"1830003":"Indústria",
"1910100":"Indústria",
"1921700":"Indústria",
"1922501":"Indústria",
"1922502":"Indústria",
"1922599":"Indústria",
"1931400":"Indústria",
"1932200":"Indústria",
"2011800":"Indústria",
"2012600":"Indústria",
"2013400":"Indústria",
"2013401":"Indústria",
"2013402":"Indústria",
"2014200":"Indústria",
"2019301":"Indústria",
"2019399":"Indústria",
"2021500":"Indústria",
"2022300":"Indústria",
"2029100":"Indústria",
"2031200":"Indústria",
"2032100":"Indústria",
"2033900":"Indústria",
"2040100":"Indústria",
"2051700":"Indústria",
"2052500":"Indústria",
"2061400":"Indústria",
"2062200":"Indústria",
"2063100":"Indústria",
"2071100":"Indústria",
"2072000":"Indústria",
"2073800":"Indústria",
"2091600":"Indústria",
"2092401":"Indústria",
"2092402":"Indústria",
"2092403":"Indústria",
"2093200":"Indústria",
"2094100":"Indústria",
"2099101":"Indústria",
"2099199":"Indústria",
"2110600":"Indústria",
"2121101":"Indústria",
"2121102":"Indústria",
"2121103":"Indústria",
"2122000":"Indústria",
"2123800":"Indústria",
"2211100":"Indústria",
"2212900":"Indústria",
"2219600":"Indústria",
"2221800":"Indústria",
"2222600":"Indústria",
"2223400":"Indústria",
"2229301":"Indústria",
"2229302":"Indústria",
"2229303":"Indústria",
"2229399":"Indústria",
"2311700":"Indústria",
"2312500":"Indústria",
"2319200":"Indústria",
"2320600":"Indústria",
"2330301":"Indústria",
"2330302":"Indústria",
"2330303":"Indústria",
"2330304":"Indústria",
"2330305":"Indústria",
"2330399":"Indústria",
"2341900":"Indústria",
"2342701":"Indústria",
"2342702":"Indústria",
"2349401":"Indústria",
"2349499":"Indústria",
"2391501":"Indústria",
"2391502":"Indústria",
"2391503":"Indústria",
"2392300":"Indústria",
"2399101":"Indústria",
"2399102":"Indústria",
"2399199":"Indústria",
"2411300":"Indústria",
"2412100":"Indústria",
"2421100":"Indústria",
"2422901":"Indústria",
"2422902":"Indústria",
"2423701":"Indústria",
"2423702":"Indústria",
"2424501":"Indústria",
"2424502":"Indústria",
"2431800":"Indústria",
"2439300":"Indústria",
"2441501":"Indústria",
"2441502":"Indústria",
"2442300":"Indústria",
"2443100":"Indústria",
"2449101":"Indústria",
"2449102":"Indústria",
"2449103":"Indústria",
"2449199":"Indústria",
"2451200":"Indústria",
"2452100":"Indústria",
"2511000":"Indústria",
"2512800":"Indústria",
"2513600":"Indústria",
"2521700":"Indústria",
"2522500":"Indústria",
"2531401":"Indústria",
"2531402":"Indústria",
"2532201":"Indústria",
"2532202":"Indústria",
"2539000":"Indústria",
"2539001":"Indústria",
"2539002":"Indústria",
"2541100":"Indústria",
"2542000":"Indústria",
"2543800":"Indústria",
"2550101":"Indústria",
"2550102":"Indústria",
"2591800":"Indústria",
"2592601":"Indústria",
"2592602":"Indústria",
"2593400":"Indústria",
"2599301":"Indústria",
"2599302":"Indústria",
"2599399":"Indústria",
"2610800":"Indústria",
"2621300":"Indústria",
"2622100":"Indústria",
"2631100":"Indústria",
"2632900":"Indústria",
"2640000":"Indústria",
"2651500":"Indústria",
"2652300":"Indústria",
"2660400":"Indústria",
"2670101":"Indústria",
"2670102":"Indústria",
"2680900":"Indústria",
"2710401":"Indústria",
"2710402":"Indústria",
"2710403":"Indústria",
"2721000":"Indústria",
"2722801":"Indústria",
"2722802":"Indústria",
"2731700":"Indústria",
"2732500":"Indústria",
"2733300":"Indústria",
"2740601":"Indústria",
"2740602":"Indústria",
"2751100":"Indústria",
"2759701":"Indústria",
"2759799":"Indústria",
"2790201":"Indústria",
"2790202":"Indústria",
"2790299":"Indústria",
"2811900":"Indústria",
"2812700":"Indústria",
"2813500":"Indústria",
"2814301":"Indústria",
"2814302":"Indústria",
"2815101":"Indústria",
"2815102":"Indústria",
"2821601":"Indústria",
"2821602":"Indústria",
"2822401":"Indústria",
"2822402":"Indústria",
"2823200":"Indústria",
"2824101":"Indústria",
"2824102":"Indústria",
"2825900":"Indústria",
"2829101":"Indústria",
"2829199":"Indústria",
"2831300":"Indústria",
"2832100":"Indústria",
"2833000":"Indústria",
"2840200":"Indústria",
"2851800":"Indústria",
"2852600":"Indústria",
"2853400":"Indústria",
"2854200":"Indústria",
"2861500":"Indústria",
"2862300":"Indústria",
"2863100":"Indústria",
"2864000":"Indústria",
"2865800":"Indústria",
"2866600":"Indústria",
"2869100":"Indústria",
"2910701":"Indústria",
"2910702":"Indústria",
"2910703":"Indústria",
"2920401":"Indústria",
"2920402":"Indústria",
"2930101":"Indústria",
"2930102":"Indústria",
"2930103":"Indústria",
"2941700":"Indústria",
"2942500":"Indústria",
"2943300":"Indústria",
"2944100":"Indústria",
"2945000":"Indústria",
"2949201":"Indústria",
"2949299":"Indústria",
"2950600":"Indústria",
"3011301":"Indústria",
"3011302":"Indústria",
"3012100":"Indústria",
"3031800":"Indústria",
"3032600":"Indústria",
"3041500":"Indústria",
"3042300":"Indústria",
"3050400":"Indústria",
"3091100":"Indústria",
"3091101":"Indústria",
"3091102":"Indústria",
"3092000":"Indústria",
"3099700":"Indústria",
"3101200":"Indústria",
"3102100":"Indústria",
"3103900":"Serviços",
"3104700":"Indústria",
"3211601":"Indústria",
"3211602":"Indústria",
"3211603":"Indústria",
"3212400":"Indústria",
"3220500":"Indústria",
"3230200":"Indústria",
"3240001":"Indústria",
"3240002":"Indústria",
"3240003":"Indústria",
"3240099":"Indústria",
"3250701":"Indústria",
"3250702":"Indústria",
"3250703":"Indústria",
"3250704":"Indústria",
"3250705":"Indústria",
"3250706":"Indústria",
"3250707":"Indústria",
"3250708":"Indústria",
"3250709":"Indústria",
"3291400":"Indústria",
"3292201":"Indústria",
"3292202":"Indústria",
"3299001":"Indústria",
"3299002":"Indústria",
"3299003":"Indústria",
"3299004":"Indústria",
"3299005":"Indústria",
"3299006":"Indústria",
"3299099":"Indústria",
"3311200":"Indústria",
"3312101":"Indústria",
"3312102":"Indústria",
"3312103":"Indústria",
"3312104":"Indústria",
"3313901":"Indústria",
"3313902":"Indústria",
"3313999":"Serviços",
"3314701":"Indústria",
"3314702":"Indústria",
"3314703":"Indústria",
"3314704":"Indústria",
"3314705":"Indústria",
"3314706":"Indústria",
"3314707":"Indústria",
"3314708":"Indústria",
"3314709":"Serviços",
"3314710":"Indústria",
"3314711":"Indústria",
"3314712":"Indústria",
"3314713":"Indústria",
"3314714":"Indústria",
"3314715":"Indústria",
"3314716":"Indústria",
"3314717":"Indústria",
"3314718":"Indústria",
"3314719":"Indústria",
"3314720":"Indústria",
"3314721":"Indústria",
"3314722":"Indústria",
"3314799":"Indústria",
"3315500":"Indústria",
"3316301":"Indústria",
"3316302":"Serviços",
"3317101":"Indústria",
"3317102":"Indústria",
"3319800":"Indústria",
"3321000":"Indústria",
"3329501":"Indústria",
"3329599":"Indústria",
"3511500":"Indústria",
"3511501":"Indústria",
"3511502":"Indústria",
"3512300":"Indústria",
"3513100":"Indústria",
"3514000":"Indústria",
"3520401":"Indústria",
"3520402":"Indústria",
"3530100":"Indústria",
"3600601":"Indústria",
"3600602":"Comércio",
"3701100":"Indústria",
"3702900":"Indústria",
"3811400":"Indústria",
"3812200":"Indústria",
"3821100":"Indústria",
"3822000":"Indústria",
"3831901":"Indústria",
"3831999":"Indústria",
"3832700":"Indústria",
"3839401":"Indústria",
"3839499":"Indústria",
"3900500":"Indústria",
"4110700":"Serviços",
"4120400":"Construção Civil",
"4211101":"Construção Civil",
"4211102":"Construção Civil",
"4212000":"Construção Civil",
"4213800":"Construção Civil",
"4221901":"Construção Civil",
"4221902":"Construção Civil",
"4221903":"Construção Civil",
"4221904":"Construção Civil",
"4221905":"Construção Civil",
"4222701":"Construção Civil",
"4222702":"Construção Civil",
"4223500":"Construção Civil",
"4291000":"Construção Civil",
"4292801":"Construção Civil",
"4292802":"Construção Civil",
"4299501":"Construção Civil",
"4299599":"Construção Civil",
"4311801":"Construção Civil",
"4311802":"Construção Civil",
"4312600":"Construção Civil",
"4313400":"Construção Civil",
"4319300":"Construção Civil",
"4321500":"Construção Civil",
"4322301":"Construção Civil",
"4322302":"Construção Civil",
"4322303":"Construção Civil",
"4329101":"Construção Civil",
"4329102":"Construção Civil",
"4329103":"Construção Civil",
"4329104":"Construção Civil",
"4329105":"Construção Civil",
"4329199":"Construção Civil",
"4330401":"Construção Civil",
"4330402":"Construção Civil",
"4330403":"Construção Civil",
"4330404":"Construção Civil",
"4330405":"Construção Civil",
"4330499":"Construção Civil",
"4391600":"Construção Civil",
"4399101":"Construção Civil",
"4399102":"Construção Civil",
"4399103":"Construção Civil",
"4399104":"Construção Civil",
"4399105":"Construção Civil",
"4399199":"Construção Civil",
"4511101":"Comércio",
"4511102":"Comércio",
"4511103":"Comércio",
"4511104":"Comércio",
"4511105":"Comércio",
"4511106":"Comércio",
"4512901":"Comércio",
"4512902":"Comércio",
"4520001":"Comércio",
"4520002":"Comércio",
"4520003":"Comércio",
"4520004":"Comércio",
"4520005":"Comércio",
"4520006":"Comércio",
"4520007":"Comércio",
"4520008":"Comércio",
"4530701":"Comércio",
"4530702":"Comércio",
"4530703":"Comércio",
"4530704":"Comércio",
"4530705":"Comércio",
"4530706":"Comércio",
"4541201":"Comércio",
"4541202":"Comércio",
"4541203":"Comércio",
"4541204":"Comércio",
"4541205":"Comércio",
"4541206":"Comércio",
"4541207":"Comércio",
"4542101":"Comércio",
"4542102":"Comércio",
"4543900":"Comércio",
"4611700":"Comércio",
"4612500":"Comércio",
"4613300":"Comércio",
"4614100":"Comércio",
"4615000":"Comércio",
"4616800":"Comércio",
"4617600":"Comércio",
"4618401":"Comércio",
"4618402":"Comércio",
"4618403":"Comércio",
"4618499":"Comércio",
"4619200":"Comércio",
"4621400":"Comércio",
"4622200":"Comércio",
"4623101":"Comércio",
"4623102":"Comércio",
"4623103":"Comércio",
"4623104":"Comércio",
"4623105":"Comércio",
"4623106":"Comércio",
"4623107":"Comércio",
"4623108":"Comércio",
"4623109":"Comércio",
"4623199":"Comércio",
"4631100":"Comércio",
"4632001":"Comércio",
"4632002":"Comércio",
"4632003":"Comércio",
"4633801":"Comércio",
"4633802":"Comércio",
"4633803":"Comércio",
"4634601":"Comércio",
"4634602":"Comércio",
"4634603":"Comércio",
"4634699":"Comércio",
"4635401":"Comércio",
"4635402":"Comércio",
"4635403":"Comércio",
"4635499":"Comércio",
"4636201":"Comércio",
"4636202":"Comércio",
"4637101":"Comércio",
"4637102":"Comércio",
"4637103":"Comércio",
"4637104":"Comércio",
"4637105":"Comércio",
"4637106":"Comércio",
"4637107":"Comércio",
"4637199":"Comércio",
"4639701":"Comércio",
"4639702":"Comércio",
"4641901":"Comércio",
"4641902":"Comércio",
"4641903":"Comércio",
"4642701":"Comércio",
"4642702":"Comércio",
"4643501":"Comércio",
"4643502":"Comércio",
"4644301":"Comércio",
"4644302":"Comércio",
"4645101":"Comércio",
"4645102":"Comércio",
"4645103":"Comércio",
"4646001":"Comércio",
"4646002":"Comércio",
"4647801":"Comércio",
"4647802":"Comércio",
"4649401":"Comércio",
"4649402":"Comércio",
"4649403":"Comércio",
"4649404":"Comércio",
"4649405":"Comércio",
"4649406":"Comércio",
"4649407":"Comércio",
"4649408":"Comércio",
"4649409":"Comércio",
"4649410":"Comércio",
"4649499":"Comércio",
"4651601":"Comércio",
"4651602":"Comércio",
"4652400":"Comércio",
"4661300":"Comércio",
"4662100":"Comércio",
"4663000":"Comércio",
"4664800":"Comércio",
"4665600":"Comércio",
"4669901":"Comércio",
"4669999":"Comércio",
"4671100":"Comércio",
"4672900":"Comércio",
"4673700":"Comércio",
"4674500":"Comércio",
"4679601":"Comércio",
"4679602":"Comércio",
"4679603":"Comércio",
"4679604":"Comércio",
"4679699":"Comércio",
"4681801":"Comércio",
"4681802":"Comércio",
"4681803":"Comércio",
"4681804":"Comércio",
"4681805":"Comércio",
"4682600":"Comércio",
"4683400":"Comércio",
"4684201":"Comércio",
"4684202":"Comércio",
"4684299":"Comércio",
"4685100":"Comércio",
"4686901":"Comércio",
"4686902":"Comércio",
"4687701":"Comércio",
"4687702":"Comércio",
"4687703":"Comércio",
"4689301":"Comércio",
"4689302":"Comércio",
"4689399":"Comércio",
"4691500":"Comércio",
"4692300":"Comércio",
"4693100":"Comércio",
"4711301":"Comércio",
"4711302":"Comércio",
"4712100":"Comércio",
"4713001":"Comércio",
"4713002":"Comércio",
"4713003":"Comércio",
"4713004":"Comércio",
"4713005":"Comércio",
"4721101":"Comércio",
"4721102":"Comércio",
"4721103":"Comércio",
"4721104":"Comércio",
"4722901":"Comércio",
"4722902":"Comércio",
"4723700":"Comércio",
"4724500":"Comércio",
"4729601":"Comércio",
"4729602":"Comércio",
"4729699":"Comércio",
"4731800":"Comércio",
"4732600":"Comércio",
"4741500":"Comércio",
"4742300":"Comércio",
"4743100":"Comércio",
"4744001":"Comércio",
"4744002":"Comércio",
"4744003":"Comércio",
"4744004":"Comércio",
"4744005":"Comércio",
"4744006":"Comércio",
"4744099":"Comércio",
"4751200":"Comércio",
"4751201":"Comércio",
"4751202":"Comércio",
"4752100":"Comércio",
"4753900":"Comércio",
"4754701":"Comércio",
"4754702":"Comércio",
"4754703":"Comércio",
"4755501":"Comércio",
"4755502":"Comércio",
"4755503":"Comércio",
"4756300":"Comércio",
"4757100":"Comércio",
"4759801":"Comércio",
"4759899":"Comércio",
"4761001":"Comércio",
"4761002":"Comércio",
"4761003":"Comércio",
"4762800":"Comércio",
"4763601":"Comércio",
"4763602":"Comércio",
"4763603":"Comércio",
"4763604":"Comércio",
"4763605":"Comércio",
"4771701":"Comércio",
"4771702":"Comércio",
"4771703":"Comércio",
"4771704":"Comércio",
"4772500":"Comércio",
"4773300":"Comércio",
"4774100":"Comércio",
"4781400":"Comércio",
"4782201":"Comércio",
"4782202":"Comércio",
"4783101":"Comércio",
"4783102":"Comércio",
"4784900":"Comércio",
"4785701":"Comércio",
"4785799":"Comércio",
"4789001":"Comércio",
"4789002":"Comércio",
"4789003":"Comércio",
"4789004":"Comércio",
"4789005":"Comércio",
"4789006":"Comércio",
"4789007":"Comércio",
"4789008":"Comércio",
"4789009":"Comércio",
"4789099":"Comércio",
"4911600":"Serviços",
"4912401":"Serviços",
"4912402":"Serviços",
"4912403":"Serviços",
"4921301":"Serviços",
"4921302":"Serviços",
"4922101":"Serviços",
"4922102":"Serviços",
"4922103":"Serviços",
"4923001":"Serviços",
"4923002":"Serviços",
"4924800":"Serviços",
"4929901":"Serviços",
"4929902":"Serviços",
"4929903":"Serviços",
"4929904":"Serviços",
"4929999":"Serviços",
"4930201":"Serviços",
"4930202":"Serviços",
"4930203":"Serviços",
"4930204":"Serviços",
"4940000":"Serviços",
"4950700":"Serviços",
"5011401":"Serviços",
"5011402":"Serviços",
"5012201":"Serviços",
"5012202":"Serviços",
"5021101":"Serviços",
"5021102":"Serviços",
"5022001":"Serviços",
"5022002":"Serviços",
"5030101":"Serviços",
"5030102":"Serviços",
"5030103":"Serviços",
"5091201":"Serviços",
"5091202":"Serviços",
"5099801":"Serviços",
"5099899":"Serviços",
"5111100":"Serviços",
"5112901":"Serviços",
"5112999":"Serviços",
"5120000":"Serviços",
"5130700":"Serviços",
"5211701":"Serviços",
"5211702":"Serviços",
"5211799":"Serviços",
"5212500":"Serviços",
"5221400":"Serviços",
"5222200":"Serviços",
"5223100":"Serviços",
"5229001":"Serviços",
"5229002":"Comércio",
"5229099":"Serviços",
"5231101":"Serviços",
"5231102":"Serviços",
"5231103":"Serviços",
"5232000":"Serviços",
"5239700":"Serviços",
"5239701":"Serviços",
"5239799":"Serviços",
"5240101":"Serviços",
"5240199":"Serviços",
"5250801":"Serviços",
"5250802":"Serviços",
"5250803":"Serviços",
"5250804":"Serviços",
"5250805":"Serviços",
"5310501":"Serviços",
"5310502":"Serviços",
"5320201":"Serviços",
"5320202":"Serviços",
"5510801":"Serviços",
"5510802":"Serviços",
"5510803":"Serviços",
"5590601":"Serviços",
"5590602":"Serviços",
"5590603":"Serviços",
"5590699":"Serviços",
"5611201":"Serviços",
"5611202":"Serviços",
"5611203":"Serviços",
"5611204":"Serviços",
"5611205":"Serviços",
"5612100":"Serviços",
"5620101":"Indústria",
"5620102":"Indústria",
"5620103":"Serviços",
"5620104":"Indústria",
"5811500":"Indústria",
"5812300":"Indústria",
"5812301":"Indústria",
"5812302":"Indústria",
"5813100":"Indústria",
"5819100":"Indústria",
"5821200":"Indústria",
"5822100":"Indústria",
"5822101":"Indústria",
"5822102":"Indústria",
"5823900":"Indústria",
"5829800":"Indústria",
"5911101":"Indústria",
"5911102":"Indústria",
"5911199":"Indústria",
"5912001":"Indústria",
"5912002":"Indústria",
"5912099":"Indústria",
"5913800":"Serviços",
"5914600":"Serviços",
"5920100":"Indústria",
"6010100":"Serviços",
"6021700":"Serviços",
"6022501":"Serviços",
"6022502":"Serviços",
"6110801":"Serviços",
"6110802":"Serviços",
"6110803":"Serviços",
"6110899":"Serviços",
"6120501":"Serviços",
"6120502":"Serviços",
"6120599":"Serviços",
"6130200":"Serviços",
"6141800":"Serviços",
"6142600":"Serviços",
"6143400":"Serviços",
"6190601":"Serviços",
"6190602":"Serviços",
"6190699":"Serviços",
"6201500":"Serviços",
"6201501":"Serviços",
"6201502":"Serviços",
"6202300":"Serviços",
"6203100":"Serviços",
"6204000":"Serviços",
"6209100":"Serviços",
"6311900":"Serviços",
"6319400":"Serviços",
"6391700":"Serviços",
"6399200":"Serviços",
"6410700":"Serviços",
"6421200":"Serviços",
"6422100":"Serviços",
"6423900":"Serviços",
"6424701":"Serviços",
"6424702":"Serviços",
"6424703":"Serviços",
"6424704":"Serviços",
"6431000":"Serviços",
"6432800":"Serviços",
"6433600":"Serviços",
"6434400":"Serviços",
"6435201":"Serviços",
"6435202":"Serviços",
"6435203":"Serviços",
"6436100":"Serviços",
"6437900":"Serviços",
"6438701":"Serviços",
"6438799":"Serviços",
"6440900":"Serviços",
"6450600":"Serviços",
"6461100":"Serviços",
"6462000":"Serviços",
"6463800":"Serviços",
"6470101":"Serviços",
"6470102":"Serviços",
"6470103":"Serviços",
"6491300":"Serviços",
"6492100":"Serviços",
"6493000":"Serviços",
"6499901":"Serviços",
"6499902":"Serviços",
"6499903":"Serviços",
"6499904":"Serviços",
"6499905":"Serviços",
"6499999":"Serviços",
"6511101":"Serviços",
"6511102":"Serviços",
"6512000":"Serviços",
"6520100":"Serviços",
"6530800":"Serviços",
"6541300":"Serviços",
"6542100":"Serviços",
"6550200":"Serviços",
"6611801":"Serviços",
"6611802":"Serviços",
"6611803":"Serviços",
"6611804":"Serviços",
"6612601":"Serviços",
"6612602":"Serviços",
"6612603":"Serviços",
"6612604":"Serviços",
"6612605":"Serviços",
"6613400":"Serviços",
"6619301":"Serviços",
"6619302":"Serviços",
"6619303":"Serviços",
"6619304":"Serviços",
"6619305":"Serviços",
"6619399":"Serviços",
"6621501":"Serviços",
"6621502":"Serviços",
"6622300":"Serviços",
"6629100":"Serviços",
"6630400":"Serviços",
"6810201":"Serviços",
"6810202":"Serviços",
"6810203":"Serviços",
"6821801":"Serviços",
"6821802":"Serviços",
"6822600":"Serviços",
"6911701":"Serviços",
"6911702":"Serviços",
"6911703":"Serviços",
"6912500":"Serviços",
"6920601":"Serviços",
"6920602":"Serviços",
"7020400":"Serviços",
"7111100":"Serviços",
"7112000":"Serviços",
"7119701":"Serviços",
"7119702":"Serviços",
"7119703":"Serviços",
"7119704":"Serviços",
"7119799":"Serviços",
"7120100":"Serviços",
"7210000":"Serviços",
"7220700":"Serviços",
"7311400":"Serviços",
"7312200":"Serviços",
"7319001":"Serviços",
"7319002":"Serviços",
"7319003":"Serviços",
"7319004":"Serviços",
"7319099":"Serviços",
"7320300":"Serviços",
"7410201":"Serviços",
"7410202":"Serviços",
"7410203":"Serviços",
"7410299":"Serviços",
"7420001":"Serviços",
"7420002":"Serviços",
"7420003":"Serviços",
"7420004":"Serviços",
"7420005":"Serviços",
"7490101":"Serviços",
"7490102":"Serviços",
"7490103":"Serviços",
"7490104":"Serviços",
"7490105":"Serviços",
"7490199":"Serviços",
"7500100":"Serviços",
"7711000":"Serviços",
"7719501":"Serviços",
"7719502":"Serviços",
"7719599":"Serviços",
"7721700":"Serviços",
"7722500":"Serviços",
"7723300":"Serviços",
"7729201":"Serviços",
"7729202":"Serviços",
"7729203":"Serviços",
"7729299":"Serviços",
"7731400":"Serviços",
"7732201":"Serviços",
"7732202":"Serviços",
"7733100":"Serviços",
"7739001":"Serviços",
"7739002":"Serviços",
"7739003":"Serviços",
"7739099":"Serviços",
"7740300":"Serviços",
"7810800":"Serviços",
"7820500":"Serviços",
"7830200":"Serviços",
"7911200":"Serviços",
"7912100":"Serviços",
"7990200":"Serviços",
"8011101":"Serviços",
"8011102":"Serviços",
"8012900":"Serviços",
"8020000":"Serviços",
"8020001":"Serviços",
"8020002":"Serviços",
"8030700":"Serviços",
"8111700":"Serviços",
"8112500":"Serviços",
"8121400":"Serviços",
"8122200":"Serviços",
"8129000":"Serviços",
"8130300":"Agropecuária",
"8211300":"Serviços",
"8219901":"Serviços",
"8219999":"Serviços",
"8220200":"Serviços",
"8230001":"Serviços",
"8230002":"Serviços",
"8291100":"Serviços",
"8292000":"Serviços",
"8299701":"Serviços",
"8299702":"Serviços",
"8299703":"Serviços",
"8299704":"Serviços",
"8299705":"Serviços",
"8299706":"Serviços",
"8299707":"Serviços",
"8299799":"Serviços",
"8411600":"Administração Pública",
"8412400":"Administração Pública",
"8413200":"Administração Pública",
"8421300":"Administração Pública",
"8422100":"Administração Pública",
"8423000":"Administração Pública",
"8424800":"Administração Pública",
"8425600":"Administração Pública",
"8430200":"Serviços",
"8511200":"Serviços",
"8512100":"Serviços",
"8513900":"Serviços",
"8520100":"Serviços",
"8531700":"Serviços",
"8532500":"Serviços",
"8533300":"Serviços",
"8541400":"Serviços",
"8542200":"Serviços",
"8550301":"Serviços",
"8550302":"Serviços",
"8591100":"Serviços",
"8592901":"Serviços",
"8592902":"Serviços",
"8592903":"Serviços",
"8592999":"Serviços",
"8593700":"Serviços",
"8599601":"Serviços",
"8599602":"Serviços",
"8599603":"Serviços",
"8599604":"Serviços",
"8599605":"Serviços",
"8599699":"Serviços",
"8610101":"Serviços",
"8610102":"Serviços",
"8621601":"Serviços",
"8621602":"Serviços",
"8622400":"Serviços",
"8630501":"Serviços",
"8630502":"Serviços",
"8630503":"Serviços",
"8630504":"Serviços",
"8630505":"Serviços",
"8630506":"Serviços",
"8630507":"Serviços",
"8630599":"Serviços",
"8640201":"Serviços",
"8640202":"Serviços",
"8640203":"Serviços",
"8640204":"Serviços",
"8640205":"Serviços",
"8640206":"Serviços",
"8640207":"Serviços",
"8640208":"Serviços",
"8640209":"Serviços",
"8640210":"Serviços",
"8640211":"Serviços",
"8640212":"Serviços",
"8640213":"Serviços",
"8640214":"Serviços",
"8640299":"Serviços",
"8650001":"Serviços",
"8650002":"Serviços",
"8650003":"Serviços",
"8650004":"Serviços",
"8650005":"Serviços",
"8650006":"Serviços",
"8650007":"Serviços",
"8650099":"Serviços",
"8660700":"Serviços",
"8690901":"Serviços",
"8690902":"Serviços",
"8690903":"Serviços",
"8690904":"Serviços",
"8690999":"Serviços",
"8711501":"Serviços",
"8711502":"Serviços",
"8711503":"Serviços",
"8711504":"Serviços",
"8711505":"Serviços",
"8712300":"Serviços",
"8720401":"Serviços",
"8720499":"Serviços",
"8730101":"Serviços",
"8730102":"Serviços",
"8730199":"Serviços",
"8800600":"Serviços",
"9001901":"Serviços",
"9001902":"Serviços",
"9001903":"Serviços",
"9001904":"Serviços",
"9001905":"Serviços",
"9001906":"Serviços",
"9001999":"Serviços",
"9002701":"Serviços",
"9002702":"Serviços",
"9003500":"Serviços",
"9101500":"Serviços",
"9102301":"Serviços",
"9102302":"Serviços",
"9103100":"Serviços",
"9200301":"Serviços",
"9200302":"Serviços",
"9200399":"Serviços",
"9311500":"Serviços",
"9312300":"Serviços",
"9313100":"Serviços",
"9319101":"Serviços",
"9319199":"Serviços",
"9321200":"Serviços",
"9329801":"Serviços",
"9329802":"Serviços",
"9329803":"Serviços",
"9329804":"Serviços",
"9329899":"Serviços",
"9411100":"Serviços",
"9412000":"Serviços",
"9412001":"Serviços",
"9412099":"Serviços",
"9420100":"Serviços",
"9430800":"Serviços",
"9491000":"Serviços",
"9492800":"Serviços",
"9493600":"Serviços",
"9499500":"Serviços",
"9511800":"Serviços",
"9512600":"Comércio",
"9521500":"Comércio",
"9529101":"Comércio",
"9529102":"Comércio",
"9529103":"Comércio",
"9529104":"Comércio",
"9529105":"Comércio",
"9529106":"Comércio",
"9529199":"Comércio",
"9601701":"Serviços",
"9601702":"Serviços",
"9601703":"Serviços",
"9602501":"Serviços",
"9602502":"Serviços",
"9603301":"Serviços",
"9603302":"Serviços",
"9603303":"Serviços",
"9603304":"Serviços",
"9603305":"Serviços",
"9603399":"Serviços",
"9609201":"Serviços",
"9609202":"Serviços",
"9609203":"Serviços",
"9609204":"Serviços",
"9609205":"Serviços",
"9609206":"Serviços",
"9609207":"Serviços",
"9609208":"Serviços",
"9609299":"Serviços",
"9700500":"Serviços",
"9900800":"Serviços",
"9999999" : "-1",
"-1":"-1"
}

In [14]:
# Dicionário para renomeação de colunas
dict_columns = {
"BAIRRO_SP"                : "bairro_sp",
"BAIRRO FORT"              : "bairro_fortaleza",
"BAIRRO_RJ"                : "bairro_rj",
"CLAS CNAE 20"             : "cnae_2_classe",
"CLAS CNAE 95"             : "cnae_1_classe",
"DISTRITO_SP"              : "distrito_sp",
"EST CLT OUT"              : "qtd_vinc_clt",
"ESTOQUE"                  : "qtd_vinc_ativos",
"ESTOQUE ESTA"             : "qtd_vinc_estat",
"IND ATIV ANO"             : "ind_ativ_ano",
"IND CEI VINC"             : "ind_cei_vinculado",
"IND PAT"                  : "ind_pat",
"IND RAIS NEG"             : "ind_rais_negativa",
"IND SIMPLES"              : "ind_simples",
"MUNICIPIO"                : "id_municipio",
"NAT JURIDICA"             : "nat_jur",
"REG ADM DF"               : "regiao_adm_df",
"SB CLAS 20"               : "cnae_2_subclasse",
"TAMESTAB"                 : "tam_estab",
"TIPO ESTBL"               : "tipo_estab",
"UF"                       : "sigla_uf",
"IBGE Subsetor"            : "subsetor_ibge",
"Bairros SP"               : "bairro_sp",
"Bairros Fortaleza"        : "bairro_fortaleza",
"Bairros RJ"               : "bairro_rj",
"CNAE 2.0 Classe"          : "cnae_2_classe",
"CNAE 95 Classe"           : "cnae_1_classe",
"Distritos SP"             : "distrito_sp",
"Qtd Vínculos CLT"         : "qtd_vinc_clt",
"Qtd Vínculos Ativos"      : "qtd_vinc_ativos",
"Qtd Vínculos Estatutários": "qtd_vinc_estat",
"Ind Atividade Ano"        : "ind_ativ_ano",
"Ind CEI Vinculado"        : "ind_cei_vinculado",
"Ind Estab Participa PAT"  : "ind_pat",
"Ind Rais Negativa"        : "ind_rais_negativa",
"Ind Simples"              : "ind_simples",
"Município"                : "id_municipio",
"Natureza Jurídica"        : "nat_jur",
"Regiões Adm DF"           : "regiao_adm_df",
"CNAE 2.0 Subclasse"       : "cnae_2_subclasse",
"Tamanho Estabelecimento"  : "tam_estab",
"Tipo Estab"               : "tipo_estab",
"Tipo Estab.1"             : "ctg_tipo_estab",
"UF"                       : "sigla_uf",
"IBGE Subsetor"            : "subsetor_ibge",
"CEP Estab"                : "cep",
"CEP Estb"                 : "cep" 
}

In [15]:
# Função para tratar o dataframe
def trat_rais_estab(caminho_arquivo, ano, ds_owner):
    print(f"Iniciando o tratamento básico dos dados de {ano}...\n")
    # Leitura do arquivo .txt em um dataframe
    df = pd.read_csv(caminho_arquivo, delimiter=";", encoding="cp1252")
    # Renomeação de colunas via dicionário
    df.rename(columns=dict_columns, inplace=True)
    
    # Criação de colunas
    df["ano"] = int(ano)  # ano da pasta em questão
    df["curr_date"] = str(date.today())  # data do dia atual
    df["ds_owner"] = ds_owner  # responsável
    df["sigla_uf"] = df["id_municipio"]//10000
    if int(ano)<2014:
        df["cep"] = -1
        df["subsetor_ibge"] = -1
    else:
       df.drop("ctg_tipo_estab", axis=1)

    # Remoção de espaços no início e final dos campos
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    #df = df.applymap(lambda x: -1 if isinstance(x, str) and "{ñ" in x else x) ##comando trocado por questões de perfomance
    #df = df.applymap(lambda x: "-1" if isinstance(x, str) and "0-1" in x else x) ##comando trocado por questões de perfomance
    df.replace({"0000-1"   : "-1",
                "000-1"    : "-1",
                "00-1"     : "-1",
                "0-1"      : "-1",
                "{ñ"       : "-1",
                "{ñ class}": "-1",
                "{ñ c"     : "-1",
                "{ñ clas"  : "-1",
                "{ñ cl"    : "-1"
                }, inplace=True)
    
    # Conversão para int das colunas int que podem ter sido lidas de outra forma
    colunas_para_int = ["bairro_sp", "bairro_fortaleza", "bairro_rj", "cnae_2_classe", "cnae_1_classe", "distrito_sp", 
                    "qtd_vinc_clt", "qtd_vinc_ativos", "qtd_vinc_estat", "ind_ativ_ano", "ind_cei_vinculado", 
                    "ind_pat", "ind_rais_negativa", "ind_simples", "id_municipio", "nat_jur", "regiao_adm_df",
                    "cnae_2_subclasse", "tam_estab", "tipo_estab", "sigla_uf", "subsetor_ibge", "cep"]
    df[colunas_para_int] = df[colunas_para_int].fillna(-1)

    # Criando coluna do setor de acordo com o padrão do sebrae
    df['cnae_2_subclasse']  = df['cnae_2_subclasse'].astype(str)
    df['setor_sebrae'] = df['cnae_2_subclasse'].map(pad_sebrae_setor)

    # Criando coluna de padrão sebrae de empresa
    df['nat_jur'] = df['nat_jur'].astype(str)
    df['subc_len'] = df['cnae_2_subclasse'].map(len)
    #como sabemos que as observações com 6 números são observações que começam com o 0, mas que por serem dados do tipo "int" não foram lidas da forma correta, vamos criar uma coluna 
    # de 0s para as observações com erro e concatenar tudo depois porque o dicionário está padronizado para 7 dígitos.
    conditions = [
        (df['subc_len'] == 6),
        (df['subc_len'] >= 7)
       ]
    values = ['0', '']
    df['subc_comp'] = np.select(conditions, values)
    df['cnae_2_subclasse'] = df['subc_comp'] + df['cnae_2_subclasse']
    #tudo certo, agora vamos dropar as colunas criadas que não iremos utilizar mais.
    df.drop(axis=1, columns=['subc_len','subc_comp'], inplace=True)
    #feito isso, vamos criar uma coluna concatenada da subclasse com a natureza jurídica para criar posteriormente o padrão sebrae 
    df['concat'] = df['cnae_2_subclasse'].str[:2] + df['nat_jur']
    df['padrao_sebrae'] = df['concat'].map(pad_sebrae_empresa)
    df.loc[df['nat_jur'].str.contains("3123", na=False), 'padrao_sebrae'] = "0"
    # Criando coluna de setor_ibge e porte_sebrae
    df['cnae_1_classe']  = df['cnae_1_classe'].astype(str)
    df['cnae1_len'] = df['cnae_1_classe'].map(len)
    #como sabemos que as observações com 4 números são observações que começam com o 0, mas que por serem dados do tipo "int" não foram lidas da forma correta, vamos criar uma coluna 
    # de 0s para as observações com erro e concatenar tudo depois porque o dicionário está padronizado para 7 dígitos.
    conditions = [
    (df['cnae1_len'] == 4),
    (df['cnae1_len'] >= 5),
    (df['cnae1_len'] == 2)
    ]
    values = ['0', '', '']
    df['cnae1_comp'] = np.select(conditions, values)
    df['cnae_1_classe'] = df['cnae1_comp'] + df['cnae_1_classe']
    df['tam_estab'] = df['tam_estab'].astype(str)
    df['setor_ibge'] = df['cnae_1_classe'].map(dpara_cnae1_setoribge)  
    df['concat'] = df['setor_ibge'] + df['tam_estab']
    df['porte_sebrae'] = df['concat'].map(pad_sebrae_porte)
    
    df.drop(axis=1, columns=['concat', 'cnae1_len', 'cnae1_comp'], inplace=True)


    colunas_para_int = ["bairro_sp", "bairro_fortaleza", "bairro_rj", "cnae_2_classe", "cnae_1_classe", "distrito_sp", 
                    "qtd_vinc_clt", "qtd_vinc_ativos", "qtd_vinc_estat", "ind_ativ_ano", "ind_cei_vinculado", 
                    "ind_pat", "ind_rais_negativa", "ind_simples", "id_municipio", "nat_jur", "regiao_adm_df",
                    "cnae_2_subclasse", "tam_estab", "tipo_estab", "sigla_uf", "subsetor_ibge", "cep", "padrao_sebrae"
                    ]
    df[colunas_para_int] = df[colunas_para_int].astype(int)
    
    # Reordenando colunas
    df = df[["ds_owner", "curr_date", 
             "ano", "sigla_uf", "id_municipio", "qtd_vinc_ativos", "qtd_vinc_clt", "qtd_vinc_estat",
             "nat_jur", "tam_estab", "tipo_estab",
             "ind_ativ_ano", "ind_cei_vinculado", "ind_pat", "ind_rais_negativa", "ind_simples",
             "cnae_1_classe", "cnae_2_classe", "cnae_2_subclasse", "subsetor_ibge",
             "cep", "bairro_sp", "bairro_fortaleza", "bairro_rj", "distrito_sp", "regiao_adm_df",
             "padrao_sebrae", "porte_sebrae", "setor_sebrae", "setor_ibge"            
             ]]
    
    for col in df.columns:
        if df[col].dtype == int:
            df[col].fillna(-1, inplace=True)
        elif df[col].dtype == object:
            df[col].fillna("-1", inplace=True)
    
    print(f"Show, tratamento básico dos dados de {ano} finalizados...")
    return df

In [16]:
# Função para exportar dataframe para arquivo .parquet
def exportar_parquet(df, ano):
    nome_arquivo = f"RAIS_ESTAB_PUB_{ano}.parquet"
    df.to_parquet(os.path.join(base, nome_arquivo))
    print(f"Arquivo exportado em parquet para {nome_arquivo} com sucesso!\n")

In [17]:
# Função para remover arquivos .txt
def remover_arquivo_txt(caminho_arquivo):
    if caminho_arquivo.endswith(".txt"):
        os.remove(caminho_arquivo)
        print(f"\nArquivo txt removido do {caminho_arquivo} com sucesso!\n")

In [18]:
# Função para gerar report do pandas profiling
def gerar_reports_ae(df, ano):
    qt_estabel = df.shape[0]
    qt_estabel_format = format(qt_estabel, ",").replace(",", ".")
    qt_vinc = df["qtd_vinc_ativos"].sum()
    qt_vinc_format = format(qt_vinc, ",").replace(",", ".")

    print(f"O shape do arquivo final é o seguinte: {df.shape}.\nCom base nisso, podemos assumir que, em {ano}, {qt_estabel_format} estabelecimentos declararam a RAIS.\nAlém disso, estes estabelecimentos declararam ter {qt_vinc_format} vínculos empregatícios ativos.")
    print(f"\nA relação de estabelecimentos e vínculos por UF foi a seguinte:\n")
    print("Vínculos:\n\n", df.groupby("sigla_uf")["qtd_vinc_ativos"].sum(), "\n\n")
    print("Estabelecimentos:\n\n", df.groupby("sigla_uf")["id_municipio"].count(), "\n\n")
    
    lista = ['padrao_sebrae', 'porte_sebrae', 'setor_sebrae', 'setor_ibge']
    for i in lista:
        print(f'\nRelatório de nulos das colunas com padrão sebrae do arquivo de {ano}:\n', df[i].isnull().sum())
 

    print(f'\nRelatório de -1s das colunas no geral do arquivo de {ano}:\n')
    contagem_valores = {}
    for col in df.columns:
        if df[col].dtype == int:
            contagem = (df[col] == -1).sum()
            contagem_valores[f'{col}_int'] = contagem
        elif df[col].dtype == object:
            contagem = (df[col] == "-1").sum()
            contagem_valores[f'{col}_str'] = contagem
    for col, contagem in contagem_valores.items():
        print(f'{col}: {contagem}')
    #print("Para um relatório mais detalhado, vamos gerar um arquivo Pandas Profiling em html. Iniciando o procedimento...")
    #report = ProfileReport(df, title=f"Pandas Profiling Report - RAIS Estab. de {ano}")
    #report.to_file(caminho_reports + f"\pd_prof_report_rais_estab_{ano}.html")
    print("Report gerado com sucesso!\n")

In [ ]:
with open('../config/config.json') as config_file:
    config = json.load(config_file)
mongodb_config = config['mongodb']

In [19]:
def upload_mongodb(df, arquivo, ano):
    
    print("------- Conectando com o mongodb ------------------")     

    client = pymongo.MongoClient(f"mongodb://{mongodb_config['user_name']}:{urllib.parse.quote_plus(mongodb_config['password'])}@{mongodb_config['host']}:{mongodb_config['port']}/{mongodb_config['db_name']}")
    db = client[mongodb_config['db_name']]
    
    type(client)

    print(client.list_database_names())
    print("------- Carregando os dados para o mongodb ------------------")    
    print(f"Lendo o  arquivo: {arquivo}, do ano de {ano} e convertendo para dict pandas...\n")
    data = df.to_dict(orient="records")
    db = client[mongodb_config['db_name']]
    col = db[arquivo]

    print("\nIniciando a gravação no mongodb...\n ")
    col.insert_many(data)
    print(f"{arquivo} de {ano} gravado com sucesso!")
    print("\n------- fim  da carga dos dados para o mongodb ------------------\n")       

In [20]:
# Função para excluir arquivo .parquet do armazenamento local
def remover_arquivo_parquet(caminho_arquivo):
   os.remove(caminho_arquivo)
   print(f'\nArquivo parquet removido do {caminho_arquivo} com sucesso!\n')

In [ ]:
with open('../config/config.json') as config_file:
    config = json.load(config_file)
mongodb_config = config['mongodb']
postgres_config = config['postgres']

In [21]:
def transfer_data_batch(ano, batch_size):
    print("Iniciando a transferência do MONGODB para o PostgreSQL...")

    # Conectando com o MongoDB
    print("Conectando ao MongoDB...")
    client = pymongo.MongoClient(f"mongodb://{mongodb_config['user_name']}:{urllib.parse.quote_plus(mongodb_config['password'])}@{mongodb_config['host']}:{mongodb_config['port']}/{mongodb_config['db_name']}")
    db = client[mongodb_config['db_name']]
    collection = db["RAIS_ESTAB_PUB"]
    print("Conexão estabelecida com sucesso!")

    # Conectando com o PostgreSQL
    print("Conectando ao PostgreSQL...")
    conn = p.connect(
        host=postgres_config['host'],
        port=postgres_config['port'],
        dbname=postgres_config['db_name'],
        user=postgres_config['user_name'],
        password=postgres_config['password']
    )
    cur = conn.cursor()
    print("Conexão estabelecida com sucesso!")

    # Campos para puxar do MongoDB
    fields = [
        "ano", "sigla_uf", "id_municipio", "qtd_vinc_ativos", "qtd_vinc_clt", "qtd_vinc_estat", 
        "nat_jur", "tam_estab", "tipo_estab", "ind_ativ_ano", "ind_cei_vinculado", "ind_pat", 
        "ind_rais_negativa", "ind_simples", "cnae_1_classe", "cnae_2_classe", "cnae_2_subclasse", 
        "subsetor_ibge", "cep", "bairro_sp", "bairro_fortaleza", "bairro_rj", "distrito_sp", 
        "regiao_adm_df", "padrao_sebrae", "porte_sebrae", "setor_sebrae", "setor_ibge", 
        "ds_owner", "curr_date"
    ]

    # Determinando a partição correta com base no ano
    if 2010 <= ano <= 2012:
        partition = "tb_rais_estab_pub_2010_2012"
    elif 2013 <= ano <= 2015:
        partition = "tb_rais_estab_pub_2013_2015"
    elif 2016 <= ano <= 2018:
        partition = "tb_rais_estab_pub_2016_2018"
    elif 2019 <= ano <= 2021:
        partition = "tb_rais_estab_pub_2019_2021"
    elif 2022 <= ano <= 2024:
        partition = "tb_rais_estab_pub_2022_2024"
    elif 2025 <= ano <= 2027:
        partition = "tb_rais_estab_pub_2025_2027"
    elif 2028 <= ano <= 2030:
        partition = "tb_rais_estab_pub_2028_2030"
    else:
        print(f"Ano {ano} não corresponde a nenhuma partição existente.")
        return

    print(f"Transferindo dados para a partição {partition}...")

    # Iterando sobre os documentos na collection do MongoDB
    batch = []
    for doc in collection.find({"ano": ano}, {field: 1 for field in fields}):  # Filtrar pelo ano
        # Preparando os dados para inserção no PostgreSQL
        data = [doc.get(field) for field in fields]
        batch.append(data)

        # Se o tamanho do lote atingir batch_size, inserir o lote no PostgreSQL
        if len(batch) == batch_size:
            p.extras.execute_batch(cur, f"INSERT INTO stg_rais.{partition} (" + ', '.join(fields) + ") VALUES (" + ', '.join(['%s'] * len(fields)) + ")", batch)
            batch = []

    # Inserir quaisquer documentos restantes que não foram inseridos
    if batch:
        p.extras.execute_batch(cur, f"INSERT INTO stg_rais.{partition} (" + ', '.join(fields) + ") VALUES (" + ', '.join(['%s'] * len(fields)) + ")", batch)

    # Commit as alterações e fechar a conexão
    conn.commit()
    cur.close()
    conn.close()

    print("Transferência de dados concluída.")

# **5. RODANDO**:

In [22]:
ds_owner = input("Digite seu 1º e 2º nome, ex: Marcilio Duarte")
inicio = datetime.now()
print("\nO código começou a rodar em:\n", inicio, "\n")
for ano in anos:
    print(f"Começando o loop para o ano de {ano}...\n")
    # Portal principal da RAIS
    ftp = FTP("ftp.mtps.gov.br")  ## SERVIDOR FTP
    link = r"ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/"
    # Comando que conecta a aba correta dentro do site ftp
    cmd = r"pdet/microdados/RAIS/" 
    ftp.login()
    # Navegando até o diretório do ano
    ftp.cwd(cmd + ano)
    
    # Obtendo a lista de arquivos no diretório do ano
    arquivos = ftp.nlst()
    for arquivo in arquivos:
        try:
            if (arquivo=="RAIS_ESTAB_PUB.7z") | (arquivo.startswith("ESTB")) | (arquivo.startswith("Estb")):
                # Checando se o arquivo já existe no diretório local
                if not checadir(base, arquivo):
                    # Baixando o arquivo
                    baixar_arquivo(ftp, arquivo, base)
                    
                    # Extraindo o arquivo
                    caminho_arquivo = os.path.join(base, arquivo)
                    extrair_arquivo_7z_txt(caminho_arquivo)
                    
                    # Removendo o arquivo zipado
                    remover_arquivo_7z(caminho_arquivo)
                    
                    # Tratando o dataframe
                    caminho_arquivo = caminho_arquivo[:-2] + "txt"
                    df = trat_rais_estab(caminho_arquivo, ano, ds_owner=ds_owner)
                    
                    # Removendo o arquivo .txt
                    remover_arquivo_txt(caminho_arquivo)
                    
                    # Exportando para .parquet
                    #exportar_parquet(df, ano)

                    # Gerando reports de AE
                    gerar_reports_ae(df, ano)

                    # Importando para mongodb
                    upload_mongodb(df, arquivo="RAIS_ESTAB_PUB", ano=ano)

                    #caminho_arquivo = caminho_arquivo[:-2] + "parquet"
                    #remover_arquivo_parquet(caminho_arquivo)

                else:
                    print(f"O arquivo {arquivo} já existe na pasta. Indo para o próximo!\n")
            else:
                print(f"Não iremos trabalhar com o arquivo {arquivo}...")
        except Exception as e:
            print(f"Um erro ocorreu na execução do ano {ano} e no arquivo {arquivo}. Iremos continuar a partir dos próximos anos, mas o erro em questão foi: {e}")
            continue
        
    # Voltando ao diretório raiz para a próxima iteração
    continue
fim = datetime.now()
print("\nO código terminou de rodar em:\n", fim, "\n")
print("\nO tempo total foi:\n", fim - inicio, "\n") 



O código começou a rodar em:
 2023-12-11 16:57:40.337756 

Começando o loop para o ano de 2021...


Baixando o arquivo do ftp...

Arquivo RAIS_ESTAB_PUB.7z baixado com sucesso!

Arquivo txt extraído do 7zip para o caminho: E:\Users\c2705\PROJETOS\AUTOMATIZACAO\git\automatizacao-etls\RAIS\ESTABELECIMENTOS\04_temp_data\0404_TESTE_DOWNLOAD_AUTOMÁTICO\RAIS_ESTAB_PUB.7z com sucesso!

Arquivo 7zip removido do caminho: E:\Users\c2705\PROJETOS\AUTOMATIZACAO\git\automatizacao-etls\RAIS\ESTABELECIMENTOS\04_temp_data\0404_TESTE_DOWNLOAD_AUTOMÁTICO\RAIS_ESTAB_PUB.7z com sucesso!

Iniciando o tratamento básico dos dados de 2021...



C:\Users\c2705\AppData\Local\Temp\9\ipykernel_29380\1991265849.py:5: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=";", encoding="cp1252")


Show, tratamento básico dos dados de 2021 finalizados...

Arquivo txt removido do E:\Users\c2705\PROJETOS\AUTOMATIZACAO\git\automatizacao-etls\RAIS\ESTABELECIMENTOS\04_temp_data\0404_TESTE_DOWNLOAD_AUTOMÁTICO\RAIS_ESTAB_PUB.txt com sucesso!

O shape do arquivo final é o seguinte: (8472949, 30).
Com base nisso, podemos assumir que, em 2021, 8.472.949 estabelecimentos declararam a RAIS.
Além disso, estes estabelecimentos declararam ter 48.728.871 vínculos empregatícios ativos.

A relação de estabelecimentos e vínculos por UF foi a seguinte:

Vínculos:

 sigla_uf
11      312314
12      142490
13      653783
14      111208
15     1167171
16      125710
17      296033
21      818989
22      459729
23     1528938
24      615645
25      676391
26     1663000
27      522990
28      392070
29     2353198
31     5148435
32      941986
33     3938871
35    13848376
41     3257533
42     2504231
43     2960685
50      699968
51      919419
52     1585167
53     1084527
99          14
Name: qtd_vin

In [26]:
inicio_geral = datetime.now()
print('\nPassando agora para a transferência do MongoDB para o PostgreSQL no Dbeaver...\n')
for ano in anos:
    inicio_ano = datetime.now()
    print(f'Iniciando loop com o ano de {ano} em {inicio_ano}...\n')
    transfer_data_batch(
    ano=int(ano), batch_size=1000)
    fim_ano = datetime.now()
    tempo_total_ano = fim_ano-inicio_ano
    print(f'\nLoop de {ano} finalizado com sucesso. O tempo total foi {tempo_total_ano}.\n')
fim_geral = datetime.now()
tempo_total_geral = fim_geral - inicio_geral
print(f'A execução completa da carga durou {tempo_total_geral}.')

Iniciando loop com o ano de 2019 em 2023-09-27 15:40:53.682448...

Iniciando a transferência do MONGODB para o PostgreSQL...
Conectando ao MongoDB...
Conexão estabelecida com sucesso!
Conectando ao PostgreSQL...
Conexão estabelecida com sucesso!
Transferindo dados para a partição tb_rais_estab_pub_2019_2021...
Transferência de dados concluída.

Loop de 2019 finalizado com sucesso. O tempo total foi 0:21:02.049685.

Iniciando loop com o ano de 2020 em 2023-09-27 16:01:55.732133...

Iniciando a transferência do MONGODB para o PostgreSQL...
Conectando ao MongoDB...
Conexão estabelecida com sucesso!
Conectando ao PostgreSQL...
Conexão estabelecida com sucesso!
Transferindo dados para a partição tb_rais_estab_pub_2019_2021...
Transferência de dados concluída.

Loop de 2020 finalizado com sucesso. O tempo total foi 0:20:54.936994.

A execução completa da carga durou 0:41:56.986679.
